In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import backend.config as config
import dataprocessing
import surfacemodelclass as sf
import util

/Users/maximilianrapp/PycharmProjects/nextgenbike/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Data preparation
extract current data from the database and prepare for processing

In [2]:
CLASSES = config.classes

In [3]:
raw_data = '[{"time": "2023-11-05 17:35:18.484000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 15.0, "acceleration_z": -53.0, "gyroscope_x": 51.0, "gyroscope_y": 54.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:18.537000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 13.0, "acceleration_z": -56.0, "gyroscope_x": 16.0, "gyroscope_y": 22.0, "gyroscope_z": 208.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:18.589000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 6.0, "acceleration_z": -57.0, "gyroscope_x": -20.0, "gyroscope_y": 78.0, "gyroscope_z": -76.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:18.641000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 8.0, "acceleration_z": -54.0, "gyroscope_x": 1.0, "gyroscope_y": -11.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:24.245000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -10.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:24.297000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 7.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:24.349000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -57.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:24.401000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -56.0, "gyroscope_x": -10.0, "gyroscope_y": 59.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:30.090000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -58.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:30.142000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -57.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:30.194000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:30.246000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -10.0, "gyroscope_y": 62.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:36.108000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 5.0, "acceleration_z": -60.0, "gyroscope_x": -11.0, "gyroscope_y": 70.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:36.160000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -8.0, "gyroscope_y": 63.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:36.212000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -58.0, "gyroscope_x": -11.0, "gyroscope_y": 60.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:36.264000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -58.0, "gyroscope_x": -11.0, "gyroscope_y": 66.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:42.224000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -57.0, "gyroscope_x": -12.0, "gyroscope_y": 61.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:42.276000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 62.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:42.328000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 61.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:42.380000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -58.0, "gyroscope_x": -11.0, "gyroscope_y": 60.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:48.209000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -60.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:48.261000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -10.0, "gyroscope_y": 69.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:48.313000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -57.0, "gyroscope_x": -12.0, "gyroscope_y": 68.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:48.365000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 7.0, "acceleration_z": -57.0, "gyroscope_x": -13.0, "gyroscope_y": 61.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:54.143000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -11.0, "gyroscope_y": 69.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:54.195000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -58.0, "gyroscope_x": -9.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:54.247000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 61.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:35:54.299000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -59.0, "gyroscope_x": -11.0, "gyroscope_y": 67.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:00.084000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -56.0, "gyroscope_x": -9.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:00.136000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -57.0, "gyroscope_x": -12.0, "gyroscope_y": 67.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:00.188000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -58.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:00.240000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -9.0, "gyroscope_y": 63.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:06.029000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -60.0, "gyroscope_x": -12.0, "gyroscope_y": 57.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:06.081000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 67.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:06.133000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -57.0, "gyroscope_x": -11.0, "gyroscope_y": 67.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:06.185000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 7.0, "acceleration_z": -59.0, "gyroscope_x": -12.0, "gyroscope_y": 61.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:11.959000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -58.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:12.011000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -58.0, "gyroscope_x": -8.0, "gyroscope_y": 64.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:12.063000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -59.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:12.115000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -60.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:17.880000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -10.0, "gyroscope_y": 61.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:17.932000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -59.0, "gyroscope_x": -13.0, "gyroscope_y": 68.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:17.984000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 10.0, "acceleration_z": -58.0, "gyroscope_x": -8.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-05 17:36:18.036000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -9.0, "gyroscope_y": 59.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:01.044000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -2.0, "acceleration_z": -78.0, "gyroscope_x": -19.0, "gyroscope_y": 96.0, "gyroscope_z": -44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:06.251000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -2.0, "acceleration_z": -88.0, "gyroscope_x": -10.0, "gyroscope_y": 62.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:06.375000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -3.0, "acceleration_z": -88.0, "gyroscope_x": -10.0, "gyroscope_y": 62.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:06.550000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": -2.0, "acceleration_z": -88.0, "gyroscope_x": -8.0, "gyroscope_y": 68.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:06.665000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": -3.0, "acceleration_z": -89.0, "gyroscope_x": -10.0, "gyroscope_y": 66.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:12.612000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": -1.0, "acceleration_z": -89.0, "gyroscope_x": -9.0, "gyroscope_y": 63.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:12.863000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": -2.0, "acceleration_z": -89.0, "gyroscope_x": -11.0, "gyroscope_y": 62.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:12.985000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": -3.0, "acceleration_z": -90.0, "gyroscope_x": -10.0, "gyroscope_y": 64.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:50:13.103000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -2.0, "acceleration_z": -89.0, "gyroscope_x": -7.0, "gyroscope_y": 64.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:06.173000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 13.0, "acceleration_z": -60.0, "gyroscope_x": -3.0, "gyroscope_y": 44.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:11.298000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -62.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": 5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:11.350000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -62.0, "gyroscope_x": -10.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:11.402000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -62.0, "gyroscope_x": -9.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:11.454000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 9.0, "acceleration_z": -61.0, "gyroscope_x": -9.0, "gyroscope_y": 59.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:16.908000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 12.0, "acceleration_z": -63.0, "gyroscope_x": -4.0, "gyroscope_y": 3.0, "gyroscope_z": 13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:16.960000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 12.0, "acceleration_z": -62.0, "gyroscope_x": -5.0, "gyroscope_y": 42.0, "gyroscope_z": 8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:17.012000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 11.0, "acceleration_z": -65.0, "gyroscope_x": -15.0, "gyroscope_y": 119.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:17.064000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 14.0, "acceleration_z": -63.0, "gyroscope_x": -8.0, "gyroscope_y": 56.0, "gyroscope_z": 23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:22.515000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -61.0, "gyroscope_x": -8.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:22.567000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 7.0, "acceleration_z": -62.0, "gyroscope_x": -9.0, "gyroscope_y": 63.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:22.619000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -63.0, "gyroscope_x": -9.0, "gyroscope_y": 60.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:22.671000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 7.0, "acceleration_z": -62.0, "gyroscope_x": -7.0, "gyroscope_y": 62.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:28.133000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 14.0, "acceleration_z": -60.0, "gyroscope_x": -9.0, "gyroscope_y": 53.0, "gyroscope_z": 13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:28.185000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 14.0, "acceleration_z": -61.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:28.237000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 15.0, "acceleration_z": -62.0, "gyroscope_x": -12.0, "gyroscope_y": 70.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:28.289000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 14.0, "acceleration_z": -62.0, "gyroscope_x": -8.0, "gyroscope_y": 60.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:34.379000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 7.0, "acceleration_z": -62.0, "gyroscope_x": -10.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:34.431000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 7.0, "acceleration_z": -62.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:34.483000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 8.0, "acceleration_z": -63.0, "gyroscope_x": -13.0, "gyroscope_y": 60.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 12:52:34.535000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 8.0, "acceleration_z": -62.0, "gyroscope_x": -11.0, "gyroscope_y": 63.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:07.237000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 11.0, "acceleration_z": -36.0, "gyroscope_x": 11.0, "gyroscope_y": 40.0, "gyroscope_z": 77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:12.408000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 6.0, "acceleration_z": -39.0, "gyroscope_x": -10.0, "gyroscope_y": 66.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:12.460000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 6.0, "acceleration_z": -38.0, "gyroscope_x": -11.0, "gyroscope_y": 63.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:12.512000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 7.0, "acceleration_z": -38.0, "gyroscope_x": -8.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:12.564000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 7.0, "acceleration_z": -38.0, "gyroscope_x": -7.0, "gyroscope_y": 61.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:18.014000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -39.0, "gyroscope_x": -9.0, "gyroscope_y": 60.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:18.066000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -38.0, "gyroscope_x": -11.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:18.118000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 7.0, "acceleration_z": -37.0, "gyroscope_x": -13.0, "gyroscope_y": 65.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:18.170000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -38.0, "gyroscope_x": -8.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:55.265000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 17.0, "acceleration_z": -37.0, "gyroscope_x": -5.0, "gyroscope_y": 10.0, "gyroscope_z": 109.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:55.317000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 10.0, "acceleration_z": -38.0, "gyroscope_x": -9.0, "gyroscope_y": 57.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:55.369000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 15.0, "acceleration_z": -39.0, "gyroscope_x": -6.0, "gyroscope_y": 55.0, "gyroscope_z": 114.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:37:55.421000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 11.0, "acceleration_z": -40.0, "gyroscope_x": -7.0, "gyroscope_y": 65.0, "gyroscope_z": 28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:00.919000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 7.0, "acceleration_z": -38.0, "gyroscope_x": -9.0, "gyroscope_y": 63.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:00.971000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -38.0, "gyroscope_x": -10.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:01.023000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -37.0, "gyroscope_x": -12.0, "gyroscope_y": 67.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:01.075000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 8.0, "acceleration_z": -39.0, "gyroscope_x": -12.0, "gyroscope_y": 61.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:06.538000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -40.0, "gyroscope_x": -10.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:06.590000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -37.0, "gyroscope_x": -9.0, "gyroscope_y": 62.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:06.642000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 8.0, "acceleration_z": -41.0, "gyroscope_x": -11.0, "gyroscope_y": 62.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:38:06.694000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -40.0, "gyroscope_x": -12.0, "gyroscope_y": 68.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:42:57.119000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 6.0, "acceleration_z": -38.0, "gyroscope_x": -12.0, "gyroscope_y": 32.0, "gyroscope_z": 33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:42:57.171000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 10.0, "acceleration_z": -38.0, "gyroscope_x": -7.0, "gyroscope_y": 37.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:42:57.223000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 6.0, "acceleration_z": -38.0, "gyroscope_x": -12.0, "gyroscope_y": 85.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:42:57.275000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 8.0, "acceleration_z": -38.0, "gyroscope_x": -13.0, "gyroscope_y": 84.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:43:02.723000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 7.0, "acceleration_z": -41.0, "gyroscope_x": -10.0, "gyroscope_y": 62.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:43:02.775000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -41.0, "gyroscope_x": -11.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:43:02.827000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -40.0, "gyroscope_x": -14.0, "gyroscope_y": 66.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:43:02.879000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 8.0, "acceleration_z": -41.0, "gyroscope_x": -10.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:43:59.973000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 1.0, "acceleration_z": -44.0, "gyroscope_x": -133.0, "gyroscope_y": -33.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:05.110000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": -7.0, "acceleration_z": -50.0, "gyroscope_x": -10.0, "gyroscope_y": 45.0, "gyroscope_z": -27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:05.162000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -5.0, "acceleration_z": -52.0, "gyroscope_x": -10.0, "gyroscope_y": 82.0, "gyroscope_z": 36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:05.214000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": -6.0, "acceleration_z": -52.0, "gyroscope_x": -18.0, "gyroscope_y": 41.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:05.266000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": -2.0, "acceleration_z": -55.0, "gyroscope_x": -19.0, "gyroscope_y": 60.0, "gyroscope_z": -78.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:10.734000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -14.0, "acceleration_z": -47.0, "gyroscope_x": -11.0, "gyroscope_y": 66.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:10.786000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -14.0, "acceleration_z": -47.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:10.838000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:10.890000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -13.0, "acceleration_z": -48.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:16.341000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -48.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:16.393000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": -14.0, "acceleration_z": -46.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:16.445000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -14.0, "acceleration_z": -47.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:16.497000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:21.945000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -16.0, "acceleration_z": -45.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:21.997000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -46.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:22.049000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -14.0, "acceleration_z": -46.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:22.101000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -49.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:27.550000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -9.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:27.602000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -46.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:27.654000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -14.0, "acceleration_z": -47.0, "gyroscope_x": -11.0, "gyroscope_y": 63.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:27.706000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -45.0, "gyroscope_x": -13.0, "gyroscope_y": 60.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:33.146000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -48.0, "gyroscope_x": -10.0, "gyroscope_y": 65.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:33.198000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -13.0, "acceleration_z": -47.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:33.250000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -14.0, "gyroscope_y": 60.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:33.302000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -16.0, "acceleration_z": -46.0, "gyroscope_x": -10.0, "gyroscope_y": 65.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:40.195000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -13.0, "acceleration_z": -47.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:40.247000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -14.0, "acceleration_z": -48.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:40.299000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -46.0, "gyroscope_x": -11.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:40.351000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -46.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:45.789000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:45.841000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -15.0, "acceleration_z": -47.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:45.893000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -15.0, "acceleration_z": -44.0, "gyroscope_x": -8.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:45.945000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": -16.0, "acceleration_z": -46.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:51.657000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 182.0, "acceleration_y": 68.0, "acceleration_z": -93.0, "gyroscope_x": 514.0, "gyroscope_y": 3486.0, "gyroscope_z": 285.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:51.709000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 160.0, "acceleration_y": -46.0, "acceleration_z": 132.0, "gyroscope_x": 675.0, "gyroscope_y": 1079.0, "gyroscope_z": 447.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:51.761000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 204.0, "acceleration_y": 49.0, "acceleration_z": -48.0, "gyroscope_x": 545.0, "gyroscope_y": -4087.0, "gyroscope_z": -1810.0, "crash": null, "terrain": null}, {"time": "2023-11-06 13:44:51.813000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 511.0, "acceleration_y": 96.0, "acceleration_z": -167.0, "gyroscope_x": -359.0, "gyroscope_y": -2220.0, "gyroscope_z": -1474.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:20.770000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 66.0, "acceleration_z": -134.0, "gyroscope_x": -118.0, "gyroscope_y": -4.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:25.988000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 45.0, "acceleration_z": -88.0, "gyroscope_x": -211.0, "gyroscope_y": -249.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:26.040000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 43.0, "acceleration_z": -192.0, "gyroscope_x": -31.0, "gyroscope_y": -228.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:26.092000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 36.0, "acceleration_z": -54.0, "gyroscope_x": 77.0, "gyroscope_y": -192.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:26.144000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 46.0, "acceleration_z": -53.0, "gyroscope_x": 107.0, "gyroscope_y": -276.0, "gyroscope_z": 33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:31.971000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 324.0, "acceleration_y": 34.0, "acceleration_z": 33.0, "gyroscope_x": -83.0, "gyroscope_y": 248.0, "gyroscope_z": 73.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:32.023000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 350.0, "acceleration_y": 46.0, "acceleration_z": 20.0, "gyroscope_x": -179.0, "gyroscope_y": 263.0, "gyroscope_z": 57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:32.075000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 357.0, "acceleration_y": 18.0, "acceleration_z": -19.0, "gyroscope_x": -336.0, "gyroscope_y": 142.0, "gyroscope_z": 87.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:32.127000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": -37.0, "acceleration_z": 4.0, "gyroscope_x": -570.0, "gyroscope_y": 124.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:37.981000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 36.0, "acceleration_z": -27.0, "gyroscope_x": 242.0, "gyroscope_y": 214.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:38.033000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 19.0, "acceleration_z": -162.0, "gyroscope_x": 172.0, "gyroscope_y": 189.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:38.085000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 40.0, "acceleration_z": -52.0, "gyroscope_x": 245.0, "gyroscope_y": 288.0, "gyroscope_z": 70.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:38.137000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 23.0, "acceleration_z": -146.0, "gyroscope_x": 230.0, "gyroscope_y": 87.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:43.841000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 324.0, "acceleration_y": 48.0, "acceleration_z": 8.0, "gyroscope_x": 9.0, "gyroscope_y": -74.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:43.893000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 161.0, "acceleration_y": 26.0, "acceleration_z": -86.0, "gyroscope_x": 39.0, "gyroscope_y": 7.0, "gyroscope_z": -91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:43.945000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 360.0, "acceleration_y": 5.0, "acceleration_z": -148.0, "gyroscope_x": 119.0, "gyroscope_y": 248.0, "gyroscope_z": -117.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:43.997000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 399.0, "acceleration_y": 35.0, "acceleration_z": 30.0, "gyroscope_x": 26.0, "gyroscope_y": -40.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:49.553000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 66.0, "acceleration_z": 48.0, "gyroscope_x": -10.0, "gyroscope_y": 5.0, "gyroscope_z": -121.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:49.605000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 193.0, "acceleration_y": 91.0, "acceleration_z": -197.0, "gyroscope_x": 118.0, "gyroscope_y": 155.0, "gyroscope_z": 24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:49.657000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 313.0, "acceleration_y": 63.0, "acceleration_z": 25.0, "gyroscope_x": -132.0, "gyroscope_y": 378.0, "gyroscope_z": 94.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:49.709000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 371.0, "acceleration_y": 46.0, "acceleration_z": -109.0, "gyroscope_x": 62.0, "gyroscope_y": 90.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:55.293000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 26.0, "acceleration_z": -124.0, "gyroscope_x": 239.0, "gyroscope_y": -18.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:55.345000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 288.0, "acceleration_y": 28.0, "acceleration_z": -115.0, "gyroscope_x": 221.0, "gyroscope_y": -9.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:55.397000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 21.0, "acceleration_z": -49.0, "gyroscope_x": 71.0, "gyroscope_y": 269.0, "gyroscope_z": 235.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:27:55.449000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 198.0, "acceleration_y": 82.0, "acceleration_z": 93.0, "gyroscope_x": 80.0, "gyroscope_y": 118.0, "gyroscope_z": 95.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:00.981000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": -24.0, "acceleration_z": -14.0, "gyroscope_x": -14.0, "gyroscope_y": 86.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:01.033000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 12.0, "acceleration_z": 11.0, "gyroscope_x": -13.0, "gyroscope_y": 59.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:01.085000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 297.0, "acceleration_y": 7.0, "acceleration_z": -25.0, "gyroscope_x": 10.0, "gyroscope_y": 83.0, "gyroscope_z": 24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:01.137000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 0.0, "acceleration_z": -68.0, "gyroscope_x": 85.0, "gyroscope_y": 132.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:06.747000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": 29.0, "acceleration_z": -38.0, "gyroscope_x": -86.0, "gyroscope_y": 1.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:06.799000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 20.0, "acceleration_z": -86.0, "gyroscope_x": 7.0, "gyroscope_y": 40.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:06.851000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 3.0, "acceleration_z": -93.0, "gyroscope_x": 110.0, "gyroscope_y": 109.0, "gyroscope_z": 30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:06.903000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 14.0, "acceleration_z": -72.0, "gyroscope_x": 125.0, "gyroscope_y": -126.0, "gyroscope_z": 24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:12.591000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 37.0, "acceleration_z": -14.0, "gyroscope_x": 54.0, "gyroscope_y": 81.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:12.643000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 35.0, "acceleration_z": -50.0, "gyroscope_x": 67.0, "gyroscope_y": 104.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:12.695000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 23.0, "acceleration_z": -51.0, "gyroscope_x": 87.0, "gyroscope_y": 82.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:12.747000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 27.0, "acceleration_z": -39.0, "gyroscope_x": 97.0, "gyroscope_y": 85.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:18.391000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 21.0, "acceleration_z": -35.0, "gyroscope_x": -47.0, "gyroscope_y": 46.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:18.443000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 17.0, "acceleration_z": -25.0, "gyroscope_x": -47.0, "gyroscope_y": 39.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:18.495000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 20.0, "acceleration_z": -35.0, "gyroscope_x": -54.0, "gyroscope_y": 36.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:18.547000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 19.0, "acceleration_z": -34.0, "gyroscope_x": -50.0, "gyroscope_y": 34.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:24.265000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 25.0, "acceleration_z": -64.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:24.317000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 22.0, "acceleration_z": -69.0, "gyroscope_x": -14.0, "gyroscope_y": 86.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:24.369000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 19.0, "acceleration_z": -63.0, "gyroscope_x": -13.0, "gyroscope_y": 87.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:24.421000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 21.0, "acceleration_z": -63.0, "gyroscope_x": -16.0, "gyroscope_y": 97.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:30.221000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 89.0, "acceleration_z": -91.0, "gyroscope_x": 19.0, "gyroscope_y": 71.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:30.273000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 328.0, "acceleration_y": 56.0, "acceleration_z": -21.0, "gyroscope_x": -595.0, "gyroscope_y": 155.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:30.325000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 96.0, "acceleration_z": -109.0, "gyroscope_x": -110.0, "gyroscope_y": 107.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:30.377000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 196.0, "acceleration_y": 1.0, "acceleration_z": -43.0, "gyroscope_x": -208.0, "gyroscope_y": -87.0, "gyroscope_z": 104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:35.891000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 336.0, "acceleration_y": 24.0, "acceleration_z": -109.0, "gyroscope_x": 9.0, "gyroscope_y": 136.0, "gyroscope_z": 24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:35.943000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 322.0, "acceleration_y": 83.0, "acceleration_z": 61.0, "gyroscope_x": 28.0, "gyroscope_y": 94.0, "gyroscope_z": -38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:35.995000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 189.0, "acceleration_y": -1.0, "acceleration_z": -75.0, "gyroscope_x": 24.0, "gyroscope_y": 60.0, "gyroscope_z": 133.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:36.047000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": 106.0, "acceleration_z": -53.0, "gyroscope_x": 76.0, "gyroscope_y": 165.0, "gyroscope_z": 155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:41.569000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 10.0, "acceleration_z": -6.0, "gyroscope_x": -21.0, "gyroscope_y": 205.0, "gyroscope_z": 84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:41.621000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 318.0, "acceleration_y": 65.0, "acceleration_z": -140.0, "gyroscope_x": -314.0, "gyroscope_y": 48.0, "gyroscope_z": -55.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:41.673000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": -52.0, "acceleration_z": -197.0, "gyroscope_x": -129.0, "gyroscope_y": 183.0, "gyroscope_z": -37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:41.725000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 306.0, "acceleration_y": 23.0, "acceleration_z": -99.0, "gyroscope_x": -258.0, "gyroscope_y": -38.0, "gyroscope_z": 46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:47.319000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 306.0, "acceleration_y": -50.0, "acceleration_z": -118.0, "gyroscope_x": -142.0, "gyroscope_y": 146.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:47.371000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 6.0, "acceleration_z": -60.0, "gyroscope_x": 17.0, "gyroscope_y": 145.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:47.423000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": -34.0, "acceleration_z": -66.0, "gyroscope_x": -58.0, "gyroscope_y": -33.0, "gyroscope_z": -49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:47.475000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": -17.0, "acceleration_z": -59.0, "gyroscope_x": -2.0, "gyroscope_y": 38.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:53.039000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": -14.0, "acceleration_z": -35.0, "gyroscope_x": -184.0, "gyroscope_y": 41.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:53.091000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 15.0, "acceleration_z": -29.0, "gyroscope_x": -222.0, "gyroscope_y": -16.0, "gyroscope_z": -52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:53.143000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 32.0, "acceleration_z": -33.0, "gyroscope_x": -295.0, "gyroscope_y": -37.0, "gyroscope_z": 41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:53.195000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 7.0, "acceleration_z": 13.0, "gyroscope_x": -312.0, "gyroscope_y": -52.0, "gyroscope_z": -43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:59.002000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -6.0, "gyroscope_x": -47.0, "gyroscope_y": 69.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:59.054000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 28.0, "acceleration_z": 14.0, "gyroscope_x": -67.0, "gyroscope_y": 52.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:59.106000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 38.0, "acceleration_z": 4.0, "gyroscope_x": -55.0, "gyroscope_y": 37.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:28:59.158000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 33.0, "acceleration_z": 12.0, "gyroscope_x": -87.0, "gyroscope_y": 40.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:04.837000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 22.0, "acceleration_z": 0.0, "gyroscope_x": -8.0, "gyroscope_y": 62.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:04.889000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 22.0, "acceleration_z": 3.0, "gyroscope_x": -4.0, "gyroscope_y": 67.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:04.941000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 22.0, "acceleration_z": 3.0, "gyroscope_x": -9.0, "gyroscope_y": 62.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:04.993000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 23.0, "acceleration_z": 2.0, "gyroscope_x": -1.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:10.583000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 30.0, "acceleration_z": -11.0, "gyroscope_x": -37.0, "gyroscope_y": 21.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:10.635000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 23.0, "acceleration_z": -23.0, "gyroscope_x": -38.0, "gyroscope_y": 45.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:10.687000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 22.0, "acceleration_z": -13.0, "gyroscope_x": -48.0, "gyroscope_y": 49.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:10.739000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -14.0, "gyroscope_x": -34.0, "gyroscope_y": 64.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:16.409000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 23.0, "acceleration_z": -19.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:16.461000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 27.0, "acceleration_z": -20.0, "gyroscope_x": -16.0, "gyroscope_y": 69.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:16.513000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -19.0, "gyroscope_x": -13.0, "gyroscope_y": 75.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:16.565000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 25.0, "acceleration_z": -16.0, "gyroscope_x": -12.0, "gyroscope_y": 73.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:22.091000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -15.0, "gyroscope_x": -11.0, "gyroscope_y": 60.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:22.143000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -15.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:22.195000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 23.0, "acceleration_z": -18.0, "gyroscope_x": -13.0, "gyroscope_y": 58.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:22.247000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 25.0, "acceleration_z": -17.0, "gyroscope_x": -15.0, "gyroscope_y": 60.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:27.979000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 44.0, "acceleration_z": -37.0, "gyroscope_x": 257.0, "gyroscope_y": 1.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:28.031000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 309.0, "acceleration_y": 77.0, "acceleration_z": -48.0, "gyroscope_x": 202.0, "gyroscope_y": 201.0, "gyroscope_z": 42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:28.083000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 52.0, "acceleration_z": -9.0, "gyroscope_x": 90.0, "gyroscope_y": 255.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:28.135000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 16.0, "acceleration_z": -73.0, "gyroscope_x": 188.0, "gyroscope_y": 162.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:33.700000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 66.0, "acceleration_z": -142.0, "gyroscope_x": -40.0, "gyroscope_y": 103.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:33.752000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 45.0, "acceleration_z": -32.0, "gyroscope_x": -177.0, "gyroscope_y": 363.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:33.804000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 90.0, "acceleration_z": -103.0, "gyroscope_x": 0.0, "gyroscope_y": 223.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:33.856000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 223.0, "acceleration_y": 65.0, "acceleration_z": 61.0, "gyroscope_x": -216.0, "gyroscope_y": -93.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:39.541000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 313.0, "acceleration_y": 56.0, "acceleration_z": 40.0, "gyroscope_x": -51.0, "gyroscope_y": 85.0, "gyroscope_z": -55.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:39.593000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 205.0, "acceleration_y": 15.0, "acceleration_z": 61.0, "gyroscope_x": -273.0, "gyroscope_y": 34.0, "gyroscope_z": -67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:39.645000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 221.0, "acceleration_y": 63.0, "acceleration_z": 65.0, "gyroscope_x": -247.0, "gyroscope_y": 107.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:39.697000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": 27.0, "acceleration_z": -181.0, "gyroscope_x": -187.0, "gyroscope_y": 121.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:45.381000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 305.0, "acceleration_y": -10.0, "acceleration_z": -36.0, "gyroscope_x": -57.0, "gyroscope_y": 248.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:45.433000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": -20.0, "acceleration_z": -79.0, "gyroscope_x": 72.0, "gyroscope_y": 84.0, "gyroscope_z": 59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:45.485000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 37.0, "acceleration_z": -16.0, "gyroscope_x": 1.0, "gyroscope_y": 48.0, "gyroscope_z": 14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:45.537000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 60.0, "acceleration_z": 55.0, "gyroscope_x": -125.0, "gyroscope_y": 316.0, "gyroscope_z": 69.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:51.325000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 28.0, "acceleration_z": -68.0, "gyroscope_x": -271.0, "gyroscope_y": -11.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:51.377000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 27.0, "acceleration_z": 6.0, "gyroscope_x": -293.0, "gyroscope_y": 118.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:51.429000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": 32.0, "acceleration_z": -67.0, "gyroscope_x": -85.0, "gyroscope_y": 41.0, "gyroscope_z": -61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:51.481000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 47.0, "acceleration_z": 29.0, "gyroscope_x": -208.0, "gyroscope_y": 263.0, "gyroscope_z": 5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:57.225000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 6.0, "acceleration_z": -74.0, "gyroscope_x": 72.0, "gyroscope_y": 170.0, "gyroscope_z": 66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:57.277000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 141.0, "acceleration_y": 35.0, "acceleration_z": 85.0, "gyroscope_x": 11.0, "gyroscope_y": -145.0, "gyroscope_z": -61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:57.329000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 340.0, "acceleration_y": 108.0, "acceleration_z": 89.0, "gyroscope_x": -112.0, "gyroscope_y": 140.0, "gyroscope_z": 87.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:29:57.381000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 221.0, "acceleration_y": 25.0, "acceleration_z": -18.0, "gyroscope_x": -344.0, "gyroscope_y": -116.0, "gyroscope_z": 57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:02.958000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 146.0, "acceleration_z": 37.0, "gyroscope_x": 15.0, "gyroscope_y": 8.0, "gyroscope_z": 41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:03.010000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 10.0, "acceleration_z": 18.0, "gyroscope_x": -259.0, "gyroscope_y": 282.0, "gyroscope_z": 91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:03.062000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": -64.0, "acceleration_z": -98.0, "gyroscope_x": -167.0, "gyroscope_y": 124.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:03.114000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 121.0, "acceleration_z": -134.0, "gyroscope_x": -422.0, "gyroscope_y": 109.0, "gyroscope_z": 134.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:08.632000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 195.0, "acceleration_y": 50.0, "acceleration_z": -106.0, "gyroscope_x": -30.0, "gyroscope_y": 62.0, "gyroscope_z": -89.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:08.684000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 50.0, "acceleration_z": -49.0, "gyroscope_x": -67.0, "gyroscope_y": 80.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:08.736000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 21.0, "acceleration_z": -128.0, "gyroscope_x": 39.0, "gyroscope_y": -61.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:08.788000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 45.0, "acceleration_z": 45.0, "gyroscope_x": -79.0, "gyroscope_y": 22.0, "gyroscope_z": -141.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:14.324000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 13.0, "acceleration_z": -62.0, "gyroscope_x": -105.0, "gyroscope_y": -212.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:14.376000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 181.0, "acceleration_y": 33.0, "acceleration_z": -101.0, "gyroscope_x": 162.0, "gyroscope_y": -43.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:14.428000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 116.0, "acceleration_y": -21.0, "acceleration_z": -23.0, "gyroscope_x": 4.0, "gyroscope_y": 101.0, "gyroscope_z": -131.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:14.480000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": 68.0, "acceleration_z": -79.0, "gyroscope_x": -140.0, "gyroscope_y": -26.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:20.250000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 49.0, "acceleration_z": -36.0, "gyroscope_x": 40.0, "gyroscope_y": 195.0, "gyroscope_z": 129.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:20.302000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 53.0, "acceleration_z": 76.0, "gyroscope_x": -47.0, "gyroscope_y": -75.0, "gyroscope_z": -72.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:20.354000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 50.0, "acceleration_z": -1.0, "gyroscope_x": 118.0, "gyroscope_y": 146.0, "gyroscope_z": 46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:20.406000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": 27.0, "acceleration_z": -171.0, "gyroscope_x": 201.0, "gyroscope_y": 30.0, "gyroscope_z": -117.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:26.118000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 219.0, "acceleration_y": -14.0, "acceleration_z": 17.0, "gyroscope_x": 181.0, "gyroscope_y": 209.0, "gyroscope_z": -86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:26.170000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": -10.0, "acceleration_z": -1.0, "gyroscope_x": 118.0, "gyroscope_y": 40.0, "gyroscope_z": 85.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:26.222000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 18.0, "acceleration_z": 49.0, "gyroscope_x": 7.0, "gyroscope_y": 108.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:26.274000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 14.0, "acceleration_z": 14.0, "gyroscope_x": -78.0, "gyroscope_y": -77.0, "gyroscope_z": 127.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:31.858000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -58.0, "gyroscope_x": -18.0, "gyroscope_y": 128.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:31.910000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 31.0, "acceleration_z": -62.0, "gyroscope_x": 8.0, "gyroscope_y": 125.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:31.962000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 30.0, "acceleration_z": -44.0, "gyroscope_x": 9.0, "gyroscope_y": 112.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:32.014000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 36.0, "acceleration_z": -46.0, "gyroscope_x": -10.0, "gyroscope_y": 71.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:37.759000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 26.0, "acceleration_z": -36.0, "gyroscope_x": -183.0, "gyroscope_y": 2.0, "gyroscope_z": -155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:37.811000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 358.0, "acceleration_y": 107.0, "acceleration_z": -76.0, "gyroscope_x": -156.0, "gyroscope_y": 90.0, "gyroscope_z": -141.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:37.863000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 0.0, "acceleration_z": -100.0, "gyroscope_x": -61.0, "gyroscope_y": 101.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:37.915000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": -40.0, "acceleration_z": -114.0, "gyroscope_x": 82.0, "gyroscope_y": -40.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:43.424000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 11.0, "acceleration_z": -96.0, "gyroscope_x": 39.0, "gyroscope_y": -363.0, "gyroscope_z": 129.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:43.476000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 165.0, "acceleration_y": 67.0, "acceleration_z": 37.0, "gyroscope_x": 2.0, "gyroscope_y": -159.0, "gyroscope_z": 121.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:43.528000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 201.0, "acceleration_y": 49.0, "acceleration_z": 21.0, "gyroscope_x": 63.0, "gyroscope_y": 37.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:43.580000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 30.0, "acceleration_z": -66.0, "gyroscope_x": 85.0, "gyroscope_y": -34.0, "gyroscope_z": -189.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:49.087000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 212.0, "acceleration_y": 75.0, "acceleration_z": -79.0, "gyroscope_x": -411.0, "gyroscope_y": -208.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:49.139000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 176.0, "acceleration_y": -45.0, "acceleration_z": -134.0, "gyroscope_x": -306.0, "gyroscope_y": -377.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:49.191000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 204.0, "acceleration_y": 98.0, "acceleration_z": -67.0, "gyroscope_x": 0.0, "gyroscope_y": -357.0, "gyroscope_z": -211.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:49.243000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 140.0, "acceleration_y": -42.0, "acceleration_z": -9.0, "gyroscope_x": 7.0, "gyroscope_y": -329.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:54.737000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": -33.0, "acceleration_z": -200.0, "gyroscope_x": -59.0, "gyroscope_y": -52.0, "gyroscope_z": 98.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:54.789000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 142.0, "acceleration_y": 28.0, "acceleration_z": -9.0, "gyroscope_x": -102.0, "gyroscope_y": 49.0, "gyroscope_z": -127.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:54.841000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 170.0, "acceleration_y": 34.0, "acceleration_z": 138.0, "gyroscope_x": -183.0, "gyroscope_y": -71.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:30:54.893000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 45.0, "acceleration_z": -11.0, "gyroscope_x": -26.0, "gyroscope_y": -7.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:00.407000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 198.0, "acceleration_y": 14.0, "acceleration_z": -84.0, "gyroscope_x": 121.0, "gyroscope_y": -56.0, "gyroscope_z": -177.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:00.459000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": -41.0, "acceleration_z": 63.0, "gyroscope_x": 81.0, "gyroscope_y": 33.0, "gyroscope_z": 78.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:00.511000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 189.0, "acceleration_y": 82.0, "acceleration_z": -156.0, "gyroscope_x": 113.0, "gyroscope_y": 74.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:00.563000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 42.0, "acceleration_z": 14.0, "gyroscope_x": 106.0, "gyroscope_y": 87.0, "gyroscope_z": -41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:06.427000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 45.0, "acceleration_z": 15.0, "gyroscope_x": -91.0, "gyroscope_y": 128.0, "gyroscope_z": 133.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:06.479000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": 17.0, "acceleration_z": -59.0, "gyroscope_x": -122.0, "gyroscope_y": 74.0, "gyroscope_z": 40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:06.531000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 23.0, "acceleration_z": -51.0, "gyroscope_x": -29.0, "gyroscope_y": -10.0, "gyroscope_z": -73.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:06.583000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": -14.0, "acceleration_z": -14.0, "gyroscope_x": -158.0, "gyroscope_y": 15.0, "gyroscope_z": -119.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:12.104000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 319.0, "acceleration_y": 33.0, "acceleration_z": -108.0, "gyroscope_x": 61.0, "gyroscope_y": 182.0, "gyroscope_z": 42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:12.156000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 175.0, "acceleration_y": 6.0, "acceleration_z": -176.0, "gyroscope_x": 67.0, "gyroscope_y": 45.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:12.208000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 17.0, "acceleration_z": -154.0, "gyroscope_x": 117.0, "gyroscope_y": 153.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:12.260000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 44.0, "acceleration_z": -112.0, "gyroscope_x": 209.0, "gyroscope_y": 114.0, "gyroscope_z": -97.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:17.779000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 305.0, "acceleration_y": 100.0, "acceleration_z": 23.0, "gyroscope_x": -125.0, "gyroscope_y": 223.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:17.831000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 65.0, "acceleration_z": -4.0, "gyroscope_x": -143.0, "gyroscope_y": -43.0, "gyroscope_z": 13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:17.883000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 42.0, "acceleration_z": 155.0, "gyroscope_x": -167.0, "gyroscope_y": 51.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:17.935000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 30.0, "acceleration_z": -48.0, "gyroscope_x": -31.0, "gyroscope_y": 130.0, "gyroscope_z": 63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:23.784000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 334.0, "acceleration_y": 75.0, "acceleration_z": -157.0, "gyroscope_x": 154.0, "gyroscope_y": 124.0, "gyroscope_z": -84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:23.836000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 23.0, "acceleration_z": -141.0, "gyroscope_x": 132.0, "gyroscope_y": 174.0, "gyroscope_z": 49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:23.888000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 140.0, "acceleration_y": 6.0, "acceleration_z": 26.0, "gyroscope_x": 132.0, "gyroscope_y": 235.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:23.940000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 345.0, "acceleration_y": 134.0, "acceleration_z": -35.0, "gyroscope_x": 140.0, "gyroscope_y": 164.0, "gyroscope_z": -103.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:29.471000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 96.0, "acceleration_z": -188.0, "gyroscope_x": -109.0, "gyroscope_y": 232.0, "gyroscope_z": 86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:29.523000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 63.0, "acceleration_z": -74.0, "gyroscope_x": 53.0, "gyroscope_y": 259.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:29.575000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 160.0, "acceleration_z": 15.0, "gyroscope_x": 22.0, "gyroscope_y": 177.0, "gyroscope_z": -59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:29.627000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 173.0, "acceleration_y": -28.0, "acceleration_z": -27.0, "gyroscope_x": 131.0, "gyroscope_y": -171.0, "gyroscope_z": 58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:35.226000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 181.0, "acceleration_y": 48.0, "acceleration_z": 16.0, "gyroscope_x": -220.0, "gyroscope_y": 206.0, "gyroscope_z": 144.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:35.278000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -4.0, "acceleration_z": -41.0, "gyroscope_x": -231.0, "gyroscope_y": 258.0, "gyroscope_z": 100.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:35.330000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 76.0, "acceleration_z": -50.0, "gyroscope_x": 37.0, "gyroscope_y": 120.0, "gyroscope_z": -69.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:35.382000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 173.0, "acceleration_y": 8.0, "acceleration_z": 90.0, "gyroscope_x": -114.0, "gyroscope_y": 180.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:41.111000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 109.0, "acceleration_z": 102.0, "gyroscope_x": -151.0, "gyroscope_y": 10.0, "gyroscope_z": -101.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:41.163000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": -11.0, "acceleration_z": -101.0, "gyroscope_x": -120.0, "gyroscope_y": -38.0, "gyroscope_z": 266.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:41.215000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 155.0, "acceleration_y": 45.0, "acceleration_z": 73.0, "gyroscope_x": -133.0, "gyroscope_y": -63.0, "gyroscope_z": 22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:41.267000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 34.0, "acceleration_z": -6.0, "gyroscope_x": -66.0, "gyroscope_y": -286.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:46.796000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 72.0, "acceleration_z": -154.0, "gyroscope_x": -46.0, "gyroscope_y": 56.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:46.848000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 316.0, "acceleration_y": 72.0, "acceleration_z": 34.0, "gyroscope_x": 47.0, "gyroscope_y": 181.0, "gyroscope_z": 109.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:46.900000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 379.0, "acceleration_y": 32.0, "acceleration_z": -174.0, "gyroscope_x": 280.0, "gyroscope_y": 99.0, "gyroscope_z": 86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:46.952000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": -24.0, "acceleration_z": 29.0, "gyroscope_x": -20.0, "gyroscope_y": 367.0, "gyroscope_z": 59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:52.523000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 30.0, "acceleration_y": 62.0, "acceleration_z": -39.0, "gyroscope_x": 3.0, "gyroscope_y": 126.0, "gyroscope_z": -96.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:52.575000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 67.0, "acceleration_z": -49.0, "gyroscope_x": 82.0, "gyroscope_y": 121.0, "gyroscope_z": 73.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:52.627000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": 94.0, "acceleration_z": -131.0, "gyroscope_x": -52.0, "gyroscope_y": 370.0, "gyroscope_z": -124.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:52.679000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 174.0, "acceleration_y": 17.0, "acceleration_z": -1.0, "gyroscope_x": -15.0, "gyroscope_y": 220.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:58.694000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 195.0, "acceleration_y": -23.0, "acceleration_z": 8.0, "gyroscope_x": -36.0, "gyroscope_y": 263.0, "gyroscope_z": 48.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:58.746000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 207.0, "acceleration_y": 7.0, "acceleration_z": -134.0, "gyroscope_x": 11.0, "gyroscope_y": 167.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:58.798000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 59.0, "acceleration_z": 72.0, "gyroscope_x": -212.0, "gyroscope_y": 40.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:31:58.850000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 200.0, "acceleration_y": -20.0, "acceleration_z": 32.0, "gyroscope_x": -21.0, "gyroscope_y": 163.0, "gyroscope_z": -119.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:04.647000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 329.0, "acceleration_y": 37.0, "acceleration_z": -29.0, "gyroscope_x": -58.0, "gyroscope_y": 321.0, "gyroscope_z": -83.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:04.699000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 303.0, "acceleration_y": 18.0, "acceleration_z": -12.0, "gyroscope_x": 6.0, "gyroscope_y": 189.0, "gyroscope_z": -150.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:04.751000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": -18.0, "acceleration_z": 91.0, "gyroscope_x": -52.0, "gyroscope_y": -19.0, "gyroscope_z": 44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:04.803000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 176.0, "acceleration_y": -9.0, "acceleration_z": -14.0, "gyroscope_x": -120.0, "gyroscope_y": -31.0, "gyroscope_z": 38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:10.630000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 200.0, "acceleration_y": 52.0, "acceleration_z": 45.0, "gyroscope_x": 177.0, "gyroscope_y": 102.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:10.682000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 147.0, "acceleration_y": -28.0, "acceleration_z": -36.0, "gyroscope_x": 109.0, "gyroscope_y": 91.0, "gyroscope_z": -74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:10.734000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 40.0, "acceleration_z": 144.0, "gyroscope_x": 139.0, "gyroscope_y": 166.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:10.786000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 346.0, "acceleration_y": 69.0, "acceleration_z": -229.0, "gyroscope_x": 298.0, "gyroscope_y": -23.0, "gyroscope_z": -143.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:16.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": -23.0, "acceleration_z": -60.0, "gyroscope_x": 33.0, "gyroscope_y": 138.0, "gyroscope_z": 38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:16.396000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": 91.0, "acceleration_z": -90.0, "gyroscope_x": 44.0, "gyroscope_y": 24.0, "gyroscope_z": 138.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:16.448000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 334.0, "acceleration_y": -13.0, "acceleration_z": -7.0, "gyroscope_x": 76.0, "gyroscope_y": 44.0, "gyroscope_z": 375.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:16.500000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 10.0, "acceleration_z": -14.0, "gyroscope_x": 10.0, "gyroscope_y": 130.0, "gyroscope_z": 371.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:22.243000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 148.0, "acceleration_y": 42.0, "acceleration_z": -137.0, "gyroscope_x": 43.0, "gyroscope_y": -121.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:22.295000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": 103.0, "acceleration_z": 116.0, "gyroscope_x": -193.0, "gyroscope_y": -137.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:22.347000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 196.0, "acceleration_y": 92.0, "acceleration_z": -12.0, "gyroscope_x": -46.0, "gyroscope_y": 50.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:22.399000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 170.0, "acceleration_y": -2.0, "acceleration_z": -107.0, "gyroscope_x": 12.0, "gyroscope_y": -272.0, "gyroscope_z": -118.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:28.157000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 208.0, "acceleration_y": -21.0, "acceleration_z": -35.0, "gyroscope_x": 141.0, "gyroscope_y": 116.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:28.209000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 78.0, "acceleration_z": -30.0, "gyroscope_x": 68.0, "gyroscope_y": 114.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:28.261000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": -28.0, "acceleration_z": 6.0, "gyroscope_x": 13.0, "gyroscope_y": 116.0, "gyroscope_z": -65.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:28.313000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": -7.0, "acceleration_z": -50.0, "gyroscope_x": 153.0, "gyroscope_y": 33.0, "gyroscope_z": -80.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:33.935000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 180.0, "acceleration_y": 26.0, "acceleration_z": -42.0, "gyroscope_x": -168.0, "gyroscope_y": -85.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:33.987000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 1.0, "acceleration_z": -68.0, "gyroscope_x": -166.0, "gyroscope_y": 32.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:34.039000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 62.0, "acceleration_z": -41.0, "gyroscope_x": -246.0, "gyroscope_y": 3.0, "gyroscope_z": -77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:34.091000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 323.0, "acceleration_y": 96.0, "acceleration_z": -100.0, "gyroscope_x": -5.0, "gyroscope_y": 129.0, "gyroscope_z": 58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:39.609000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 66.0, "acceleration_z": -63.0, "gyroscope_x": 85.0, "gyroscope_y": 108.0, "gyroscope_z": -83.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:39.661000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 195.0, "acceleration_y": 50.0, "acceleration_z": -80.0, "gyroscope_x": -63.0, "gyroscope_y": 143.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:39.713000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 29.0, "acceleration_z": -35.0, "gyroscope_x": 73.0, "gyroscope_y": 19.0, "gyroscope_z": -292.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:39.765000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 53.0, "acceleration_z": -22.0, "gyroscope_x": 19.0, "gyroscope_y": -134.0, "gyroscope_z": -146.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:45.914000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 50.0, "acceleration_z": -44.0, "gyroscope_x": 54.0, "gyroscope_y": 79.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:45.966000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 224.0, "acceleration_y": 20.0, "acceleration_z": -12.0, "gyroscope_x": -49.0, "gyroscope_y": 230.0, "gyroscope_z": -65.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:46.018000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 77.0, "acceleration_z": -21.0, "gyroscope_x": -1.0, "gyroscope_y": 163.0, "gyroscope_z": 112.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:46.070000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 184.0, "acceleration_y": 9.0, "acceleration_z": 22.0, "gyroscope_x": -56.0, "gyroscope_y": 81.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:51.669000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 44.0, "acceleration_z": -26.0, "gyroscope_x": -48.0, "gyroscope_y": 53.0, "gyroscope_z": -84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:51.721000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 21.0, "acceleration_z": -87.0, "gyroscope_x": -2.0, "gyroscope_y": -5.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:51.773000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 39.0, "acceleration_z": -45.0, "gyroscope_x": -47.0, "gyroscope_y": -19.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:51.825000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 29.0, "acceleration_z": 71.0, "gyroscope_x": -111.0, "gyroscope_y": 157.0, "gyroscope_z": -39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:57.883000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 36.0, "acceleration_z": -37.0, "gyroscope_x": -12.0, "gyroscope_y": 115.0, "gyroscope_z": 29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:57.935000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 33.0, "acceleration_z": 51.0, "gyroscope_x": -131.0, "gyroscope_y": 243.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:57.987000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 52.0, "acceleration_z": -8.0, "gyroscope_x": -117.0, "gyroscope_y": 171.0, "gyroscope_z": 67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:32:58.039000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 4.0, "acceleration_z": -16.0, "gyroscope_x": -102.0, "gyroscope_y": 176.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:03.701000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 22.0, "acceleration_z": -38.0, "gyroscope_x": -22.0, "gyroscope_y": 80.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:03.753000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 18.0, "acceleration_z": -38.0, "gyroscope_x": -15.0, "gyroscope_y": 90.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:03.805000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 19.0, "acceleration_z": -31.0, "gyroscope_x": -15.0, "gyroscope_y": 92.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:03.857000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 15.0, "acceleration_z": -26.0, "gyroscope_x": -14.0, "gyroscope_y": 93.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:09.401000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 23.0, "acceleration_z": -34.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:09.453000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -32.0, "gyroscope_x": -14.0, "gyroscope_y": 68.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:09.505000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 25.0, "acceleration_z": -32.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:09.557000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -30.0, "gyroscope_x": -15.0, "gyroscope_y": 68.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:15.158000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 21.0, "acceleration_z": -27.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:15.210000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 20.0, "acceleration_z": -29.0, "gyroscope_x": -13.0, "gyroscope_y": 59.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:15.262000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 19.0, "acceleration_z": -31.0, "gyroscope_x": -13.0, "gyroscope_y": 58.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:15.314000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 20.0, "acceleration_z": -32.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:20.885000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -29.0, "gyroscope_x": -14.0, "gyroscope_y": 60.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:20.937000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 25.0, "acceleration_z": -31.0, "gyroscope_x": -13.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:20.989000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -29.0, "gyroscope_x": -12.0, "gyroscope_y": 60.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:21.041000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -33.0, "gyroscope_x": -14.0, "gyroscope_y": 61.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:26.597000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -30.0, "gyroscope_x": -10.0, "gyroscope_y": 60.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:26.649000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 22.0, "acceleration_z": -31.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:26.701000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -28.0, "gyroscope_x": -16.0, "gyroscope_y": 67.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:26.753000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -29.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:32.298000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 12.0, "acceleration_z": -88.0, "gyroscope_x": 4.0, "gyroscope_y": 104.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:32.350000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 14.0, "acceleration_z": -46.0, "gyroscope_x": -18.0, "gyroscope_y": 88.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:32.402000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 28.0, "acceleration_z": -77.0, "gyroscope_x": -9.0, "gyroscope_y": 107.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:32.454000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 22.0, "acceleration_z": -54.0, "gyroscope_x": -15.0, "gyroscope_y": 99.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:37.940000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 22.0, "acceleration_z": -10.0, "gyroscope_x": 264.0, "gyroscope_y": 287.0, "gyroscope_z": 14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:37.992000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 62.0, "acceleration_z": 43.0, "gyroscope_x": 155.0, "gyroscope_y": 171.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:38.044000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 55.0, "acceleration_z": -53.0, "gyroscope_x": 367.0, "gyroscope_y": 303.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:38.096000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 39.0, "acceleration_z": -57.0, "gyroscope_x": 149.0, "gyroscope_y": 158.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:43.678000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 62.0, "acceleration_z": -13.0, "gyroscope_x": 53.0, "gyroscope_y": -24.0, "gyroscope_z": 243.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:43.730000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": -7.0, "acceleration_z": 55.0, "gyroscope_x": -60.0, "gyroscope_y": 42.0, "gyroscope_z": 104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:43.782000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": 92.0, "gyroscope_y": -76.0, "gyroscope_z": 104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:43.834000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": 62.0, "acceleration_z": 14.0, "gyroscope_x": 27.0, "gyroscope_y": -72.0, "gyroscope_z": 41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:49.372000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 427.0, "acceleration_y": -11.0, "acceleration_z": 45.0, "gyroscope_x": 164.0, "gyroscope_y": 131.0, "gyroscope_z": -74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:49.424000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 360.0, "acceleration_y": 239.0, "acceleration_z": 52.0, "gyroscope_x": 14.0, "gyroscope_y": 269.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:49.476000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 185.0, "acceleration_y": 83.0, "acceleration_z": -40.0, "gyroscope_x": 77.0, "gyroscope_y": 112.0, "gyroscope_z": -85.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:49.528000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 29.0, "acceleration_z": -96.0, "gyroscope_x": 169.0, "gyroscope_y": -7.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:56.764000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 191.0, "acceleration_y": 8.0, "acceleration_z": 197.0, "gyroscope_x": -255.0, "gyroscope_y": -35.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:56.816000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 301.0, "acceleration_y": 102.0, "acceleration_z": -87.0, "gyroscope_x": -214.0, "gyroscope_y": -22.0, "gyroscope_z": -49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:56.868000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 66.0, "acceleration_z": -73.0, "gyroscope_x": -214.0, "gyroscope_y": 123.0, "gyroscope_z": -179.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:33:56.920000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": -12.0, "acceleration_z": -31.0, "gyroscope_x": -42.0, "gyroscope_y": -208.0, "gyroscope_z": -70.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:02.473000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 30.0, "acceleration_z": -43.0, "gyroscope_x": 114.0, "gyroscope_y": 89.0, "gyroscope_z": 60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:02.525000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 59.0, "acceleration_z": 53.0, "gyroscope_x": -43.0, "gyroscope_y": 314.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:02.577000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 133.0, "acceleration_y": 32.0, "acceleration_z": 31.0, "gyroscope_x": -62.0, "gyroscope_y": 119.0, "gyroscope_z": -177.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:02.629000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 108.0, "acceleration_z": -70.0, "gyroscope_x": -72.0, "gyroscope_y": 95.0, "gyroscope_z": 105.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:08.401000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 339.0, "acceleration_y": 56.0, "acceleration_z": 1.0, "gyroscope_x": -20.0, "gyroscope_y": 392.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:08.453000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 166.0, "acceleration_y": 39.0, "acceleration_z": 5.0, "gyroscope_x": -175.0, "gyroscope_y": 135.0, "gyroscope_z": -234.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:08.505000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 47.0, "acceleration_z": -1.0, "gyroscope_x": -179.0, "gyroscope_y": 191.0, "gyroscope_z": -139.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:08.557000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": -5.0, "acceleration_z": -64.0, "gyroscope_x": -197.0, "gyroscope_y": -6.0, "gyroscope_z": -221.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:14.645000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 7.0, "acceleration_z": -111.0, "gyroscope_x": -152.0, "gyroscope_y": 73.0, "gyroscope_z": 23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:14.697000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 143.0, "acceleration_y": -39.0, "acceleration_z": -182.0, "gyroscope_x": -130.0, "gyroscope_y": -148.0, "gyroscope_z": 36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:14.749000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 11.0, "acceleration_z": -93.0, "gyroscope_x": 63.0, "gyroscope_y": 188.0, "gyroscope_z": -67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:14.801000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 193.0, "acceleration_y": 19.0, "acceleration_z": -117.0, "gyroscope_x": 53.0, "gyroscope_y": 163.0, "gyroscope_z": 75.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:20.415000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": -15.0, "acceleration_z": -104.0, "gyroscope_x": -184.0, "gyroscope_y": 131.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:20.467000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": -3.0, "acceleration_z": -94.0, "gyroscope_x": -112.0, "gyroscope_y": 7.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:20.519000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 10.0, "acceleration_z": -114.0, "gyroscope_x": -67.0, "gyroscope_y": 109.0, "gyroscope_z": 23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:20.571000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 177.0, "acceleration_y": 32.0, "acceleration_z": -51.0, "gyroscope_x": -85.0, "gyroscope_y": 71.0, "gyroscope_z": -167.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:26.242000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 289.0, "acceleration_y": 47.0, "acceleration_z": -58.0, "gyroscope_x": -88.0, "gyroscope_y": 72.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:26.294000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 302.0, "acceleration_y": 34.0, "acceleration_z": -46.0, "gyroscope_x": -84.0, "gyroscope_y": 99.0, "gyroscope_z": 126.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:26.346000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": -60.0, "acceleration_z": 16.0, "gyroscope_x": -121.0, "gyroscope_y": 32.0, "gyroscope_z": 228.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:26.398000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 65.0, "acceleration_z": -51.0, "gyroscope_x": -50.0, "gyroscope_y": -85.0, "gyroscope_z": 162.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:32.077000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 372.0, "acceleration_y": 11.0, "acceleration_z": -230.0, "gyroscope_x": -536.0, "gyroscope_y": -161.0, "gyroscope_z": -80.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:32.129000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 342.0, "acceleration_y": 30.0, "acceleration_z": 91.0, "gyroscope_x": -683.0, "gyroscope_y": 43.0, "gyroscope_z": -152.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:32.181000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 54.0, "acceleration_z": -59.0, "gyroscope_x": -478.0, "gyroscope_y": -202.0, "gyroscope_z": -247.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:32.233000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": -24.0, "acceleration_z": 39.0, "gyroscope_x": -586.0, "gyroscope_y": -99.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:37.818000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": -60.0, "acceleration_z": -51.0, "gyroscope_x": 149.0, "gyroscope_y": 149.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:37.870000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 48.0, "acceleration_z": -33.0, "gyroscope_x": -59.0, "gyroscope_y": 114.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:37.922000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": 38.0, "acceleration_z": -8.0, "gyroscope_x": 5.0, "gyroscope_y": 127.0, "gyroscope_z": -89.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:37.974000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 337.0, "acceleration_y": 24.0, "acceleration_z": -131.0, "gyroscope_x": -75.0, "gyroscope_y": -161.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:43.538000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 207.0, "acceleration_y": -19.0, "acceleration_z": -24.0, "gyroscope_x": -211.0, "gyroscope_y": 26.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:43.590000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 56.0, "acceleration_z": -33.0, "gyroscope_x": -175.0, "gyroscope_y": 37.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:43.642000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 368.0, "acceleration_y": 78.0, "acceleration_z": -70.0, "gyroscope_x": -45.0, "gyroscope_y": -172.0, "gyroscope_z": -81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:43.694000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 216.0, "acceleration_y": -61.0, "acceleration_z": -136.0, "gyroscope_x": 100.0, "gyroscope_y": -10.0, "gyroscope_z": -55.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:49.212000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": -12.0, "acceleration_z": -21.0, "gyroscope_x": -153.0, "gyroscope_y": 20.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:49.264000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": -27.0, "acceleration_z": 47.0, "gyroscope_x": -289.0, "gyroscope_y": 155.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:49.316000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 41.0, "acceleration_z": -41.0, "gyroscope_x": -205.0, "gyroscope_y": 75.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:49.368000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 44.0, "acceleration_z": -45.0, "gyroscope_x": -145.0, "gyroscope_y": 101.0, "gyroscope_z": -67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:55.056000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 17.0, "acceleration_z": -98.0, "gyroscope_x": 157.0, "gyroscope_y": 192.0, "gyroscope_z": 67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:55.108000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 53.0, "acceleration_z": -59.0, "gyroscope_x": 153.0, "gyroscope_y": 161.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:55.160000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": -29.0, "acceleration_z": -48.0, "gyroscope_x": 108.0, "gyroscope_y": -96.0, "gyroscope_z": 64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:34:55.212000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 79.0, "acceleration_z": 41.0, "gyroscope_x": -25.0, "gyroscope_y": -23.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:01.074000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 221.0, "acceleration_y": 26.0, "acceleration_z": -88.0, "gyroscope_x": 158.0, "gyroscope_y": 98.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:01.126000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 16.0, "acceleration_z": -156.0, "gyroscope_x": 172.0, "gyroscope_y": 248.0, "gyroscope_z": 86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:01.178000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 305.0, "acceleration_y": -1.0, "acceleration_z": -117.0, "gyroscope_x": 117.0, "gyroscope_y": -118.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:01.230000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 5.0, "acceleration_z": 92.0, "gyroscope_x": -28.0, "gyroscope_y": 254.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:06.735000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 38.0, "acceleration_z": -69.0, "gyroscope_x": -132.0, "gyroscope_y": -12.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:06.787000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 60.0, "acceleration_z": 40.0, "gyroscope_x": -271.0, "gyroscope_y": -116.0, "gyroscope_z": 68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:06.839000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 38.0, "acceleration_z": -80.0, "gyroscope_x": -62.0, "gyroscope_y": 130.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:06.891000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 20.0, "acceleration_z": -89.0, "gyroscope_x": -147.0, "gyroscope_y": 145.0, "gyroscope_z": 40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:12.426000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 376.0, "acceleration_y": 51.0, "acceleration_z": -181.0, "gyroscope_x": 14.0, "gyroscope_y": 301.0, "gyroscope_z": -344.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:12.478000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 117.0, "acceleration_z": -115.0, "gyroscope_x": 33.0, "gyroscope_y": -110.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:12.530000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 182.0, "acceleration_y": -39.0, "acceleration_z": -135.0, "gyroscope_x": 271.0, "gyroscope_y": 220.0, "gyroscope_z": -147.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:12.582000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 125.0, "acceleration_y": 9.0, "acceleration_z": 54.0, "gyroscope_x": -42.0, "gyroscope_y": 117.0, "gyroscope_z": -317.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:18.105000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 44.0, "acceleration_z": 5.0, "gyroscope_x": -316.0, "gyroscope_y": 26.0, "gyroscope_z": 68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:18.157000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": 21.0, "acceleration_z": -123.0, "gyroscope_x": -207.0, "gyroscope_y": -162.0, "gyroscope_z": 41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:18.209000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": -10.0, "acceleration_z": -57.0, "gyroscope_x": -140.0, "gyroscope_y": -131.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:18.261000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 88.0, "acceleration_z": -125.0, "gyroscope_x": -209.0, "gyroscope_y": -262.0, "gyroscope_z": 49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:23.783000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 26.0, "acceleration_z": 31.0, "gyroscope_x": -34.0, "gyroscope_y": -56.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:23.835000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 26.0, "acceleration_z": -11.0, "gyroscope_x": 25.0, "gyroscope_y": 49.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:23.887000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 14.0, "acceleration_z": 56.0, "gyroscope_x": -73.0, "gyroscope_y": 85.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:23.939000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 7.0, "acceleration_z": -13.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:29.758000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 36.0, "acceleration_z": -38.0, "gyroscope_x": -21.0, "gyroscope_y": 25.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:29.810000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 35.0, "acceleration_z": -39.0, "gyroscope_x": -18.0, "gyroscope_y": 37.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:29.862000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 32.0, "acceleration_z": -25.0, "gyroscope_x": -3.0, "gyroscope_y": 17.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:29.914000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 39.0, "acceleration_z": -31.0, "gyroscope_x": 7.0, "gyroscope_y": 17.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:35.439000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 38.0, "acceleration_z": -33.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:35.491000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 41.0, "acceleration_z": -26.0, "gyroscope_x": -21.0, "gyroscope_y": 57.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:35.543000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 40.0, "acceleration_z": -30.0, "gyroscope_x": -16.0, "gyroscope_y": 53.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:35.595000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 42.0, "acceleration_z": -33.0, "gyroscope_x": -12.0, "gyroscope_y": 42.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:41.626000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": 27.0, "acceleration_z": -11.0, "gyroscope_x": -206.0, "gyroscope_y": 96.0, "gyroscope_z": -52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:41.678000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 40.0, "acceleration_z": -55.0, "gyroscope_x": -131.0, "gyroscope_y": 118.0, "gyroscope_z": 76.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:41.730000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 21.0, "acceleration_z": 14.0, "gyroscope_x": -158.0, "gyroscope_y": 153.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:41.782000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 40.0, "acceleration_z": -56.0, "gyroscope_x": -135.0, "gyroscope_y": 179.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:47.322000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 375.0, "acceleration_y": 72.0, "acceleration_z": -107.0, "gyroscope_x": 470.0, "gyroscope_y": 68.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:47.374000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 182.0, "acceleration_y": 23.0, "acceleration_z": 31.0, "gyroscope_x": 261.0, "gyroscope_y": 229.0, "gyroscope_z": 195.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:47.426000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 72.0, "acceleration_y": -27.0, "acceleration_z": 39.0, "gyroscope_x": 200.0, "gyroscope_y": -78.0, "gyroscope_z": -108.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:47.478000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 25.0, "acceleration_y": -5.0, "acceleration_z": 122.0, "gyroscope_x": 278.0, "gyroscope_y": -107.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:53.193000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 21.0, "acceleration_z": -49.0, "gyroscope_x": -14.0, "gyroscope_y": -36.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:53.245000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 27.0, "acceleration_z": -116.0, "gyroscope_x": -52.0, "gyroscope_y": -173.0, "gyroscope_z": -27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:53.297000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -82.0, "gyroscope_x": 82.0, "gyroscope_y": -43.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:53.349000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 6.0, "acceleration_z": -36.0, "gyroscope_x": 166.0, "gyroscope_y": -24.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:58.885000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 88.0, "acceleration_z": -104.0, "gyroscope_x": -228.0, "gyroscope_y": -185.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:58.937000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 24.0, "acceleration_z": 39.0, "gyroscope_x": -357.0, "gyroscope_y": -214.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:58.989000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 30.0, "acceleration_z": -40.0, "gyroscope_x": -150.0, "gyroscope_y": -201.0, "gyroscope_z": 36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:35:59.041000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 25.0, "acceleration_z": -52.0, "gyroscope_x": 4.0, "gyroscope_y": -251.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:04.879000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 383.0, "acceleration_y": 21.0, "acceleration_z": -94.0, "gyroscope_x": -130.0, "gyroscope_y": 48.0, "gyroscope_z": 178.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:04.931000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 216.0, "acceleration_y": 8.0, "acceleration_z": 47.0, "gyroscope_x": 162.0, "gyroscope_y": 74.0, "gyroscope_z": -38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:04.983000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 47.0, "acceleration_z": -22.0, "gyroscope_x": 120.0, "gyroscope_y": 168.0, "gyroscope_z": 107.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:05.035000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 200.0, "acceleration_y": 31.0, "acceleration_z": 56.0, "gyroscope_x": 113.0, "gyroscope_y": 134.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:10.629000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": -9.0, "acceleration_z": 60.0, "gyroscope_x": -109.0, "gyroscope_y": -60.0, "gyroscope_z": 94.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:10.681000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 219.0, "acceleration_y": 26.0, "acceleration_z": -63.0, "gyroscope_x": 158.0, "gyroscope_y": 33.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:10.733000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 212.0, "acceleration_y": 36.0, "acceleration_z": -122.0, "gyroscope_x": 123.0, "gyroscope_y": 133.0, "gyroscope_z": 66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:10.785000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 29.0, "acceleration_z": -21.0, "gyroscope_x": 188.0, "gyroscope_y": 249.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:16.577000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 334.0, "acceleration_y": 49.0, "acceleration_z": -84.0, "gyroscope_x": -51.0, "gyroscope_y": 316.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:16.629000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 71.0, "acceleration_z": -49.0, "gyroscope_x": 41.0, "gyroscope_y": 126.0, "gyroscope_z": 84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:16.681000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 202.0, "acceleration_y": 32.0, "acceleration_z": -95.0, "gyroscope_x": 36.0, "gyroscope_y": 177.0, "gyroscope_z": 56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:16.733000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 19.0, "acceleration_z": -56.0, "gyroscope_x": 74.0, "gyroscope_y": 186.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:22.262000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 39.0, "acceleration_z": -56.0, "gyroscope_x": 22.0, "gyroscope_y": 195.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:22.314000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 303.0, "acceleration_y": -47.0, "acceleration_z": 24.0, "gyroscope_x": -82.0, "gyroscope_y": 336.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:22.366000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 4.0, "acceleration_z": -53.0, "gyroscope_x": 99.0, "gyroscope_y": 40.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:22.418000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 15.0, "acceleration_z": -80.0, "gyroscope_x": -49.0, "gyroscope_y": 103.0, "gyroscope_z": 67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:27.950000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 107.0, "acceleration_z": -146.0, "gyroscope_x": 4.0, "gyroscope_y": 89.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:28.002000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 188.0, "acceleration_y": -54.0, "acceleration_z": -48.0, "gyroscope_x": -31.0, "gyroscope_y": 230.0, "gyroscope_z": -144.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:28.054000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 184.0, "acceleration_y": 6.0, "acceleration_z": 6.0, "gyroscope_x": -131.0, "gyroscope_y": 128.0, "gyroscope_z": -332.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:28.106000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 56.0, "acceleration_z": -59.0, "gyroscope_x": 63.0, "gyroscope_y": 137.0, "gyroscope_z": -112.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:33.638000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 82.0, "acceleration_z": -29.0, "gyroscope_x": -298.0, "gyroscope_y": -58.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:33.690000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 49.0, "acceleration_z": 50.0, "gyroscope_x": -632.0, "gyroscope_y": 142.0, "gyroscope_z": -113.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:33.742000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 9.0, "acceleration_z": -57.0, "gyroscope_x": -316.0, "gyroscope_y": 143.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:33.794000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 20.0, "acceleration_z": 45.0, "gyroscope_x": -430.0, "gyroscope_y": -31.0, "gyroscope_z": -86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:39.795000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 205.0, "acceleration_y": 34.0, "acceleration_z": -5.0, "gyroscope_x": -179.0, "gyroscope_y": 92.0, "gyroscope_z": -93.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:39.847000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 321.0, "acceleration_y": 38.0, "acceleration_z": -2.0, "gyroscope_x": -125.0, "gyroscope_y": -28.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:39.899000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 30.0, "acceleration_z": -127.0, "gyroscope_x": -116.0, "gyroscope_y": 112.0, "gyroscope_z": 28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:39.951000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 58.0, "acceleration_z": -11.0, "gyroscope_x": -159.0, "gyroscope_y": 188.0, "gyroscope_z": -66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:45.619000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 59.0, "acceleration_z": -116.0, "gyroscope_x": -29.0, "gyroscope_y": -17.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:45.671000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 12.0, "acceleration_z": -35.0, "gyroscope_x": -35.0, "gyroscope_y": -54.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:45.723000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 33.0, "acceleration_z": -140.0, "gyroscope_x": 167.0, "gyroscope_y": -56.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:45.775000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 337.0, "acceleration_y": 46.0, "acceleration_z": -140.0, "gyroscope_x": 50.0, "gyroscope_y": 53.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:51.316000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 302.0, "acceleration_y": 28.0, "acceleration_z": -58.0, "gyroscope_x": 142.0, "gyroscope_y": 27.0, "gyroscope_z": 344.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:51.368000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 219.0, "acceleration_y": -172.0, "acceleration_z": 41.0, "gyroscope_x": -68.0, "gyroscope_y": 106.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:51.420000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 314.0, "acceleration_y": 150.0, "acceleration_z": -58.0, "gyroscope_x": 152.0, "gyroscope_y": -106.0, "gyroscope_z": -222.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:51.472000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 27.0, "acceleration_z": -97.0, "gyroscope_x": 197.0, "gyroscope_y": -175.0, "gyroscope_z": -203.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:57.123000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 33.0, "acceleration_z": 55.0, "gyroscope_x": 64.0, "gyroscope_y": 56.0, "gyroscope_z": -40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:57.175000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 156.0, "acceleration_y": 10.0, "acceleration_z": 109.0, "gyroscope_x": -58.0, "gyroscope_y": 337.0, "gyroscope_z": -146.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:57.227000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": -7.0, "acceleration_z": 72.0, "gyroscope_x": 0.0, "gyroscope_y": 122.0, "gyroscope_z": 107.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:36:57.279000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": -45.0, "acceleration_z": -5.0, "gyroscope_x": -143.0, "gyroscope_y": 182.0, "gyroscope_z": -68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:02.840000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 22.0, "acceleration_z": -42.0, "gyroscope_x": -114.0, "gyroscope_y": 61.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:02.892000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 69.0, "acceleration_z": -40.0, "gyroscope_x": -108.0, "gyroscope_y": 70.0, "gyroscope_z": -79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:02.944000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 18.0, "acceleration_z": -167.0, "gyroscope_x": -159.0, "gyroscope_y": 102.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:02.996000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 29.0, "acceleration_z": -10.0, "gyroscope_x": -105.0, "gyroscope_y": -22.0, "gyroscope_z": -27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:08.575000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 5.0, "acceleration_z": 30.0, "gyroscope_x": 165.0, "gyroscope_y": -98.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:08.627000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 50.0, "acceleration_z": -14.0, "gyroscope_x": 128.0, "gyroscope_y": -29.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:08.679000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 46.0, "acceleration_z": 4.0, "gyroscope_x": 175.0, "gyroscope_y": 97.0, "gyroscope_z": 68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:08.731000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 19.0, "acceleration_z": 17.0, "gyroscope_x": 178.0, "gyroscope_y": -16.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:14.628000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 41.0, "acceleration_z": -15.0, "gyroscope_x": -206.0, "gyroscope_y": -71.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:14.680000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 24.0, "acceleration_z": -38.0, "gyroscope_x": -122.0, "gyroscope_y": -29.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:14.732000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 34.0, "acceleration_z": -39.0, "gyroscope_x": -117.0, "gyroscope_y": -45.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:14.784000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 34.0, "acceleration_z": -30.0, "gyroscope_x": -23.0, "gyroscope_y": 21.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:20.297000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 9.0, "acceleration_z": -45.0, "gyroscope_x": -830.0, "gyroscope_y": -293.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:20.349000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 27.0, "acceleration_z": -66.0, "gyroscope_x": -624.0, "gyroscope_y": -227.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:20.401000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 36.0, "acceleration_z": -48.0, "gyroscope_x": -613.0, "gyroscope_y": -213.0, "gyroscope_z": -59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:20.453000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 16.0, "acceleration_z": -79.0, "gyroscope_x": -271.0, "gyroscope_y": -85.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:26.109000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -7.0, "gyroscope_x": -16.0, "gyroscope_y": 79.0, "gyroscope_z": 5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:26.161000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 29.0, "acceleration_z": 9.0, "gyroscope_x": -25.0, "gyroscope_y": 73.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:26.213000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 33.0, "acceleration_z": -5.0, "gyroscope_x": -12.0, "gyroscope_y": 76.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:26.265000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 35.0, "acceleration_z": -11.0, "gyroscope_x": -23.0, "gyroscope_y": 85.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:31.874000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 20.0, "acceleration_z": 1.0, "gyroscope_x": -13.0, "gyroscope_y": 81.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:31.926000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 24.0, "acceleration_z": 7.0, "gyroscope_x": -20.0, "gyroscope_y": 80.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:31.978000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": 1.0, "gyroscope_x": -19.0, "gyroscope_y": 78.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:32.030000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": 4.0, "gyroscope_x": -17.0, "gyroscope_y": 82.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:37.544000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 25.0, "acceleration_z": 27.0, "gyroscope_x": -18.0, "gyroscope_y": 95.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:37.596000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 34.0, "acceleration_z": 24.0, "gyroscope_x": -8.0, "gyroscope_y": 79.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:37.648000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 27.0, "acceleration_z": 17.0, "gyroscope_x": -12.0, "gyroscope_y": 73.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:37.700000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 26.0, "acceleration_z": 1.0, "gyroscope_x": -30.0, "gyroscope_y": 129.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:43.226000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 16.0, "acceleration_z": -24.0, "gyroscope_x": -19.0, "gyroscope_y": 35.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:43.278000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 26.0, "acceleration_z": -35.0, "gyroscope_x": -28.0, "gyroscope_y": 83.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:43.330000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 35.0, "acceleration_z": -15.0, "gyroscope_x": -25.0, "gyroscope_y": 81.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:43.382000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 32.0, "acceleration_z": -14.0, "gyroscope_x": -28.0, "gyroscope_y": 113.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:48.907000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 25.0, "acceleration_z": -17.0, "gyroscope_x": -17.0, "gyroscope_y": 62.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:48.959000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 24.0, "acceleration_z": -13.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:49.011000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 23.0, "acceleration_z": -16.0, "gyroscope_x": -14.0, "gyroscope_y": 57.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:49.063000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 21.0, "acceleration_z": -11.0, "gyroscope_x": -10.0, "gyroscope_y": 70.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:54.612000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 8.0, "acceleration_z": -4.0, "gyroscope_x": -106.0, "gyroscope_y": 51.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:54.664000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 20.0, "acceleration_z": 21.0, "gyroscope_x": -133.0, "gyroscope_y": -213.0, "gyroscope_z": 104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:54.716000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 44.0, "acceleration_z": 82.0, "gyroscope_x": -296.0, "gyroscope_y": -161.0, "gyroscope_z": 81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:37:54.768000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 55.0, "acceleration_z": -137.0, "gyroscope_x": -310.0, "gyroscope_y": -126.0, "gyroscope_z": -68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:00.337000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 224.0, "acceleration_y": 33.0, "acceleration_z": 34.0, "gyroscope_x": -140.0, "gyroscope_y": -102.0, "gyroscope_z": -72.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:00.389000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 8.0, "acceleration_z": -71.0, "gyroscope_x": -132.0, "gyroscope_y": 81.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:00.441000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 24.0, "acceleration_z": -3.0, "gyroscope_x": -126.0, "gyroscope_y": -71.0, "gyroscope_z": 59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:00.493000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 344.0, "acceleration_y": -25.0, "acceleration_z": -19.0, "gyroscope_x": -213.0, "gyroscope_y": -334.0, "gyroscope_z": 247.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:06.023000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 400.0, "acceleration_y": 113.0, "acceleration_z": 87.0, "gyroscope_x": -702.0, "gyroscope_y": -5.0, "gyroscope_z": -166.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:06.075000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 376.0, "acceleration_y": 33.0, "acceleration_z": -35.0, "gyroscope_x": -566.0, "gyroscope_y": 73.0, "gyroscope_z": -155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:06.127000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 10.0, "acceleration_z": -140.0, "gyroscope_x": -492.0, "gyroscope_y": -5.0, "gyroscope_z": -233.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:06.179000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 91.0, "acceleration_z": -42.0, "gyroscope_x": -478.0, "gyroscope_y": 73.0, "gyroscope_z": -207.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:11.688000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 40.0, "acceleration_z": -68.0, "gyroscope_x": -26.0, "gyroscope_y": 303.0, "gyroscope_z": 46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:11.740000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 34.0, "acceleration_z": 32.0, "gyroscope_x": -293.0, "gyroscope_y": 160.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:11.792000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 48.0, "acceleration_z": -23.0, "gyroscope_x": -351.0, "gyroscope_y": 136.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:11.844000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 85.0, "acceleration_z": 26.0, "gyroscope_x": -504.0, "gyroscope_y": 232.0, "gyroscope_z": 14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:17.376000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 324.0, "acceleration_y": 150.0, "acceleration_z": -42.0, "gyroscope_x": -66.0, "gyroscope_y": 66.0, "gyroscope_z": 161.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:17.428000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 202.0, "acceleration_y": 40.0, "acceleration_z": -63.0, "gyroscope_x": -26.0, "gyroscope_y": -105.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:17.480000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 17.0, "acceleration_z": 84.0, "gyroscope_x": -162.0, "gyroscope_y": -101.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:17.532000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": -58.0, "acceleration_z": -339.0, "gyroscope_x": -69.0, "gyroscope_y": 167.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:23.272000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 45.0, "acceleration_z": -100.0, "gyroscope_x": 4.0, "gyroscope_y": -30.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:23.324000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 133.0, "acceleration_y": 5.0, "acceleration_z": 87.0, "gyroscope_x": 45.0, "gyroscope_y": 73.0, "gyroscope_z": 125.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:23.376000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 423.0, "acceleration_y": 57.0, "acceleration_z": -79.0, "gyroscope_x": 51.0, "gyroscope_y": -131.0, "gyroscope_z": -39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:23.428000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 79.0, "acceleration_y": -143.0, "acceleration_z": 84.0, "gyroscope_x": 160.0, "gyroscope_y": -89.0, "gyroscope_z": -203.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:29.087000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": 73.0, "acceleration_z": 20.0, "gyroscope_x": 33.0, "gyroscope_y": 139.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:29.139000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 399.0, "acceleration_y": 64.0, "acceleration_z": -30.0, "gyroscope_x": -54.0, "gyroscope_y": 358.0, "gyroscope_z": -135.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:29.191000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 340.0, "acceleration_y": -24.0, "acceleration_z": 42.0, "gyroscope_x": 8.0, "gyroscope_y": 303.0, "gyroscope_z": -121.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:29.243000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 314.0, "acceleration_y": 39.0, "acceleration_z": 16.0, "gyroscope_x": 50.0, "gyroscope_y": 99.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:35.101000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 11.0, "acceleration_z": 51.0, "gyroscope_x": -182.0, "gyroscope_y": -152.0, "gyroscope_z": 90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:35.153000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 326.0, "acceleration_y": 59.0, "acceleration_z": -70.0, "gyroscope_x": -59.0, "gyroscope_y": -112.0, "gyroscope_z": 101.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:35.205000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 326.0, "acceleration_y": -35.0, "acceleration_z": 14.0, "gyroscope_x": -228.0, "gyroscope_y": 208.0, "gyroscope_z": 153.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:35.257000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 148.0, "acceleration_y": -1.0, "acceleration_z": -39.0, "gyroscope_x": -70.0, "gyroscope_y": 82.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:40.833000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 191.0, "acceleration_y": 15.0, "acceleration_z": 125.0, "gyroscope_x": -281.0, "gyroscope_y": 132.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:40.885000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 365.0, "acceleration_y": 47.0, "acceleration_z": -76.0, "gyroscope_x": -221.0, "gyroscope_y": 28.0, "gyroscope_z": -47.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:40.937000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 391.0, "acceleration_y": 22.0, "acceleration_z": -88.0, "gyroscope_x": -282.0, "gyroscope_y": -161.0, "gyroscope_z": 76.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:40.989000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 30.0, "acceleration_z": 49.0, "gyroscope_x": -99.0, "gyroscope_y": -145.0, "gyroscope_z": 110.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:47.994000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 297.0, "acceleration_y": 28.0, "acceleration_z": -80.0, "gyroscope_x": -81.0, "gyroscope_y": 52.0, "gyroscope_z": 59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:48.046000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 30.0, "acceleration_z": -171.0, "gyroscope_x": -72.0, "gyroscope_y": 126.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:48.098000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 82.0, "acceleration_z": -74.0, "gyroscope_x": -262.0, "gyroscope_y": -120.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:48.150000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 319.0, "acceleration_y": 34.0, "acceleration_z": -93.0, "gyroscope_x": -244.0, "gyroscope_y": -10.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:53.711000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 0.0, "acceleration_z": -179.0, "gyroscope_x": 122.0, "gyroscope_y": 65.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:53.763000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": 49.0, "acceleration_z": -9.0, "gyroscope_x": 62.0, "gyroscope_y": 210.0, "gyroscope_z": -114.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:53.815000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 45.0, "acceleration_z": 44.0, "gyroscope_x": -40.0, "gyroscope_y": 153.0, "gyroscope_z": 7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:53.867000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": 2.0, "acceleration_z": 35.0, "gyroscope_x": -60.0, "gyroscope_y": 59.0, "gyroscope_z": -199.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:59.393000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 34.0, "acceleration_z": -67.0, "gyroscope_x": 39.0, "gyroscope_y": 82.0, "gyroscope_z": -74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:59.445000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 48.0, "acceleration_z": -108.0, "gyroscope_x": 112.0, "gyroscope_y": 260.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:59.497000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 301.0, "acceleration_y": 22.0, "acceleration_z": 48.0, "gyroscope_x": -102.0, "gyroscope_y": -13.0, "gyroscope_z": 79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:38:59.549000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 57.0, "acceleration_z": -22.0, "gyroscope_x": 76.0, "gyroscope_y": 9.0, "gyroscope_z": -69.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:05.202000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": -5.0, "acceleration_z": -13.0, "gyroscope_x": -53.0, "gyroscope_y": -145.0, "gyroscope_z": -138.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:05.254000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 170.0, "acceleration_y": -34.0, "acceleration_z": -6.0, "gyroscope_x": -230.0, "gyroscope_y": -14.0, "gyroscope_z": 28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:05.306000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 511.0, "acceleration_y": 63.0, "acceleration_z": 0.0, "gyroscope_x": 97.0, "gyroscope_y": -296.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:05.358000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": -35.0, "acceleration_z": -7.0, "gyroscope_x": -35.0, "gyroscope_y": 362.0, "gyroscope_z": 327.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:10.835000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 79.0, "acceleration_z": 34.0, "gyroscope_x": -36.0, "gyroscope_y": 63.0, "gyroscope_z": 150.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:10.887000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 205.0, "acceleration_y": -4.0, "acceleration_z": -121.0, "gyroscope_x": -2.0, "gyroscope_y": 228.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:10.939000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 23.0, "acceleration_z": -159.0, "gyroscope_x": -39.0, "gyroscope_y": 277.0, "gyroscope_z": -104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:10.991000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 179.0, "acceleration_y": 39.0, "acceleration_z": -174.0, "gyroscope_x": 33.0, "gyroscope_y": 131.0, "gyroscope_z": -91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:16.502000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 32.0, "acceleration_z": -100.0, "gyroscope_x": -52.0, "gyroscope_y": -63.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:16.554000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 14.0, "acceleration_z": -104.0, "gyroscope_x": 28.0, "gyroscope_y": -102.0, "gyroscope_z": -116.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:16.606000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 60.0, "acceleration_z": -90.0, "gyroscope_x": 52.0, "gyroscope_y": 84.0, "gyroscope_z": 88.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:16.658000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 43.0, "acceleration_z": 12.0, "gyroscope_x": 130.0, "gyroscope_y": 69.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:22.268000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 34.0, "acceleration_z": -58.0, "gyroscope_x": 193.0, "gyroscope_y": 68.0, "gyroscope_z": 89.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:22.320000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 94.0, "acceleration_z": -6.0, "gyroscope_x": 145.0, "gyroscope_y": 123.0, "gyroscope_z": 97.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:22.372000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 60.0, "acceleration_z": 23.0, "gyroscope_x": 226.0, "gyroscope_y": 72.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:22.424000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": -56.0, "acceleration_z": -50.0, "gyroscope_x": 200.0, "gyroscope_y": 327.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:27.934000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 321.0, "acceleration_y": 63.0, "acceleration_z": -7.0, "gyroscope_x": -194.0, "gyroscope_y": -29.0, "gyroscope_z": 88.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:27.986000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 183.0, "acceleration_y": 73.0, "acceleration_z": 143.0, "gyroscope_x": -175.0, "gyroscope_y": -169.0, "gyroscope_z": -90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:28.038000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 7.0, "acceleration_z": -8.0, "gyroscope_x": -232.0, "gyroscope_y": -25.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:28.090000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 142.0, "acceleration_y": 43.0, "acceleration_z": 48.0, "gyroscope_x": -272.0, "gyroscope_y": -155.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:33.626000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 7.0, "acceleration_z": -11.0, "gyroscope_x": 59.0, "gyroscope_y": 112.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:33.678000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 52.0, "acceleration_z": 43.0, "gyroscope_x": 72.0, "gyroscope_y": 181.0, "gyroscope_z": -119.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:33.730000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 27.0, "acceleration_z": -126.0, "gyroscope_x": 39.0, "gyroscope_y": 101.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:33.782000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 6.0, "acceleration_z": 46.0, "gyroscope_x": -46.0, "gyroscope_y": 251.0, "gyroscope_z": -74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:39.422000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 66.0, "acceleration_z": -51.0, "gyroscope_x": -69.0, "gyroscope_y": 96.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:39.474000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 1.0, "acceleration_z": -103.0, "gyroscope_x": -30.0, "gyroscope_y": -2.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:39.526000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 40.0, "acceleration_z": -35.0, "gyroscope_x": -74.0, "gyroscope_y": -54.0, "gyroscope_z": -77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:39.578000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 19.0, "acceleration_z": -124.0, "gyroscope_x": -14.0, "gyroscope_y": 35.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:45.112000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 81.0, "acceleration_z": -168.0, "gyroscope_x": 209.0, "gyroscope_y": 219.0, "gyroscope_z": 38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:45.164000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 81.0, "acceleration_z": -145.0, "gyroscope_x": 109.0, "gyroscope_y": 34.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:45.216000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 7.0, "acceleration_z": -49.0, "gyroscope_x": 161.0, "gyroscope_y": -208.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:45.268000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 68.0, "acceleration_z": -26.0, "gyroscope_x": 167.0, "gyroscope_y": 135.0, "gyroscope_z": 60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:51.599000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 22.0, "acceleration_z": -8.0, "gyroscope_x": 155.0, "gyroscope_y": -23.0, "gyroscope_z": 153.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:51.651000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 472.0, "acceleration_y": -89.0, "acceleration_z": -15.0, "gyroscope_x": 228.0, "gyroscope_y": -271.0, "gyroscope_z": 138.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:51.703000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 9.0, "acceleration_z": -32.0, "gyroscope_x": 278.0, "gyroscope_y": 36.0, "gyroscope_z": 211.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:51.755000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": 41.0, "acceleration_z": -28.0, "gyroscope_x": 305.0, "gyroscope_y": 39.0, "gyroscope_z": 184.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:57.249000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": 29.0, "acceleration_z": -18.0, "gyroscope_x": -174.0, "gyroscope_y": 148.0, "gyroscope_z": -78.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:57.301000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 25.0, "acceleration_z": -138.0, "gyroscope_x": -127.0, "gyroscope_y": 117.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:57.353000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 46.0, "acceleration_z": 71.0, "gyroscope_x": -266.0, "gyroscope_y": -13.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:39:57.405000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 43.0, "acceleration_z": -144.0, "gyroscope_x": -107.0, "gyroscope_y": -156.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:02.938000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 435.0, "acceleration_y": 32.0, "acceleration_z": 97.0, "gyroscope_x": -164.0, "gyroscope_y": 51.0, "gyroscope_z": -52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:02.990000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 14.0, "acceleration_z": -3.0, "gyroscope_x": -245.0, "gyroscope_y": 51.0, "gyroscope_z": -68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:03.042000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 4.0, "acceleration_z": -22.0, "gyroscope_x": -282.0, "gyroscope_y": 15.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:03.094000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 184.0, "acceleration_y": 6.0, "acceleration_z": 74.0, "gyroscope_x": -436.0, "gyroscope_y": -101.0, "gyroscope_z": -116.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:08.740000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 10.0, "acceleration_z": -23.0, "gyroscope_x": -20.0, "gyroscope_y": 54.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:08.792000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 2.0, "acceleration_z": -51.0, "gyroscope_x": -88.0, "gyroscope_y": 84.0, "gyroscope_z": 67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:08.844000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 20.0, "acceleration_z": -26.0, "gyroscope_x": -45.0, "gyroscope_y": -21.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:08.896000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 12.0, "acceleration_z": -113.0, "gyroscope_x": -43.0, "gyroscope_y": 37.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:14.401000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 41.0, "acceleration_z": -4.0, "gyroscope_x": -29.0, "gyroscope_y": 102.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:14.453000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 39.0, "acceleration_z": -13.0, "gyroscope_x": -15.0, "gyroscope_y": 106.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:14.505000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 40.0, "acceleration_z": -1.0, "gyroscope_x": -25.0, "gyroscope_y": 94.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:14.557000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 42.0, "acceleration_z": -8.0, "gyroscope_x": -15.0, "gyroscope_y": 90.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:20.070000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 30.0, "acceleration_z": 8.0, "gyroscope_x": -26.0, "gyroscope_y": -26.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:20.122000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 43.0, "acceleration_z": 24.0, "gyroscope_x": -43.0, "gyroscope_y": -69.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:20.174000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 53.0, "acceleration_z": -12.0, "gyroscope_x": -9.0, "gyroscope_y": -45.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:20.226000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 54.0, "acceleration_z": -10.0, "gyroscope_x": -2.0, "gyroscope_y": -5.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:25.768000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -11.0, "gyroscope_x": -18.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:25.820000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -10.0, "gyroscope_x": -18.0, "gyroscope_y": 63.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:25.872000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -12.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:25.924000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 39.0, "acceleration_z": -10.0, "gyroscope_x": -11.0, "gyroscope_y": 61.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:31.466000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -16.0, "gyroscope_x": -17.0, "gyroscope_y": 67.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:31.518000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 40.0, "acceleration_z": -15.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:31.570000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 41.0, "acceleration_z": -13.0, "gyroscope_x": -16.0, "gyroscope_y": 69.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:31.622000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 40.0, "acceleration_z": -14.0, "gyroscope_x": -19.0, "gyroscope_y": 73.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:37.130000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 41.0, "acceleration_z": -11.0, "gyroscope_x": -13.0, "gyroscope_y": 58.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:37.182000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 39.0, "acceleration_z": -9.0, "gyroscope_x": -13.0, "gyroscope_y": 58.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:37.234000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 41.0, "acceleration_z": -12.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:37.286000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 39.0, "acceleration_z": -10.0, "gyroscope_x": -17.0, "gyroscope_y": 61.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:42.798000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 36.0, "acceleration_z": -11.0, "gyroscope_x": -23.0, "gyroscope_y": 83.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:42.850000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 38.0, "acceleration_z": -10.0, "gyroscope_x": -17.0, "gyroscope_y": 70.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:42.902000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -10.0, "gyroscope_x": -18.0, "gyroscope_y": 73.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:42.954000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 36.0, "acceleration_z": -9.0, "gyroscope_x": -18.0, "gyroscope_y": 59.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:48.481000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 40.0, "acceleration_z": -18.0, "gyroscope_x": -18.0, "gyroscope_y": 62.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:48.533000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 42.0, "acceleration_z": -19.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:48.585000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 43.0, "acceleration_z": -18.0, "gyroscope_x": -12.0, "gyroscope_y": 59.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:48.637000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 42.0, "acceleration_z": -19.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:54.114000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 42.0, "acceleration_z": -17.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:54.166000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 41.0, "acceleration_z": -17.0, "gyroscope_x": -17.0, "gyroscope_y": 63.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:54.218000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 40.0, "acceleration_z": -17.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:54.270000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 41.0, "acceleration_z": -17.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:59.793000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 12.0, "acceleration_z": -40.0, "gyroscope_x": -9.0, "gyroscope_y": 101.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:59.845000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 27.0, "acceleration_z": -21.0, "gyroscope_x": -30.0, "gyroscope_y": 117.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:59.897000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -20.0, "gyroscope_x": -31.0, "gyroscope_y": 124.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:40:59.949000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -28.0, "gyroscope_x": -7.0, "gyroscope_y": 138.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:05.603000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 45.0, "acceleration_z": -22.0, "gyroscope_x": -23.0, "gyroscope_y": 101.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:05.655000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 38.0, "acceleration_z": -24.0, "gyroscope_x": -21.0, "gyroscope_y": 95.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:05.707000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 36.0, "acceleration_z": -18.0, "gyroscope_x": -34.0, "gyroscope_y": 95.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:05.759000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 41.0, "acceleration_z": -21.0, "gyroscope_x": -20.0, "gyroscope_y": 91.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:11.257000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -22.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:11.309000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -19.0, "gyroscope_x": -13.0, "gyroscope_y": 61.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:11.361000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -18.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:11.413000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 34.0, "acceleration_z": -20.0, "gyroscope_x": -14.0, "gyroscope_y": 61.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:17.072000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 41.0, "acceleration_z": -22.0, "gyroscope_x": -13.0, "gyroscope_y": 68.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:17.124000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 39.0, "acceleration_z": -23.0, "gyroscope_x": -12.0, "gyroscope_y": 73.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:17.176000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -19.0, "gyroscope_x": -18.0, "gyroscope_y": 69.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:17.228000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -17.0, "gyroscope_x": -22.0, "gyroscope_y": 70.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:22.931000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -25.0, "gyroscope_x": -10.0, "gyroscope_y": 67.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:22.983000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -20.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:23.035000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 38.0, "acceleration_z": -25.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:23.087000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 38.0, "acceleration_z": -25.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:28.960000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 54.0, "acceleration_z": 1.0, "gyroscope_x": -32.0, "gyroscope_y": 118.0, "gyroscope_z": 40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:29.012000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 54.0, "acceleration_z": 3.0, "gyroscope_x": -41.0, "gyroscope_y": 35.0, "gyroscope_z": 49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:29.064000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 66.0, "acceleration_z": -19.0, "gyroscope_x": -1.0, "gyroscope_y": 110.0, "gyroscope_z": 58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:29.116000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 52.0, "acceleration_z": 9.0, "gyroscope_x": 36.0, "gyroscope_y": 86.0, "gyroscope_z": 48.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:34.681000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 42.0, "acceleration_z": 19.0, "gyroscope_x": 227.0, "gyroscope_y": 159.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:34.733000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 44.0, "acceleration_z": -1.0, "gyroscope_x": 91.0, "gyroscope_y": 360.0, "gyroscope_z": -57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:34.785000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 18.0, "acceleration_z": -11.0, "gyroscope_x": 220.0, "gyroscope_y": 127.0, "gyroscope_z": -155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:34.837000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 10.0, "acceleration_z": -3.0, "gyroscope_x": 5.0, "gyroscope_y": 130.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:40.548000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 2.0, "acceleration_z": -39.0, "gyroscope_x": -89.0, "gyroscope_y": -121.0, "gyroscope_z": -144.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:40.600000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 15.0, "acceleration_z": 21.0, "gyroscope_x": -11.0, "gyroscope_y": -85.0, "gyroscope_z": -64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:40.652000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": -14.0, "acceleration_z": -55.0, "gyroscope_x": 43.0, "gyroscope_y": -30.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:40.704000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 81.0, "acceleration_z": -57.0, "gyroscope_x": 119.0, "gyroscope_y": -9.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:46.240000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 44.0, "acceleration_z": -30.0, "gyroscope_x": -29.0, "gyroscope_y": 121.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:46.292000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 31.0, "acceleration_z": 17.0, "gyroscope_x": -23.0, "gyroscope_y": 106.0, "gyroscope_z": -110.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:46.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 87.0, "acceleration_z": -21.0, "gyroscope_x": 47.0, "gyroscope_y": -83.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:46.396000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 26.0, "acceleration_z": -81.0, "gyroscope_x": -2.0, "gyroscope_y": 129.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:51.922000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 331.0, "acceleration_y": 94.0, "acceleration_z": -36.0, "gyroscope_x": 296.0, "gyroscope_y": 9.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:51.974000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 201.0, "acceleration_y": -8.0, "acceleration_z": 81.0, "gyroscope_x": 383.0, "gyroscope_y": 126.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:52.026000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 25.0, "acceleration_z": -147.0, "gyroscope_x": 300.0, "gyroscope_y": 151.0, "gyroscope_z": -116.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:52.078000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 82.0, "acceleration_z": -75.0, "gyroscope_x": 159.0, "gyroscope_y": 245.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:57.591000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 31.0, "acceleration_z": -73.0, "gyroscope_x": -186.0, "gyroscope_y": 14.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:57.643000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 47.0, "acceleration_z": 2.0, "gyroscope_x": -317.0, "gyroscope_y": -40.0, "gyroscope_z": -91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:57.695000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 28.0, "acceleration_z": -15.0, "gyroscope_x": -440.0, "gyroscope_y": -34.0, "gyroscope_z": -119.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:41:57.747000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 112.0, "acceleration_z": -102.0, "gyroscope_x": -308.0, "gyroscope_y": 36.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:03.288000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 90.0, "acceleration_y": -77.0, "acceleration_z": -63.0, "gyroscope_x": 149.0, "gyroscope_y": 109.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:03.340000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 377.0, "acceleration_y": 185.0, "acceleration_z": -111.0, "gyroscope_x": 41.0, "gyroscope_y": 204.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:03.392000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 139.0, "acceleration_y": -1.0, "acceleration_z": -137.0, "gyroscope_x": 216.0, "gyroscope_y": 205.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:03.444000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 68.0, "acceleration_z": 1.0, "gyroscope_x": 130.0, "gyroscope_y": 144.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:09.001000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 316.0, "acceleration_y": 55.0, "acceleration_z": -66.0, "gyroscope_x": -147.0, "gyroscope_y": -170.0, "gyroscope_z": -65.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:09.053000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 11.0, "acceleration_z": 55.0, "gyroscope_x": -228.0, "gyroscope_y": 38.0, "gyroscope_z": -105.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:09.105000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 26.0, "acceleration_z": -28.0, "gyroscope_x": -71.0, "gyroscope_y": -56.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:09.157000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 32.0, "acceleration_z": 29.0, "gyroscope_x": -124.0, "gyroscope_y": 55.0, "gyroscope_z": 48.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:14.854000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 32.0, "acceleration_z": -7.0, "gyroscope_x": 170.0, "gyroscope_y": -147.0, "gyroscope_z": -85.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:14.906000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 223.0, "acceleration_y": 52.0, "acceleration_z": -150.0, "gyroscope_x": 343.0, "gyroscope_y": -431.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:14.958000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 28.0, "acceleration_z": -22.0, "gyroscope_x": 503.0, "gyroscope_y": -156.0, "gyroscope_z": -139.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:15.010000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 46.0, "acceleration_z": -150.0, "gyroscope_x": 732.0, "gyroscope_y": -84.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:20.535000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 209.0, "acceleration_y": 58.0, "acceleration_z": -57.0, "gyroscope_x": -363.0, "gyroscope_y": 36.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:20.587000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": -10.0, "acceleration_z": 35.0, "gyroscope_x": -401.0, "gyroscope_y": 237.0, "gyroscope_z": -124.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:20.639000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 5.0, "acceleration_z": 160.0, "gyroscope_x": -568.0, "gyroscope_y": 544.0, "gyroscope_z": -105.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:20.691000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 295.0, "acceleration_y": 43.0, "acceleration_z": -13.0, "gyroscope_x": -357.0, "gyroscope_y": -24.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:26.550000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 61.0, "acceleration_z": 112.0, "gyroscope_x": 586.0, "gyroscope_y": 349.0, "gyroscope_z": -223.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:26.602000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 217.0, "acceleration_y": 41.0, "acceleration_z": 1.0, "gyroscope_x": 621.0, "gyroscope_y": 274.0, "gyroscope_z": -290.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:26.654000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -4.0, "gyroscope_x": 354.0, "gyroscope_y": 291.0, "gyroscope_z": -172.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:26.706000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 5.0, "acceleration_z": -40.0, "gyroscope_x": 156.0, "gyroscope_y": 138.0, "gyroscope_z": -187.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:32.255000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 324.0, "acceleration_y": 73.0, "acceleration_z": -162.0, "gyroscope_x": -122.0, "gyroscope_y": 195.0, "gyroscope_z": 61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:32.307000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": 20.0, "acceleration_z": -112.0, "gyroscope_x": -127.0, "gyroscope_y": -155.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:32.359000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 58.0, "acceleration_z": 82.0, "gyroscope_x": -179.0, "gyroscope_y": 115.0, "gyroscope_z": -90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:32.411000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": -11.0, "acceleration_z": 106.0, "gyroscope_x": -264.0, "gyroscope_y": 273.0, "gyroscope_z": -82.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:38.099000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 50.0, "acceleration_y": 19.0, "acceleration_z": 15.0, "gyroscope_x": 109.0, "gyroscope_y": 43.0, "gyroscope_z": 179.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:38.151000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 57.0, "acceleration_y": 57.0, "acceleration_z": -86.0, "gyroscope_x": 288.0, "gyroscope_y": 50.0, "gyroscope_z": -221.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:38.203000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": 138.0, "acceleration_z": -105.0, "gyroscope_x": 286.0, "gyroscope_y": 47.0, "gyroscope_z": 121.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:38.255000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 137.0, "acceleration_y": -239.0, "acceleration_z": 14.0, "gyroscope_x": 357.0, "gyroscope_y": 161.0, "gyroscope_z": -76.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:43.851000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -4.0, "acceleration_z": -36.0, "gyroscope_x": 465.0, "gyroscope_y": 149.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:43.903000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 313.0, "acceleration_y": 2.0, "acceleration_z": -136.0, "gyroscope_x": 439.0, "gyroscope_y": 7.0, "gyroscope_z": 34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:43.955000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 14.0, "acceleration_z": -89.0, "gyroscope_x": 573.0, "gyroscope_y": 90.0, "gyroscope_z": -109.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:44.007000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 10.0, "acceleration_z": 20.0, "gyroscope_x": 576.0, "gyroscope_y": 22.0, "gyroscope_z": -96.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:49.563000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 320.0, "acceleration_y": 80.0, "acceleration_z": -57.0, "gyroscope_x": -204.0, "gyroscope_y": -132.0, "gyroscope_z": 55.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:49.615000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 26.0, "acceleration_z": -132.0, "gyroscope_x": 3.0, "gyroscope_y": -11.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:49.667000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 35.0, "acceleration_z": -73.0, "gyroscope_x": -95.0, "gyroscope_y": 76.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:49.719000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 78.0, "acceleration_z": -129.0, "gyroscope_x": 37.0, "gyroscope_y": 119.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:55.304000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 197.0, "acceleration_y": 18.0, "acceleration_z": -41.0, "gyroscope_x": -87.0, "gyroscope_y": -101.0, "gyroscope_z": -47.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:55.356000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 18.0, "acceleration_z": -38.0, "gyroscope_x": -54.0, "gyroscope_y": 11.0, "gyroscope_z": -123.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:55.408000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -33.0, "gyroscope_x": -124.0, "gyroscope_y": 53.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:42:55.460000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 4.0, "acceleration_z": -46.0, "gyroscope_x": 8.0, "gyroscope_y": -72.0, "gyroscope_z": 86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:00.968000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 49.0, "acceleration_z": -6.0, "gyroscope_x": 196.0, "gyroscope_y": 134.0, "gyroscope_z": -113.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:01.020000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 27.0, "acceleration_z": -39.0, "gyroscope_x": 283.0, "gyroscope_y": 207.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:01.072000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 25.0, "acceleration_z": -13.0, "gyroscope_x": 383.0, "gyroscope_y": 77.0, "gyroscope_z": -71.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:01.124000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 306.0, "acceleration_y": 46.0, "acceleration_z": -60.0, "gyroscope_x": 345.0, "gyroscope_y": 145.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:06.686000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 360.0, "acceleration_y": -8.0, "acceleration_z": -13.0, "gyroscope_x": 140.0, "gyroscope_y": -162.0, "gyroscope_z": -100.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:06.738000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 219.0, "acceleration_y": 5.0, "acceleration_z": -66.0, "gyroscope_x": 219.0, "gyroscope_y": 37.0, "gyroscope_z": -44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:06.790000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 5.0, "acceleration_z": -177.0, "gyroscope_x": 172.0, "gyroscope_y": 106.0, "gyroscope_z": -64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:06.842000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 30.0, "acceleration_z": -70.0, "gyroscope_x": 325.0, "gyroscope_y": 269.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:12.402000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 183.0, "acceleration_y": 28.0, "acceleration_z": 18.0, "gyroscope_x": 53.0, "gyroscope_y": 26.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:12.454000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": 21.0, "acceleration_z": -22.0, "gyroscope_x": 94.0, "gyroscope_y": -36.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:12.506000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 43.0, "acceleration_z": -112.0, "gyroscope_x": 28.0, "gyroscope_y": 66.0, "gyroscope_z": -95.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:12.558000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": 30.0, "acceleration_z": -139.0, "gyroscope_x": 16.0, "gyroscope_y": -115.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:18.105000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": -18.0, "acceleration_z": -76.0, "gyroscope_x": 168.0, "gyroscope_y": 119.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:18.157000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 434.0, "acceleration_y": -18.0, "acceleration_z": 101.0, "gyroscope_x": 146.0, "gyroscope_y": 87.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:18.209000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 201.0, "acceleration_y": 16.0, "acceleration_z": -27.0, "gyroscope_x": 227.0, "gyroscope_y": -111.0, "gyroscope_z": 64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:18.261000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 22.0, "acceleration_z": 81.0, "gyroscope_x": 221.0, "gyroscope_y": 121.0, "gyroscope_z": -79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:23.958000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 45.0, "acceleration_z": 36.0, "gyroscope_x": 391.0, "gyroscope_y": 2.0, "gyroscope_z": -185.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:24.010000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 355.0, "acceleration_y": 3.0, "acceleration_z": -67.0, "gyroscope_x": 454.0, "gyroscope_y": 215.0, "gyroscope_z": -99.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:24.062000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 342.0, "acceleration_y": 28.0, "acceleration_z": -20.0, "gyroscope_x": 405.0, "gyroscope_y": -5.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:24.114000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": 14.0, "acceleration_z": 2.0, "gyroscope_x": 184.0, "gyroscope_y": 286.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:29.710000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 17.0, "acceleration_z": -18.0, "gyroscope_x": 520.0, "gyroscope_y": -146.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:29.762000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 29.0, "acceleration_z": -79.0, "gyroscope_x": 615.0, "gyroscope_y": 122.0, "gyroscope_z": -66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:29.814000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 16.0, "acceleration_z": 39.0, "gyroscope_x": 501.0, "gyroscope_y": 104.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:29.866000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 75.0, "acceleration_z": 56.0, "gyroscope_x": 447.0, "gyroscope_y": 284.0, "gyroscope_z": -204.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:35.440000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": -29.0, "acceleration_z": -6.0, "gyroscope_x": 442.0, "gyroscope_y": -223.0, "gyroscope_z": -75.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:35.492000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 313.0, "acceleration_y": 52.0, "acceleration_z": -83.0, "gyroscope_x": 515.0, "gyroscope_y": -220.0, "gyroscope_z": -196.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:35.544000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 339.0, "acceleration_y": 16.0, "acceleration_z": -154.0, "gyroscope_x": 632.0, "gyroscope_y": -225.0, "gyroscope_z": -204.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:35.596000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 73.0, "acceleration_z": -16.0, "gyroscope_x": 646.0, "gyroscope_y": -95.0, "gyroscope_z": -142.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:41.152000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 1.0, "acceleration_z": 11.0, "gyroscope_x": 697.0, "gyroscope_y": -381.0, "gyroscope_z": -95.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:41.204000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": -48.0, "acceleration_z": -127.0, "gyroscope_x": 871.0, "gyroscope_y": -329.0, "gyroscope_z": -57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:41.256000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 65.0, "acceleration_z": -66.0, "gyroscope_x": 814.0, "gyroscope_y": 5.0, "gyroscope_z": -137.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:41.308000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 34.0, "acceleration_z": -22.0, "gyroscope_x": 909.0, "gyroscope_y": 170.0, "gyroscope_z": -199.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:47.226000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": -15.0, "acceleration_z": 32.0, "gyroscope_x": 104.0, "gyroscope_y": 138.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:47.278000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 8.0, "acceleration_z": -7.0, "gyroscope_x": 215.0, "gyroscope_y": 104.0, "gyroscope_z": 70.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:47.330000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": -7.0, "acceleration_z": -17.0, "gyroscope_x": 185.0, "gyroscope_y": 145.0, "gyroscope_z": 48.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:47.382000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 12.0, "acceleration_z": 15.0, "gyroscope_x": 180.0, "gyroscope_y": 137.0, "gyroscope_z": 70.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:53.273000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 6.0, "acceleration_z": -25.0, "gyroscope_x": -17.0, "gyroscope_y": 71.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:53.325000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 3.0, "acceleration_z": -24.0, "gyroscope_x": -18.0, "gyroscope_y": 67.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:53.377000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 3.0, "acceleration_z": -27.0, "gyroscope_x": -21.0, "gyroscope_y": 63.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:53.429000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 6.0, "acceleration_z": -33.0, "gyroscope_x": -18.0, "gyroscope_y": 67.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:59.093000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 20.0, "acceleration_z": -24.0, "gyroscope_x": 20.0, "gyroscope_y": 58.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:59.145000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 6.0, "acceleration_z": -40.0, "gyroscope_x": 2.0, "gyroscope_y": 44.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:59.197000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 0.0, "acceleration_z": -14.0, "gyroscope_x": -18.0, "gyroscope_y": 42.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:43:59.249000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 1.0, "acceleration_z": -16.0, "gyroscope_x": -7.0, "gyroscope_y": 60.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:05.852000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 11.0, "acceleration_z": -34.0, "gyroscope_x": -24.0, "gyroscope_y": 146.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:05.904000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 9.0, "acceleration_z": -29.0, "gyroscope_x": -26.0, "gyroscope_y": 139.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:05.956000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -21.0, "gyroscope_x": -28.0, "gyroscope_y": 124.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:06.008000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 4.0, "acceleration_z": -16.0, "gyroscope_x": -24.0, "gyroscope_y": 102.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:11.545000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -30.0, "gyroscope_x": -8.0, "gyroscope_y": 21.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:11.597000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -32.0, "gyroscope_x": -10.0, "gyroscope_y": 21.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:11.649000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -36.0, "gyroscope_x": -8.0, "gyroscope_y": 29.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:11.701000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 4.0, "acceleration_z": -38.0, "gyroscope_x": -8.0, "gyroscope_y": 41.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:17.379000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 5.0, "acceleration_z": -31.0, "gyroscope_x": -9.0, "gyroscope_y": 26.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:17.431000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 5.0, "acceleration_z": -38.0, "gyroscope_x": -6.0, "gyroscope_y": 31.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:17.483000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 3.0, "acceleration_z": -41.0, "gyroscope_x": -9.0, "gyroscope_y": 38.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:17.535000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 2.0, "acceleration_z": -39.0, "gyroscope_x": -11.0, "gyroscope_y": 49.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:23.071000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 7.0, "acceleration_z": -45.0, "gyroscope_x": -11.0, "gyroscope_y": 71.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:23.123000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 3.0, "acceleration_z": -43.0, "gyroscope_x": -14.0, "gyroscope_y": 88.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:23.175000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -34.0, "gyroscope_x": -16.0, "gyroscope_y": 98.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:23.227000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 11.0, "acceleration_z": -31.0, "gyroscope_x": -20.0, "gyroscope_y": 96.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:28.771000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 11.0, "acceleration_z": -36.0, "gyroscope_x": 2.0, "gyroscope_y": 5.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:28.823000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -37.0, "gyroscope_x": -9.0, "gyroscope_y": 13.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:28.875000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 4.0, "acceleration_z": -40.0, "gyroscope_x": -7.0, "gyroscope_y": 15.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:28.927000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 1.0, "acceleration_z": -41.0, "gyroscope_x": -8.0, "gyroscope_y": 20.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:34.554000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 4.0, "acceleration_z": -111.0, "gyroscope_x": 46.0, "gyroscope_y": -301.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:34.606000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 4.0, "acceleration_z": -56.0, "gyroscope_x": 94.0, "gyroscope_y": -227.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:34.658000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 8.0, "acceleration_z": -77.0, "gyroscope_x": 12.0, "gyroscope_y": -182.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:34.710000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 3.0, "acceleration_z": -131.0, "gyroscope_x": 78.0, "gyroscope_y": -148.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:40.258000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 7.0, "acceleration_z": -37.0, "gyroscope_x": -19.0, "gyroscope_y": 75.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:40.310000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -38.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:40.362000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 7.0, "acceleration_z": -41.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:40.414000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 8.0, "acceleration_z": -44.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:46.003000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 5.0, "acceleration_z": -39.0, "gyroscope_x": -12.0, "gyroscope_y": 60.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:46.055000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 6.0, "acceleration_z": -41.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:46.107000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 5.0, "acceleration_z": -41.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:46.159000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 5.0, "acceleration_z": -39.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:52.042000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 5.0, "acceleration_z": -40.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:52.094000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 5.0, "acceleration_z": -39.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:52.146000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 5.0, "acceleration_z": -42.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:52.198000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 4.0, "acceleration_z": -41.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:57.948000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 4.0, "acceleration_z": -40.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:58", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 2.0, "acceleration_z": -43.0, "gyroscope_x": -11.0, "gyroscope_y": 61.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:58.052000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 3.0, "acceleration_z": -39.0, "gyroscope_x": -14.0, "gyroscope_y": 61.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:44:58.104000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 4.0, "acceleration_z": -45.0, "gyroscope_x": -15.0, "gyroscope_y": 62.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:04.043000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": -1.0, "acceleration_z": -29.0, "gyroscope_x": -13.0, "gyroscope_y": 71.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:04.095000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": -1.0, "acceleration_z": -31.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:04.147000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 2.0, "acceleration_z": -35.0, "gyroscope_x": -13.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:04.199000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 2.0, "acceleration_z": -38.0, "gyroscope_x": -9.0, "gyroscope_y": 63.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:09.764000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": -4.0, "acceleration_z": -92.0, "gyroscope_x": -7.0, "gyroscope_y": 29.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:09.816000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 6.0, "acceleration_z": -93.0, "gyroscope_x": 18.0, "gyroscope_y": 48.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:09.868000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 8.0, "acceleration_z": -80.0, "gyroscope_x": -23.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:09.920000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 12.0, "acceleration_z": -84.0, "gyroscope_x": 5.0, "gyroscope_y": 76.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:15.893000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 4.0, "acceleration_z": -62.0, "gyroscope_x": 4.0, "gyroscope_y": 59.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:15.945000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 0.0, "acceleration_z": -85.0, "gyroscope_x": -2.0, "gyroscope_y": 44.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:15.997000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 4.0, "acceleration_z": -92.0, "gyroscope_x": -8.0, "gyroscope_y": 54.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:16.049000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": -6.0, "acceleration_z": -93.0, "gyroscope_x": 2.0, "gyroscope_y": 72.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:21.750000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 3.0, "acceleration_z": -74.0, "gyroscope_x": -16.0, "gyroscope_y": 55.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:21.802000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 5.0, "acceleration_z": -78.0, "gyroscope_x": -14.0, "gyroscope_y": 50.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:21.854000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 4.0, "acceleration_z": -75.0, "gyroscope_x": -15.0, "gyroscope_y": 54.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:21.906000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 5.0, "acceleration_z": -75.0, "gyroscope_x": -17.0, "gyroscope_y": 53.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:27.606000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 67.0, "acceleration_z": -60.0, "gyroscope_x": -354.0, "gyroscope_y": 175.0, "gyroscope_z": 15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:27.658000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 295.0, "acceleration_y": 71.0, "acceleration_z": -37.0, "gyroscope_x": -261.0, "gyroscope_y": 172.0, "gyroscope_z": 173.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:27.710000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 49.0, "acceleration_z": -67.0, "gyroscope_x": -211.0, "gyroscope_y": -10.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:27.762000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 83.0, "acceleration_z": -74.0, "gyroscope_x": -271.0, "gyroscope_y": 85.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:33.422000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 14.0, "acceleration_z": -46.0, "gyroscope_x": -42.0, "gyroscope_y": -178.0, "gyroscope_z": -27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:33.474000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": 50.0, "acceleration_z": -74.0, "gyroscope_x": -33.0, "gyroscope_y": -84.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:33.526000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 306.0, "acceleration_y": 48.0, "acceleration_z": 66.0, "gyroscope_x": -159.0, "gyroscope_y": 20.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:33.578000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 38.0, "acceleration_z": -66.0, "gyroscope_x": 42.0, "gyroscope_y": -317.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:39.252000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 204.0, "acceleration_y": -1.0, "acceleration_z": -20.0, "gyroscope_x": -5.0, "gyroscope_y": 44.0, "gyroscope_z": -68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:39.304000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 333.0, "acceleration_y": 20.0, "acceleration_z": 9.0, "gyroscope_x": -41.0, "gyroscope_y": -65.0, "gyroscope_z": 94.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:39.356000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 28.0, "acceleration_z": -1.0, "gyroscope_x": -60.0, "gyroscope_y": -67.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:39.408000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 52.0, "acceleration_z": -21.0, "gyroscope_x": 4.0, "gyroscope_y": 57.0, "gyroscope_z": 59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:45.366000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 26.0, "acceleration_z": -69.0, "gyroscope_x": -38.0, "gyroscope_y": 197.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:45.418000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 150.0, "acceleration_y": -12.0, "acceleration_z": -79.0, "gyroscope_x": 197.0, "gyroscope_y": 162.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:45.470000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 183.0, "acceleration_y": 30.0, "acceleration_z": 67.0, "gyroscope_x": 3.0, "gyroscope_y": 369.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:45.522000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 51.0, "acceleration_z": -127.0, "gyroscope_x": 193.0, "gyroscope_y": 279.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:51.853000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 167.0, "acceleration_y": -30.0, "acceleration_z": -28.0, "gyroscope_x": -270.0, "gyroscope_y": -32.0, "gyroscope_z": 68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:51.905000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 68.0, "acceleration_z": 8.0, "gyroscope_x": -226.0, "gyroscope_y": -2.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:51.957000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 125.0, "acceleration_y": 17.0, "acceleration_z": -58.0, "gyroscope_x": -329.0, "gyroscope_y": -102.0, "gyroscope_z": -141.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:52.009000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 369.0, "acceleration_y": 116.0, "acceleration_z": -55.0, "gyroscope_x": -289.0, "gyroscope_y": -93.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:57.790000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 48.0, "acceleration_z": -58.0, "gyroscope_x": 517.0, "gyroscope_y": 241.0, "gyroscope_z": -105.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:57.842000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 31.0, "acceleration_z": 28.0, "gyroscope_x": 525.0, "gyroscope_y": 195.0, "gyroscope_z": -118.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:57.894000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 49.0, "acceleration_z": 35.0, "gyroscope_x": 510.0, "gyroscope_y": 328.0, "gyroscope_z": -123.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:45:57.946000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 85.0, "acceleration_z": 38.0, "gyroscope_x": 505.0, "gyroscope_y": 204.0, "gyroscope_z": -146.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:03.502000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 53.0, "acceleration_z": 19.0, "gyroscope_x": 214.0, "gyroscope_y": 77.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:03.554000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 50.0, "acceleration_z": -122.0, "gyroscope_x": 193.0, "gyroscope_y": 159.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:03.606000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 98.0, "acceleration_z": 5.0, "gyroscope_x": 145.0, "gyroscope_y": 242.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:03.658000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 336.0, "acceleration_y": 25.0, "acceleration_z": -67.0, "gyroscope_x": 32.0, "gyroscope_y": 340.0, "gyroscope_z": 155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:09.240000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 191.0, "acceleration_y": 25.0, "acceleration_z": -18.0, "gyroscope_x": -62.0, "gyroscope_y": -30.0, "gyroscope_z": -77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:09.292000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 52.0, "acceleration_z": -52.0, "gyroscope_x": 3.0, "gyroscope_y": 20.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:09.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 75.0, "acceleration_z": 24.0, "gyroscope_x": 25.0, "gyroscope_y": 88.0, "gyroscope_z": 36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:09.396000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": -45.0, "acceleration_z": -95.0, "gyroscope_x": 142.0, "gyroscope_y": 43.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:14.972000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 349.0, "acceleration_y": 9.0, "acceleration_z": 4.0, "gyroscope_x": 94.0, "gyroscope_y": 451.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:15.024000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 234.0, "acceleration_y": -29.0, "acceleration_z": -71.0, "gyroscope_x": 208.0, "gyroscope_y": 232.0, "gyroscope_z": 164.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:15.076000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 52.0, "acceleration_z": 96.0, "gyroscope_x": 40.0, "gyroscope_y": 240.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:15.128000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 53.0, "acceleration_z": 8.0, "gyroscope_x": 145.0, "gyroscope_y": 138.0, "gyroscope_z": 92.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:20.732000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 48.0, "acceleration_z": -91.0, "gyroscope_x": -7.0, "gyroscope_y": 86.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:20.784000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 223.0, "acceleration_y": 40.0, "acceleration_z": 93.0, "gyroscope_x": -55.0, "gyroscope_y": 257.0, "gyroscope_z": -161.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:20.836000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 366.0, "acceleration_y": 28.0, "acceleration_z": 77.0, "gyroscope_x": -168.0, "gyroscope_y": 258.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:20.888000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 41.0, "acceleration_z": 3.0, "gyroscope_x": -15.0, "gyroscope_y": 3.0, "gyroscope_z": -82.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:26.655000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 6.0, "acceleration_z": -6.0, "gyroscope_x": -18.0, "gyroscope_y": 105.0, "gyroscope_z": -64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:26.707000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 392.0, "acceleration_y": 31.0, "acceleration_z": -116.0, "gyroscope_x": 37.0, "gyroscope_y": 65.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:26.759000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 65.0, "acceleration_z": -31.0, "gyroscope_x": -4.0, "gyroscope_y": -16.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:26.811000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 152.0, "acceleration_y": 26.0, "acceleration_z": -44.0, "gyroscope_x": 111.0, "gyroscope_y": 105.0, "gyroscope_z": -90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:32.400000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 89.0, "acceleration_z": 36.0, "gyroscope_x": 174.0, "gyroscope_y": 113.0, "gyroscope_z": 113.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:32.452000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 315.0, "acceleration_y": 16.0, "acceleration_z": -45.0, "gyroscope_x": 70.0, "gyroscope_y": 215.0, "gyroscope_z": 362.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:32.504000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 364.0, "acceleration_y": 89.0, "acceleration_z": -7.0, "gyroscope_x": 89.0, "gyroscope_y": 227.0, "gyroscope_z": 291.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:32.556000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": -82.0, "acceleration_z": -110.0, "gyroscope_x": 112.0, "gyroscope_y": 70.0, "gyroscope_z": 439.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:38.376000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 363.0, "acceleration_y": 10.0, "acceleration_z": 18.0, "gyroscope_x": 81.0, "gyroscope_y": -141.0, "gyroscope_z": -84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:38.428000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 298.0, "acceleration_y": 62.0, "acceleration_z": -49.0, "gyroscope_x": 86.0, "gyroscope_y": -78.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:38.480000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": 45.0, "acceleration_z": 18.0, "gyroscope_x": -3.0, "gyroscope_y": 68.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:38.532000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 68.0, "acceleration_z": -161.0, "gyroscope_x": 165.0, "gyroscope_y": -10.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:47.252000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 51.0, "acceleration_z": 18.0, "gyroscope_x": 47.0, "gyroscope_y": 227.0, "gyroscope_z": -61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:47.304000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 36.0, "acceleration_z": -66.0, "gyroscope_x": 15.0, "gyroscope_y": 136.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:47.356000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 343.0, "acceleration_y": 56.0, "acceleration_z": 62.0, "gyroscope_x": -136.0, "gyroscope_y": 328.0, "gyroscope_z": 8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:47.408000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 204.0, "acceleration_y": -16.0, "acceleration_z": -100.0, "gyroscope_x": -102.0, "gyroscope_y": 187.0, "gyroscope_z": 44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:53.311000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 200.0, "acceleration_y": 21.0, "acceleration_z": 107.0, "gyroscope_x": -201.0, "gyroscope_y": -74.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:53.363000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 37.0, "acceleration_z": 110.0, "gyroscope_x": -245.0, "gyroscope_y": -13.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:53.415000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 32.0, "acceleration_z": -16.0, "gyroscope_x": -85.0, "gyroscope_y": -196.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:53.467000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 2.0, "acceleration_z": -65.0, "gyroscope_x": -100.0, "gyroscope_y": -180.0, "gyroscope_z": -140.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:58.977000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 39.0, "acceleration_z": -47.0, "gyroscope_x": 61.0, "gyroscope_y": 10.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:59.029000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 25.0, "acceleration_z": -35.0, "gyroscope_x": 3.0, "gyroscope_y": 51.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:59.081000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": 57.0, "acceleration_z": 133.0, "gyroscope_x": -127.0, "gyroscope_y": 233.0, "gyroscope_z": -146.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:46:59.133000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 309.0, "acceleration_y": 71.0, "acceleration_z": -61.0, "gyroscope_x": 103.0, "gyroscope_y": 29.0, "gyroscope_z": 84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:04.666000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 51.0, "acceleration_z": 26.0, "gyroscope_x": 32.0, "gyroscope_y": 244.0, "gyroscope_z": -93.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:04.718000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": -1.0, "acceleration_z": 36.0, "gyroscope_x": -61.0, "gyroscope_y": 1.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:04.770000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": -25.0, "acceleration_z": 39.0, "gyroscope_x": -8.0, "gyroscope_y": 298.0, "gyroscope_z": 91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:04.822000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 14.0, "acceleration_z": -69.0, "gyroscope_x": 48.0, "gyroscope_y": 3.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:10.380000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 188.0, "acceleration_y": 42.0, "acceleration_z": -46.0, "gyroscope_x": 79.0, "gyroscope_y": 111.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:10.432000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": -17.0, "acceleration_z": -52.0, "gyroscope_x": 47.0, "gyroscope_y": -67.0, "gyroscope_z": -126.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:10.484000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 163.0, "acceleration_y": 45.0, "acceleration_z": 103.0, "gyroscope_x": -36.0, "gyroscope_y": -233.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:10.536000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 58.0, "acceleration_z": -12.0, "gyroscope_x": 225.0, "gyroscope_y": 15.0, "gyroscope_z": 23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:16.064000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 3.0, "acceleration_z": -59.0, "gyroscope_x": 66.0, "gyroscope_y": 277.0, "gyroscope_z": 115.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:16.116000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 161.0, "acceleration_y": 20.0, "acceleration_z": -3.0, "gyroscope_x": -10.0, "gyroscope_y": 231.0, "gyroscope_z": 120.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:16.168000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 2.0, "acceleration_z": 76.0, "gyroscope_x": -126.0, "gyroscope_y": -109.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:16.220000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 15.0, "acceleration_z": -34.0, "gyroscope_x": 59.0, "gyroscope_y": -17.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:21.904000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 378.0, "acceleration_y": 13.0, "acceleration_z": -127.0, "gyroscope_x": 6.0, "gyroscope_y": 127.0, "gyroscope_z": 158.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:21.956000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 343.0, "acceleration_y": -66.0, "acceleration_z": 41.0, "gyroscope_x": -93.0, "gyroscope_y": 105.0, "gyroscope_z": 140.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:22.008000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 83.0, "acceleration_z": -100.0, "gyroscope_x": -24.0, "gyroscope_y": 120.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:22.060000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 202.0, "acceleration_y": 6.0, "acceleration_z": -110.0, "gyroscope_x": -18.0, "gyroscope_y": -76.0, "gyroscope_z": 159.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:27.814000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": -12.0, "acceleration_z": -41.0, "gyroscope_x": 38.0, "gyroscope_y": 227.0, "gyroscope_z": 175.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:27.866000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": -21.0, "acceleration_z": -48.0, "gyroscope_x": 4.0, "gyroscope_y": 4.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:27.918000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": -5.0, "acceleration_z": 33.0, "gyroscope_x": -42.0, "gyroscope_y": 27.0, "gyroscope_z": 64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:27.970000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 18.0, "acceleration_z": -86.0, "gyroscope_x": 96.0, "gyroscope_y": 112.0, "gyroscope_z": -120.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:33.726000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 56.0, "acceleration_z": 91.0, "gyroscope_x": 21.0, "gyroscope_y": 182.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:33.778000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 331.0, "acceleration_y": 78.0, "acceleration_z": -32.0, "gyroscope_x": 13.0, "gyroscope_y": 10.0, "gyroscope_z": 125.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:33.830000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 325.0, "acceleration_y": -21.0, "acceleration_z": -121.0, "gyroscope_x": 29.0, "gyroscope_y": -10.0, "gyroscope_z": 160.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:33.882000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 176.0, "acceleration_y": -2.0, "acceleration_z": -97.0, "gyroscope_x": 77.0, "gyroscope_y": 38.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:40.102000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 315.0, "acceleration_y": 8.0, "acceleration_z": 64.0, "gyroscope_x": 140.0, "gyroscope_y": -109.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:40.154000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 61.0, "acceleration_z": -1.0, "gyroscope_x": -68.0, "gyroscope_y": -69.0, "gyroscope_z": 30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:40.206000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 207.0, "acceleration_y": 48.0, "acceleration_z": 75.0, "gyroscope_x": -2.0, "gyroscope_y": 130.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:40.258000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 69.0, "acceleration_z": -77.0, "gyroscope_x": 123.0, "gyroscope_y": 103.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:45.963000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 27.0, "acceleration_z": -87.0, "gyroscope_x": -188.0, "gyroscope_y": 36.0, "gyroscope_z": 13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:46.015000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 282.0, "acceleration_y": 65.0, "acceleration_z": 13.0, "gyroscope_x": -185.0, "gyroscope_y": -6.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:46.067000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 43.0, "acceleration_z": 28.0, "gyroscope_x": -256.0, "gyroscope_y": -201.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:46.119000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 18.0, "acceleration_z": -15.0, "gyroscope_x": -148.0, "gyroscope_y": -4.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:52.077000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 22.0, "acceleration_z": -59.0, "gyroscope_x": -11.0, "gyroscope_y": 72.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:52.129000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 26.0, "acceleration_z": -42.0, "gyroscope_x": -9.0, "gyroscope_y": 47.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:52.181000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 21.0, "acceleration_z": -67.0, "gyroscope_x": -12.0, "gyroscope_y": 34.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:52.233000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 30.0, "acceleration_z": -58.0, "gyroscope_x": -20.0, "gyroscope_y": 29.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:57.804000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 80.0, "acceleration_z": -15.0, "gyroscope_x": -547.0, "gyroscope_y": -66.0, "gyroscope_z": -80.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:57.856000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 207.0, "acceleration_y": 18.0, "acceleration_z": 13.0, "gyroscope_x": -557.0, "gyroscope_y": 106.0, "gyroscope_z": -129.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:57.908000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 64.0, "acceleration_z": -38.0, "gyroscope_x": -590.0, "gyroscope_y": 109.0, "gyroscope_z": -156.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:47:57.960000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 71.0, "acceleration_z": -39.0, "gyroscope_x": -568.0, "gyroscope_y": -26.0, "gyroscope_z": -203.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:03.553000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 69.0, "acceleration_z": -13.0, "gyroscope_x": -149.0, "gyroscope_y": 130.0, "gyroscope_z": 29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:03.605000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 202.0, "acceleration_y": 61.0, "acceleration_z": -85.0, "gyroscope_x": 22.0, "gyroscope_y": 26.0, "gyroscope_z": -90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:03.657000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": -17.0, "acceleration_z": -96.0, "gyroscope_x": -25.0, "gyroscope_y": 6.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:03.709000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 221.0, "acceleration_y": 8.0, "acceleration_z": -129.0, "gyroscope_x": 7.0, "gyroscope_y": 119.0, "gyroscope_z": -75.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:09.229000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 332.0, "acceleration_y": -48.0, "acceleration_z": -47.0, "gyroscope_x": 62.0, "gyroscope_y": 204.0, "gyroscope_z": 27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:09.281000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 301.0, "acceleration_y": 17.0, "acceleration_z": -93.0, "gyroscope_x": -32.0, "gyroscope_y": 270.0, "gyroscope_z": -118.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:09.333000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 150.0, "acceleration_y": 49.0, "acceleration_z": 77.0, "gyroscope_x": -141.0, "gyroscope_y": -172.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:09.385000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 26.0, "acceleration_z": -64.0, "gyroscope_x": 2.0, "gyroscope_y": -92.0, "gyroscope_z": -134.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:14.972000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 79.0, "acceleration_z": -9.0, "gyroscope_x": -32.0, "gyroscope_y": -174.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:15.024000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 35.0, "acceleration_z": -17.0, "gyroscope_x": -37.0, "gyroscope_y": 42.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:15.076000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 404.0, "acceleration_y": 62.0, "acceleration_z": 20.0, "gyroscope_x": -94.0, "gyroscope_y": 147.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:15.128000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 351.0, "acceleration_y": 36.0, "acceleration_z": -57.0, "gyroscope_x": 19.0, "gyroscope_y": 98.0, "gyroscope_z": 134.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:20.675000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 25.0, "acceleration_z": -105.0, "gyroscope_x": -391.0, "gyroscope_y": -165.0, "gyroscope_z": -39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:20.727000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 78.0, "acceleration_z": -94.0, "gyroscope_x": -146.0, "gyroscope_y": -117.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:20.779000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 33.0, "acceleration_z": -72.0, "gyroscope_x": 51.0, "gyroscope_y": -85.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:20.831000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 17.0, "acceleration_z": 11.0, "gyroscope_x": -49.0, "gyroscope_y": -110.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:29.658000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 339.0, "acceleration_y": 84.0, "acceleration_z": -88.0, "gyroscope_x": 294.0, "gyroscope_y": -20.0, "gyroscope_z": -277.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:29.710000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": -88.0, "acceleration_z": 34.0, "gyroscope_x": 253.0, "gyroscope_y": -245.0, "gyroscope_z": -263.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:29.762000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 468.0, "acceleration_y": 25.0, "acceleration_z": -12.0, "gyroscope_x": 280.0, "gyroscope_y": 30.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:29.814000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": -16.0, "acceleration_z": -25.0, "gyroscope_x": 541.0, "gyroscope_y": 21.0, "gyroscope_z": -151.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:35.491000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 33.0, "acceleration_z": -57.0, "gyroscope_x": 125.0, "gyroscope_y": -60.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:35.543000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": -4.0, "acceleration_z": -7.0, "gyroscope_x": -139.0, "gyroscope_y": 20.0, "gyroscope_z": -88.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:35.595000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": -11.0, "acceleration_z": -58.0, "gyroscope_x": 68.0, "gyroscope_y": -13.0, "gyroscope_z": 64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:35.647000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 110.0, "acceleration_z": -31.0, "gyroscope_x": -91.0, "gyroscope_y": 1.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:41.409000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 4.0, "acceleration_z": -102.0, "gyroscope_x": -34.0, "gyroscope_y": 101.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:41.461000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 175.0, "acceleration_y": 47.0, "acceleration_z": -81.0, "gyroscope_x": -9.0, "gyroscope_y": -183.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:41.513000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 282.0, "acceleration_y": 72.0, "acceleration_z": -30.0, "gyroscope_x": 71.0, "gyroscope_y": -5.0, "gyroscope_z": -64.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:41.565000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 307.0, "acceleration_y": 17.0, "acceleration_z": 7.0, "gyroscope_x": -101.0, "gyroscope_y": 188.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:47.070000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 42.0, "acceleration_z": 59.0, "gyroscope_x": -386.0, "gyroscope_y": -143.0, "gyroscope_z": 98.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:47.122000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 39.0, "acceleration_z": -11.0, "gyroscope_x": -385.0, "gyroscope_y": -229.0, "gyroscope_z": -56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:47.174000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 29.0, "acceleration_z": -5.0, "gyroscope_x": -362.0, "gyroscope_y": -112.0, "gyroscope_z": -59.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:47.226000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 314.0, "acceleration_y": 26.0, "acceleration_z": -113.0, "gyroscope_x": -467.0, "gyroscope_y": -274.0, "gyroscope_z": -155.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:52.742000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 80.0, "acceleration_z": -8.0, "gyroscope_x": 206.0, "gyroscope_y": 137.0, "gyroscope_z": 5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:52.794000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": -20.0, "acceleration_z": 32.0, "gyroscope_x": 293.0, "gyroscope_y": 182.0, "gyroscope_z": -88.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:52.846000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 25.0, "acceleration_z": 51.0, "gyroscope_x": 29.0, "gyroscope_y": 67.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:52.898000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 59.0, "acceleration_z": -64.0, "gyroscope_x": 197.0, "gyroscope_y": 150.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:58.791000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 31.0, "acceleration_z": -8.0, "gyroscope_x": -280.0, "gyroscope_y": 66.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:58.843000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 32.0, "acceleration_z": -82.0, "gyroscope_x": -216.0, "gyroscope_y": 24.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:58.895000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 26.0, "acceleration_z": -81.0, "gyroscope_x": -92.0, "gyroscope_y": 40.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:48:58.947000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 50.0, "acceleration_z": -30.0, "gyroscope_x": -145.0, "gyroscope_y": 74.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:04.556000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 49.0, "acceleration_z": -121.0, "gyroscope_x": 73.0, "gyroscope_y": 16.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:04.608000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": 42.0, "acceleration_z": -69.0, "gyroscope_x": 154.0, "gyroscope_y": 58.0, "gyroscope_z": 42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:04.660000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 26.0, "acceleration_z": 34.0, "gyroscope_x": 63.0, "gyroscope_y": 148.0, "gyroscope_z": 53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:04.712000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": -4.0, "acceleration_z": -30.0, "gyroscope_x": 101.0, "gyroscope_y": 203.0, "gyroscope_z": 120.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:10.313000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 33.0, "acceleration_z": -32.0, "gyroscope_x": -5.0, "gyroscope_y": 107.0, "gyroscope_z": 99.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:10.365000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -3.0, "acceleration_z": -44.0, "gyroscope_x": 4.0, "gyroscope_y": 76.0, "gyroscope_z": 79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:10.417000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 39.0, "acceleration_z": 4.0, "gyroscope_x": -59.0, "gyroscope_y": 40.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:10.469000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 36.0, "acceleration_z": 18.0, "gyroscope_x": -158.0, "gyroscope_y": 31.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:16.355000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 37.0, "acceleration_z": -114.0, "gyroscope_x": 14.0, "gyroscope_y": -111.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:16.407000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 240.0, "acceleration_y": 39.0, "acceleration_z": -64.0, "gyroscope_x": 53.0, "gyroscope_y": 4.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:16.459000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 47.0, "acceleration_z": -154.0, "gyroscope_x": 184.0, "gyroscope_y": 52.0, "gyroscope_z": -44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:16.511000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 16.0, "acceleration_z": -27.0, "gyroscope_x": 166.0, "gyroscope_y": 205.0, "gyroscope_z": -74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:22.360000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 56.0, "acceleration_z": 85.0, "gyroscope_x": 521.0, "gyroscope_y": 100.0, "gyroscope_z": -117.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:22.412000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 48.0, "acceleration_z": -73.0, "gyroscope_x": 414.0, "gyroscope_y": 85.0, "gyroscope_z": -160.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:22.464000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 44.0, "acceleration_z": -75.0, "gyroscope_x": 524.0, "gyroscope_y": 67.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:22.516000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -48.0, "gyroscope_x": 443.0, "gyroscope_y": 105.0, "gyroscope_z": -38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:28.316000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 1.0, "acceleration_z": -64.0, "gyroscope_x": 147.0, "gyroscope_y": 134.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:28.368000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 17.0, "acceleration_z": -158.0, "gyroscope_x": 91.0, "gyroscope_y": 102.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:28.420000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 79.0, "acceleration_z": 51.0, "gyroscope_x": 98.0, "gyroscope_y": 383.0, "gyroscope_z": 71.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:28.472000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 32.0, "acceleration_z": -35.0, "gyroscope_x": 208.0, "gyroscope_y": 165.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:34.309000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 15.0, "acceleration_z": -53.0, "gyroscope_x": -245.0, "gyroscope_y": 76.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:34.361000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 29.0, "acceleration_z": -214.0, "gyroscope_x": -99.0, "gyroscope_y": 4.0, "gyroscope_z": 47.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:34.413000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 28.0, "acceleration_z": -34.0, "gyroscope_x": -30.0, "gyroscope_y": -22.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:34.465000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 9.0, "acceleration_z": -107.0, "gyroscope_x": -268.0, "gyroscope_y": 88.0, "gyroscope_z": -50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:40.420000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 46.0, "acceleration_z": -70.0, "gyroscope_x": 4.0, "gyroscope_y": -9.0, "gyroscope_z": 57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:40.472000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 240.0, "acceleration_y": 42.0, "acceleration_z": -84.0, "gyroscope_x": -6.0, "gyroscope_y": 132.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:40.524000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": -1.0, "acceleration_z": -187.0, "gyroscope_x": 207.0, "gyroscope_y": 86.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:40.576000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 62.0, "acceleration_z": -106.0, "gyroscope_x": 260.0, "gyroscope_y": 238.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:46.285000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 33.0, "acceleration_z": -117.0, "gyroscope_x": -151.0, "gyroscope_y": 17.0, "gyroscope_z": 39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:46.337000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 38.0, "acceleration_z": 9.0, "gyroscope_x": -208.0, "gyroscope_y": 9.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:46.389000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 47.0, "acceleration_z": 111.0, "gyroscope_x": -413.0, "gyroscope_y": -4.0, "gyroscope_z": 77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:46.441000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 54.0, "acceleration_z": -18.0, "gyroscope_x": -196.0, "gyroscope_y": 21.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:51.964000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 32.0, "acceleration_z": -31.0, "gyroscope_x": 53.0, "gyroscope_y": 3.0, "gyroscope_z": 5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:52.016000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 28.0, "acceleration_z": -64.0, "gyroscope_x": 145.0, "gyroscope_y": 51.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:52.068000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 30.0, "acceleration_z": -58.0, "gyroscope_x": 268.0, "gyroscope_y": 74.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:52.120000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 39.0, "acceleration_z": -21.0, "gyroscope_x": 246.0, "gyroscope_y": 126.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:58.006000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 296.0, "acceleration_y": 74.0, "acceleration_z": 34.0, "gyroscope_x": -529.0, "gyroscope_y": -134.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:58.058000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 46.0, "acceleration_z": -97.0, "gyroscope_x": -549.0, "gyroscope_y": -141.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:58.110000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 312.0, "acceleration_y": 54.0, "acceleration_z": -160.0, "gyroscope_x": -392.0, "gyroscope_y": -319.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:49:58.162000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 19.0, "acceleration_z": -81.0, "gyroscope_x": -185.0, "gyroscope_y": -183.0, "gyroscope_z": 40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:03.803000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 55.0, "acceleration_z": 85.0, "gyroscope_x": -269.0, "gyroscope_y": -101.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:03.855000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 13.0, "acceleration_z": -26.0, "gyroscope_x": -237.0, "gyroscope_y": -124.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:03.907000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 6.0, "acceleration_z": -61.0, "gyroscope_x": -73.0, "gyroscope_y": -83.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:03.959000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 27.0, "acceleration_z": -72.0, "gyroscope_x": 23.0, "gyroscope_y": -1.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:13.210000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 18.0, "acceleration_z": -61.0, "gyroscope_x": -3.0, "gyroscope_y": 154.0, "gyroscope_z": -138.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:13.262000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 30.0, "acceleration_z": -56.0, "gyroscope_x": -12.0, "gyroscope_y": 72.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:13.314000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 28.0, "acceleration_z": -55.0, "gyroscope_x": -14.0, "gyroscope_y": 32.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 18:50:13.366000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 22.0, "acceleration_z": -54.0, "gyroscope_x": -13.0, "gyroscope_y": 20.0, "gyroscope_z": -27.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:05:07.953000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 18.0, "acceleration_z": -73.0, "gyroscope_x": -13.0, "gyroscope_y": 100.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:40.312000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 28.0, "acceleration_z": -69.0, "gyroscope_x": -17.0, "gyroscope_y": 88.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:40.364000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 29.0, "acceleration_z": -75.0, "gyroscope_x": -16.0, "gyroscope_y": 81.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:40.416000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 27.0, "acceleration_z": -79.0, "gyroscope_x": -14.0, "gyroscope_y": 79.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:45.911000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 26.0, "acceleration_z": -71.0, "gyroscope_x": -14.0, "gyroscope_y": 81.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:45.963000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 26.0, "acceleration_z": -68.0, "gyroscope_x": -15.0, "gyroscope_y": 79.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:46.015000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 25.0, "acceleration_z": -69.0, "gyroscope_x": -14.0, "gyroscope_y": 73.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:46.067000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 26.0, "acceleration_z": -67.0, "gyroscope_x": -17.0, "gyroscope_y": 76.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:51.713000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 26.0, "acceleration_z": -65.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:51.765000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 27.0, "acceleration_z": -66.0, "gyroscope_x": -15.0, "gyroscope_y": 62.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:51.817000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 25.0, "acceleration_z": -67.0, "gyroscope_x": -15.0, "gyroscope_y": 58.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:51.869000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 26.0, "acceleration_z": -68.0, "gyroscope_x": -12.0, "gyroscope_y": 57.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:57.436000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 24.0, "acceleration_z": -67.0, "gyroscope_x": -11.0, "gyroscope_y": 73.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:57.488000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 25.0, "acceleration_z": -65.0, "gyroscope_x": -15.0, "gyroscope_y": 74.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:57.540000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -66.0, "gyroscope_x": -17.0, "gyroscope_y": 69.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:06:57.592000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 26.0, "acceleration_z": -66.0, "gyroscope_x": -15.0, "gyroscope_y": 71.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:03.167000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 109.0, "acceleration_z": -102.0, "gyroscope_x": -64.0, "gyroscope_y": 33.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:03.219000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": 91.0, "acceleration_z": -57.0, "gyroscope_x": -84.0, "gyroscope_y": 79.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:03.271000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": 117.0, "acceleration_z": -155.0, "gyroscope_x": -76.0, "gyroscope_y": 247.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:03.323000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 158.0, "acceleration_y": 38.0, "acceleration_z": -114.0, "gyroscope_x": -32.0, "gyroscope_y": 534.0, "gyroscope_z": 60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:08.835000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 64.0, "acceleration_z": -35.0, "gyroscope_x": -84.0, "gyroscope_y": 34.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:08.887000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 294.0, "acceleration_y": 26.0, "acceleration_z": 24.0, "gyroscope_x": -152.0, "gyroscope_y": -1.0, "gyroscope_z": -117.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:08.939000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 11.0, "acceleration_z": -45.0, "gyroscope_x": -182.0, "gyroscope_y": -183.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:08.991000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 324.0, "acceleration_y": 12.0, "acceleration_z": 104.0, "gyroscope_x": -256.0, "gyroscope_y": -42.0, "gyroscope_z": -73.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:14.710000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 197.0, "acceleration_y": 19.0, "acceleration_z": 37.0, "gyroscope_x": 330.0, "gyroscope_y": -25.0, "gyroscope_z": -84.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:14.762000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 38.0, "acceleration_z": -114.0, "gyroscope_x": 360.0, "gyroscope_y": 216.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:14.814000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 204.0, "acceleration_y": 29.0, "acceleration_z": 106.0, "gyroscope_x": 235.0, "gyroscope_y": 256.0, "gyroscope_z": -124.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:14.866000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 219.0, "acceleration_y": 61.0, "acceleration_z": 150.0, "gyroscope_x": 253.0, "gyroscope_y": 202.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:20.903000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 301.0, "acceleration_y": 43.0, "acceleration_z": -26.0, "gyroscope_x": 40.0, "gyroscope_y": -52.0, "gyroscope_z": 66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:20.955000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 25.0, "acceleration_z": 77.0, "gyroscope_x": 54.0, "gyroscope_y": 73.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:21.007000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 295.0, "acceleration_y": 17.0, "acceleration_z": 30.0, "gyroscope_x": 279.0, "gyroscope_y": 82.0, "gyroscope_z": 36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:21.059000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": -78.0, "acceleration_z": 43.0, "gyroscope_x": 198.0, "gyroscope_y": 44.0, "gyroscope_z": -187.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:27.056000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 39.0, "acceleration_z": 107.0, "gyroscope_x": -138.0, "gyroscope_y": 39.0, "gyroscope_z": -52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:27.108000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 325.0, "acceleration_y": 55.0, "acceleration_z": 55.0, "gyroscope_x": -21.0, "gyroscope_y": 101.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:27.160000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 6.0, "acceleration_z": 17.0, "gyroscope_x": 25.0, "gyroscope_y": -31.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:27.212000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 294.0, "acceleration_y": -15.0, "acceleration_z": -61.0, "gyroscope_x": 23.0, "gyroscope_y": 54.0, "gyroscope_z": 53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:32.902000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 190.0, "acceleration_y": -52.0, "acceleration_z": -1.0, "gyroscope_x": 82.0, "gyroscope_y": 15.0, "gyroscope_z": 203.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:32.954000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 83.0, "acceleration_z": -1.0, "gyroscope_x": -16.0, "gyroscope_y": -5.0, "gyroscope_z": 264.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:33.006000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 163.0, "acceleration_y": 62.0, "acceleration_z": -56.0, "gyroscope_x": -25.0, "gyroscope_y": -45.0, "gyroscope_z": 97.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:33.058000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 24.0, "acceleration_z": 184.0, "gyroscope_x": -139.0, "gyroscope_y": -224.0, "gyroscope_z": 236.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:38.594000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 281.0, "acceleration_y": 37.0, "acceleration_z": -37.0, "gyroscope_x": 79.0, "gyroscope_y": 231.0, "gyroscope_z": 69.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:38.646000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 168.0, "acceleration_y": 23.0, "acceleration_z": 39.0, "gyroscope_x": 126.0, "gyroscope_y": 171.0, "gyroscope_z": -43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:38.698000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 282.0, "acceleration_y": 63.0, "acceleration_z": -17.0, "gyroscope_x": 39.0, "gyroscope_y": 193.0, "gyroscope_z": -116.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:38.750000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 51.0, "acceleration_z": -42.0, "gyroscope_x": -113.0, "gyroscope_y": 27.0, "gyroscope_z": -110.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:44.368000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 3.0, "acceleration_z": 75.0, "gyroscope_x": 33.0, "gyroscope_y": -5.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:44.420000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": -33.0, "acceleration_z": -103.0, "gyroscope_x": 50.0, "gyroscope_y": 357.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:44.472000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 48.0, "acceleration_z": -16.0, "gyroscope_x": 169.0, "gyroscope_y": 127.0, "gyroscope_z": -160.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:44.524000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 303.0, "acceleration_y": -21.0, "acceleration_z": -207.0, "gyroscope_x": 276.0, "gyroscope_y": 102.0, "gyroscope_z": 122.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:50.022000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 16.0, "acceleration_z": -232.0, "gyroscope_x": 120.0, "gyroscope_y": -212.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:50.074000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 305.0, "acceleration_y": 42.0, "acceleration_z": -120.0, "gyroscope_x": 124.0, "gyroscope_y": -178.0, "gyroscope_z": 56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:50.126000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 54.0, "acceleration_z": -51.0, "gyroscope_x": 70.0, "gyroscope_y": -142.0, "gyroscope_z": 56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:50.178000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 155.0, "acceleration_y": 75.0, "acceleration_z": -62.0, "gyroscope_x": 118.0, "gyroscope_y": 302.0, "gyroscope_z": -119.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:55.696000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 3.0, "acceleration_z": -49.0, "gyroscope_x": 25.0, "gyroscope_y": 205.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:55.748000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 13.0, "acceleration_z": -26.0, "gyroscope_x": 36.0, "gyroscope_y": -94.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:55.800000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 27.0, "acceleration_z": 34.0, "gyroscope_x": -8.0, "gyroscope_y": 223.0, "gyroscope_z": 53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:07:55.852000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 29.0, "acceleration_z": -32.0, "gyroscope_x": 77.0, "gyroscope_y": 189.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:01.362000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 289.0, "acceleration_y": 47.0, "acceleration_z": -116.0, "gyroscope_x": 204.0, "gyroscope_y": 169.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:01.414000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 194.0, "acceleration_y": 40.0, "acceleration_z": -27.0, "gyroscope_x": 189.0, "gyroscope_y": 319.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:01.466000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 32.0, "acceleration_z": -75.0, "gyroscope_x": 167.0, "gyroscope_y": 136.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:01.518000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 8.0, "acceleration_z": -39.0, "gyroscope_x": 149.0, "gyroscope_y": 242.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:07.045000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 21.0, "acceleration_z": 16.0, "gyroscope_x": 71.0, "gyroscope_y": 179.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:07.097000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": -64.0, "acceleration_z": -35.0, "gyroscope_x": 100.0, "gyroscope_y": 167.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:07.149000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 42.0, "acceleration_z": -102.0, "gyroscope_x": -13.0, "gyroscope_y": 58.0, "gyroscope_z": -97.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:07.201000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 31.0, "acceleration_z": -106.0, "gyroscope_x": 88.0, "gyroscope_y": 126.0, "gyroscope_z": 58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:12.753000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 218.0, "acceleration_y": 77.0, "acceleration_z": 83.0, "gyroscope_x": -123.0, "gyroscope_y": 25.0, "gyroscope_z": 118.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:12.805000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 32.0, "acceleration_z": -56.0, "gyroscope_x": 33.0, "gyroscope_y": -42.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:12.857000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 24.0, "acceleration_z": -60.0, "gyroscope_x": 12.0, "gyroscope_y": -77.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:12.909000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 40.0, "acceleration_z": -6.0, "gyroscope_x": -91.0, "gyroscope_y": 139.0, "gyroscope_z": 63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:18.557000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 203.0, "acceleration_y": -7.0, "acceleration_z": -98.0, "gyroscope_x": 52.0, "gyroscope_y": 165.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:18.609000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 183.0, "acceleration_y": 54.0, "acceleration_z": 40.0, "gyroscope_x": -70.0, "gyroscope_y": 78.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:18.661000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 39.0, "acceleration_z": -58.0, "gyroscope_x": 43.0, "gyroscope_y": 195.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:18.713000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 32.0, "acceleration_z": 49.0, "gyroscope_x": -105.0, "gyroscope_y": 66.0, "gyroscope_z": 77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:24.342000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": -6.0, "acceleration_z": 28.0, "gyroscope_x": -20.0, "gyroscope_y": -2.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:24.394000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": -14.0, "acceleration_z": -11.0, "gyroscope_x": 35.0, "gyroscope_y": -36.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:24.446000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 223.0, "acceleration_y": 15.0, "acceleration_z": 3.0, "gyroscope_x": 72.0, "gyroscope_y": -41.0, "gyroscope_z": -92.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:24.498000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 186.0, "acceleration_y": -41.0, "acceleration_z": -46.0, "gyroscope_x": 24.0, "gyroscope_y": -89.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:31.976000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 205.0, "acceleration_y": 60.0, "acceleration_z": -32.0, "gyroscope_x": 503.0, "gyroscope_y": 287.0, "gyroscope_z": -102.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:32.028000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 50.0, "acceleration_z": -8.0, "gyroscope_x": 494.0, "gyroscope_y": 264.0, "gyroscope_z": -65.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:32.080000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 295.0, "acceleration_y": 82.0, "acceleration_z": 45.0, "gyroscope_x": 463.0, "gyroscope_y": 187.0, "gyroscope_z": -75.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:32.132000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": -4.0, "acceleration_z": 25.0, "gyroscope_x": 156.0, "gyroscope_y": 79.0, "gyroscope_z": -56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:38.019000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 12.0, "acceleration_z": -82.0, "gyroscope_x": 150.0, "gyroscope_y": 69.0, "gyroscope_z": -61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:38.071000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": 20.0, "acceleration_z": -23.0, "gyroscope_x": 196.0, "gyroscope_y": -103.0, "gyroscope_z": 17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:38.123000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 9.0, "acceleration_z": 64.0, "gyroscope_x": 77.0, "gyroscope_y": -16.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:38.175000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 48.0, "acceleration_z": -64.0, "gyroscope_x": 46.0, "gyroscope_y": -16.0, "gyroscope_z": -109.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:44.263000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 6.0, "acceleration_z": -14.0, "gyroscope_x": 289.0, "gyroscope_y": 27.0, "gyroscope_z": 47.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:44.315000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": -16.0, "acceleration_z": 9.0, "gyroscope_x": 241.0, "gyroscope_y": 14.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:44.367000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 21.0, "acceleration_z": 1.0, "gyroscope_x": 304.0, "gyroscope_y": 176.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:44.419000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 15.0, "acceleration_z": -31.0, "gyroscope_x": 301.0, "gyroscope_y": 176.0, "gyroscope_z": 53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:49.971000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 33.0, "acceleration_z": -50.0, "gyroscope_x": 45.0, "gyroscope_y": 53.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:50.023000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 314.0, "acceleration_y": 23.0, "acceleration_z": -110.0, "gyroscope_x": 34.0, "gyroscope_y": -60.0, "gyroscope_z": 41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:50.075000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 10.0, "acceleration_z": -5.0, "gyroscope_x": -11.0, "gyroscope_y": -52.0, "gyroscope_z": -81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:50.127000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 31.0, "acceleration_z": -29.0, "gyroscope_x": -34.0, "gyroscope_y": 76.0, "gyroscope_z": -40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:55.993000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 63.0, "acceleration_z": -141.0, "gyroscope_x": -178.0, "gyroscope_y": -80.0, "gyroscope_z": -43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:56.045000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 234.0, "acceleration_y": -15.0, "acceleration_z": -8.0, "gyroscope_x": -185.0, "gyroscope_y": -343.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:56.097000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 23.0, "acceleration_z": -43.0, "gyroscope_x": -216.0, "gyroscope_y": -169.0, "gyroscope_z": 7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:08:56.149000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 41.0, "acceleration_z": -96.0, "gyroscope_x": -183.0, "gyroscope_y": -80.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:01.833000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 54.0, "acceleration_z": 12.0, "gyroscope_x": -7.0, "gyroscope_y": 94.0, "gyroscope_z": 95.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:01.885000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 29.0, "acceleration_z": -64.0, "gyroscope_x": 3.0, "gyroscope_y": -57.0, "gyroscope_z": 79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:01.937000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 34.0, "acceleration_z": 12.0, "gyroscope_x": 58.0, "gyroscope_y": 154.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:01.989000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 30.0, "acceleration_z": -87.0, "gyroscope_x": 119.0, "gyroscope_y": 97.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:07.520000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 29.0, "acceleration_z": -28.0, "gyroscope_x": -125.0, "gyroscope_y": -54.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:07.572000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 37.0, "acceleration_z": -65.0, "gyroscope_x": -141.0, "gyroscope_y": -18.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:07.624000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 19.0, "acceleration_z": -57.0, "gyroscope_x": -2.0, "gyroscope_y": 138.0, "gyroscope_z": -86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:07.676000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 21.0, "acceleration_z": -66.0, "gyroscope_x": -2.0, "gyroscope_y": -18.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:13.573000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 46.0, "acceleration_z": -60.0, "gyroscope_x": 739.0, "gyroscope_y": 340.0, "gyroscope_z": -126.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:13.625000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 220.0, "acceleration_y": 31.0, "acceleration_z": -179.0, "gyroscope_x": 904.0, "gyroscope_y": 424.0, "gyroscope_z": -130.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:13.677000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 52.0, "acceleration_z": -39.0, "gyroscope_x": 869.0, "gyroscope_y": 526.0, "gyroscope_z": -122.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:13.729000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 73.0, "acceleration_z": -31.0, "gyroscope_x": 668.0, "gyroscope_y": 438.0, "gyroscope_z": -38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:19.269000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 21.0, "acceleration_z": -48.0, "gyroscope_x": 126.0, "gyroscope_y": 149.0, "gyroscope_z": -57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:19.321000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 185.0, "acceleration_y": -4.0, "acceleration_z": -8.0, "gyroscope_x": 60.0, "gyroscope_y": 258.0, "gyroscope_z": -36.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:19.373000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 42.0, "acceleration_z": -66.0, "gyroscope_x": 180.0, "gyroscope_y": 97.0, "gyroscope_z": -71.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:19.425000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 39.0, "acceleration_z": -19.0, "gyroscope_x": 13.0, "gyroscope_y": 71.0, "gyroscope_z": 104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:25.431000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 8.0, "acceleration_z": 83.0, "gyroscope_x": -383.0, "gyroscope_y": -120.0, "gyroscope_z": -110.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:25.483000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 58.0, "acceleration_z": -21.0, "gyroscope_x": -176.0, "gyroscope_y": -284.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:25.535000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 15.0, "acceleration_z": 59.0, "gyroscope_x": -259.0, "gyroscope_y": -58.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:25.587000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 234.0, "acceleration_y": 34.0, "acceleration_z": -63.0, "gyroscope_x": -112.0, "gyroscope_y": -7.0, "gyroscope_z": -126.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:31.285000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 316.0, "acceleration_y": 65.0, "acceleration_z": 57.0, "gyroscope_x": -98.0, "gyroscope_y": 84.0, "gyroscope_z": 32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:31.337000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 15.0, "acceleration_z": 26.0, "gyroscope_x": -228.0, "gyroscope_y": 53.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:31.389000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 35.0, "acceleration_z": 15.0, "gyroscope_x": -138.0, "gyroscope_y": 29.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:31.441000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 54.0, "acceleration_z": -46.0, "gyroscope_x": -246.0, "gyroscope_y": 34.0, "gyroscope_z": -96.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:37.165000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 27.0, "acceleration_z": -31.0, "gyroscope_x": -380.0, "gyroscope_y": 37.0, "gyroscope_z": -66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:37.217000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 18.0, "acceleration_z": 3.0, "gyroscope_x": -451.0, "gyroscope_y": 26.0, "gyroscope_z": -98.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:37.269000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 49.0, "acceleration_z": -44.0, "gyroscope_x": -328.0, "gyroscope_y": 140.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:37.321000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 288.0, "acceleration_y": 57.0, "acceleration_z": -81.0, "gyroscope_x": -318.0, "gyroscope_y": 9.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:43.129000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 224.0, "acceleration_y": 11.0, "acceleration_z": 36.0, "gyroscope_x": -150.0, "gyroscope_y": 144.0, "gyroscope_z": 34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:43.181000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": -28.0, "acceleration_z": -62.0, "gyroscope_x": -85.0, "gyroscope_y": 70.0, "gyroscope_z": 50.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:43.233000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 283.0, "acceleration_y": 16.0, "acceleration_z": 139.0, "gyroscope_x": -229.0, "gyroscope_y": 38.0, "gyroscope_z": -98.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:43.285000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 9.0, "acceleration_z": -42.0, "gyroscope_x": -57.0, "gyroscope_y": -43.0, "gyroscope_z": 7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:48.876000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 9.0, "acceleration_z": -76.0, "gyroscope_x": -53.0, "gyroscope_y": 17.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:48.928000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 295.0, "acceleration_y": -18.0, "acceleration_z": 130.0, "gyroscope_x": -216.0, "gyroscope_y": 4.0, "gyroscope_z": 29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:48.980000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 303.0, "acceleration_y": 19.0, "acceleration_z": -87.0, "gyroscope_x": 61.0, "gyroscope_y": 74.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:49.032000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 308.0, "acceleration_y": 33.0, "acceleration_z": -11.0, "gyroscope_x": 30.0, "gyroscope_y": 192.0, "gyroscope_z": 35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:54.613000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 284.0, "acceleration_y": 20.0, "acceleration_z": -13.0, "gyroscope_x": -473.0, "gyroscope_y": -141.0, "gyroscope_z": -135.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:54.665000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": -8.0, "acceleration_z": -127.0, "gyroscope_x": -404.0, "gyroscope_y": -135.0, "gyroscope_z": -142.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:54.717000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 33.0, "acceleration_z": -24.0, "gyroscope_x": -386.0, "gyroscope_y": -82.0, "gyroscope_z": -81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:54.769000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": -22.0, "acceleration_z": -69.0, "gyroscope_x": -337.0, "gyroscope_y": -124.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:00.305000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 37.0, "acceleration_z": -65.0, "gyroscope_x": 150.0, "gyroscope_y": 53.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:00.357000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 17.0, "acceleration_z": -55.0, "gyroscope_x": 213.0, "gyroscope_y": -83.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:00.409000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 20.0, "acceleration_z": -12.0, "gyroscope_x": 294.0, "gyroscope_y": -11.0, "gyroscope_z": -19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:00.461000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 24.0, "acceleration_z": 2.0, "gyroscope_x": 305.0, "gyroscope_y": -14.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:06.138000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 312.0, "acceleration_y": 29.0, "acceleration_z": -52.0, "gyroscope_x": -273.0, "gyroscope_y": 137.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:06.190000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 21.0, "acceleration_z": 69.0, "gyroscope_x": -360.0, "gyroscope_y": -66.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:06.242000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 24.0, "acceleration_z": -33.0, "gyroscope_x": -294.0, "gyroscope_y": -99.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:06.294000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 35.0, "acceleration_z": -28.0, "gyroscope_x": -174.0, "gyroscope_y": -57.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:12.468000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 13.0, "acceleration_z": -4.0, "gyroscope_x": -6.0, "gyroscope_y": 83.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:12.520000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 25.0, "acceleration_z": -23.0, "gyroscope_x": 68.0, "gyroscope_y": 101.0, "gyroscope_z": 26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:12.572000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 275.0, "acceleration_y": 37.0, "acceleration_z": -40.0, "gyroscope_x": 47.0, "gyroscope_y": 32.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:12.624000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 19.0, "acceleration_z": -27.0, "gyroscope_x": 41.0, "gyroscope_y": 117.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:18.470000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 224.0, "acceleration_y": 36.0, "acceleration_z": 54.0, "gyroscope_x": 324.0, "gyroscope_y": 220.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:18.522000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 62.0, "acceleration_z": 3.0, "gyroscope_x": 268.0, "gyroscope_y": 289.0, "gyroscope_z": 52.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:18.574000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 277.0, "acceleration_y": 22.0, "acceleration_z": 25.0, "gyroscope_x": 131.0, "gyroscope_y": 89.0, "gyroscope_z": 37.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:18.626000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": 53.0, "acceleration_z": 11.0, "gyroscope_x": 9.0, "gyroscope_y": 166.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:24.278000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 263.0, "acceleration_y": 27.0, "acceleration_z": -83.0, "gyroscope_x": -51.0, "gyroscope_y": -133.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:24.330000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 140.0, "acceleration_y": 12.0, "acceleration_z": 32.0, "gyroscope_x": -78.0, "gyroscope_y": -90.0, "gyroscope_z": -104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:24.382000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": 136.0, "acceleration_z": 146.0, "gyroscope_x": 68.0, "gyroscope_y": 279.0, "gyroscope_z": -87.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:24.434000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 214.0, "acceleration_y": 88.0, "acceleration_z": -109.0, "gyroscope_x": 40.0, "gyroscope_y": -18.0, "gyroscope_z": -41.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:30.186000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 372.0, "acceleration_y": 101.0, "acceleration_z": 4.0, "gyroscope_x": 59.0, "gyroscope_y": 139.0, "gyroscope_z": 91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:30.238000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": -13.0, "acceleration_z": -16.0, "gyroscope_x": 118.0, "gyroscope_y": 291.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:30.290000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 305.0, "acceleration_y": 26.0, "acceleration_z": -125.0, "gyroscope_x": 96.0, "gyroscope_y": 289.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:30.342000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 294.0, "acceleration_y": 35.0, "acceleration_z": -53.0, "gyroscope_x": 165.0, "gyroscope_y": 78.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:36.001000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 304.0, "acceleration_y": 9.0, "acceleration_z": 2.0, "gyroscope_x": 77.0, "gyroscope_y": 326.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:36.053000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": 37.0, "acceleration_z": -30.0, "gyroscope_x": 85.0, "gyroscope_y": 240.0, "gyroscope_z": -128.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:36.105000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 170.0, "acceleration_y": 3.0, "acceleration_z": -47.0, "gyroscope_x": 86.0, "gyroscope_y": 272.0, "gyroscope_z": -144.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:36.157000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 49.0, "acceleration_z": -58.0, "gyroscope_x": -21.0, "gyroscope_y": 399.0, "gyroscope_z": -122.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:42.240000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 327.0, "acceleration_y": -14.0, "acceleration_z": -131.0, "gyroscope_x": -65.0, "gyroscope_y": 90.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:42.292000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": -2.0, "acceleration_z": -89.0, "gyroscope_x": 96.0, "gyroscope_y": 337.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:42.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": -14.0, "acceleration_z": -158.0, "gyroscope_x": 25.0, "gyroscope_y": 63.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:42.396000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 36.0, "acceleration_z": -58.0, "gyroscope_x": 6.0, "gyroscope_y": 76.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:48.296000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 48.0, "acceleration_z": -41.0, "gyroscope_x": 32.0, "gyroscope_y": -51.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:48.348000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 306.0, "acceleration_y": 32.0, "acceleration_z": -46.0, "gyroscope_x": 85.0, "gyroscope_y": -90.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:48.400000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 290.0, "acceleration_y": 60.0, "acceleration_z": 21.0, "gyroscope_x": -15.0, "gyroscope_y": 252.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:48.452000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 32.0, "acceleration_z": -7.0, "gyroscope_x": 33.0, "gyroscope_y": 184.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:53.991000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 7.0, "acceleration_z": -26.0, "gyroscope_x": -44.0, "gyroscope_y": -19.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:54.043000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 240.0, "acceleration_y": 27.0, "acceleration_z": -1.0, "gyroscope_x": 8.0, "gyroscope_y": -81.0, "gyroscope_z": -89.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:54.095000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 276.0, "acceleration_y": 23.0, "acceleration_z": -39.0, "gyroscope_x": 27.0, "gyroscope_y": -18.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:54.147000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 212.0, "acceleration_y": 19.0, "acceleration_z": -5.0, "gyroscope_x": 68.0, "gyroscope_y": -108.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:59.883000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 31.0, "acceleration_z": 1.0, "gyroscope_x": -6.0, "gyroscope_y": 86.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:59.935000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 272.0, "acceleration_y": 20.0, "acceleration_z": -29.0, "gyroscope_x": 16.0, "gyroscope_y": 99.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:10:59.987000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 23.0, "acceleration_z": -27.0, "gyroscope_x": 16.0, "gyroscope_y": 92.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:00.039000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 25.0, "acceleration_z": -17.0, "gyroscope_x": -5.0, "gyroscope_y": 106.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:05.848000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 266.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -254.0, "gyroscope_y": -38.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:05.900000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 46.0, "acceleration_z": -12.0, "gyroscope_x": -319.0, "gyroscope_y": -26.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:05.952000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 54.0, "acceleration_z": -15.0, "gyroscope_x": -230.0, "gyroscope_y": -83.0, "gyroscope_z": 33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:06.004000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 234.0, "acceleration_y": 41.0, "acceleration_z": -30.0, "gyroscope_x": -254.0, "gyroscope_y": -85.0, "gyroscope_z": -43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:11.556000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 15.0, "acceleration_z": -67.0, "gyroscope_x": -141.0, "gyroscope_y": 16.0, "gyroscope_z": -35.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:11.608000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 232.0, "acceleration_y": -11.0, "acceleration_z": 15.0, "gyroscope_x": -216.0, "gyroscope_y": -11.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:11.660000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": 50.0, "acceleration_z": -27.0, "gyroscope_x": -143.0, "gyroscope_y": -12.0, "gyroscope_z": 43.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:11.712000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 17.0, "acceleration_z": -7.0, "gyroscope_x": -95.0, "gyroscope_y": 61.0, "gyroscope_z": 66.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:17.281000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 41.0, "acceleration_z": -32.0, "gyroscope_x": 9.0, "gyroscope_y": 76.0, "gyroscope_z": 20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:17.333000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 321.0, "acceleration_y": 37.0, "acceleration_z": -44.0, "gyroscope_x": -73.0, "gyroscope_y": 123.0, "gyroscope_z": 77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:17.385000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": -16.0, "acceleration_z": -7.0, "gyroscope_x": -16.0, "gyroscope_y": 100.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:17.437000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 7.0, "acceleration_z": -63.0, "gyroscope_x": 8.0, "gyroscope_y": 157.0, "gyroscope_z": 49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:23.507000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 49.0, "acceleration_z": -41.0, "gyroscope_x": -570.0, "gyroscope_y": 219.0, "gyroscope_z": -133.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:23.559000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": 46.0, "acceleration_z": -83.0, "gyroscope_x": -486.0, "gyroscope_y": 324.0, "gyroscope_z": -105.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:23.611000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 21.0, "acceleration_z": 24.0, "gyroscope_x": -659.0, "gyroscope_y": 80.0, "gyroscope_z": -106.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:23.663000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": 73.0, "acceleration_z": -38.0, "gyroscope_x": -699.0, "gyroscope_y": 156.0, "gyroscope_z": -157.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:29.185000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 26.0, "acceleration_z": 21.0, "gyroscope_x": -85.0, "gyroscope_y": 103.0, "gyroscope_z": 25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:29.237000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 1.0, "acceleration_z": -11.0, "gyroscope_x": -26.0, "gyroscope_y": 49.0, "gyroscope_z": 19.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:29.289000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -19.0, "gyroscope_x": -67.0, "gyroscope_y": 60.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:29.341000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 45.0, "acceleration_z": -46.0, "gyroscope_x": -34.0, "gyroscope_y": 118.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:22.153000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 30.0, "acceleration_z": -40.0, "gyroscope_x": -9.0, "gyroscope_y": 54.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:27.360000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 32.0, "acceleration_z": -60.0, "gyroscope_x": -18.0, "gyroscope_y": 29.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:27.412000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 32.0, "acceleration_z": -59.0, "gyroscope_x": -8.0, "gyroscope_y": 40.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:27.464000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 33.0, "acceleration_z": -61.0, "gyroscope_x": -14.0, "gyroscope_y": 52.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:27.516000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 32.0, "acceleration_z": -62.0, "gyroscope_x": -9.0, "gyroscope_y": 46.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:33.229000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -43.0, "gyroscope_x": -17.0, "gyroscope_y": 69.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:33.281000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 32.0, "acceleration_z": -44.0, "gyroscope_x": -13.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:33.333000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 33.0, "acceleration_z": -47.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:33.385000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 33.0, "acceleration_z": -42.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:38.943000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 33.0, "acceleration_z": -57.0, "gyroscope_x": -8.0, "gyroscope_y": 46.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:38.995000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 33.0, "acceleration_z": -50.0, "gyroscope_x": -14.0, "gyroscope_y": 50.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:39.047000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 33.0, "acceleration_z": -51.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:39.099000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 33.0, "acceleration_z": -55.0, "gyroscope_x": -18.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:44.832000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 34.0, "acceleration_z": -49.0, "gyroscope_x": -14.0, "gyroscope_y": 68.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:44.884000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 33.0, "acceleration_z": -52.0, "gyroscope_x": -15.0, "gyroscope_y": 69.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:44.936000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 32.0, "acceleration_z": -51.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:44.988000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 32.0, "acceleration_z": -50.0, "gyroscope_x": -19.0, "gyroscope_y": 70.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:50.524000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 33.0, "acceleration_z": -53.0, "gyroscope_x": -14.0, "gyroscope_y": 49.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:50.576000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 32.0, "acceleration_z": -54.0, "gyroscope_x": -11.0, "gyroscope_y": 48.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:50.628000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 33.0, "acceleration_z": -52.0, "gyroscope_x": -14.0, "gyroscope_y": 58.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:50.680000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 31.0, "acceleration_z": -57.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:56.285000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 36.0, "acceleration_z": -82.0, "gyroscope_x": -28.0, "gyroscope_y": 109.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:56.337000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 33.0, "acceleration_z": -74.0, "gyroscope_x": -21.0, "gyroscope_y": 95.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:56.389000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 32.0, "acceleration_z": -73.0, "gyroscope_x": -17.0, "gyroscope_y": 216.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:09:56.441000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 30.0, "acceleration_z": -57.0, "gyroscope_x": -18.0, "gyroscope_y": 138.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:11:11.464000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 36.0, "acceleration_z": -86.0, "gyroscope_x": -14.0, "gyroscope_y": 43.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:50.224000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 21.0, "acceleration_z": -68.0, "gyroscope_x": -11.0, "gyroscope_y": 44.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:50.276000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 11.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 44.0, "gyroscope_z": -40.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:50.328000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 16.0, "acceleration_z": -64.0, "gyroscope_x": -13.0, "gyroscope_y": 44.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:50.380000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 14.0, "acceleration_z": -71.0, "gyroscope_x": -17.0, "gyroscope_y": 32.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:56.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 21.0, "acceleration_z": -56.0, "gyroscope_x": -18.0, "gyroscope_y": 39.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:56.396000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 20.0, "acceleration_z": -56.0, "gyroscope_x": -22.0, "gyroscope_y": 35.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:56.448000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 20.0, "acceleration_z": -63.0, "gyroscope_x": -17.0, "gyroscope_y": 32.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:15:56.500000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 15.0, "acceleration_z": -61.0, "gyroscope_x": -24.0, "gyroscope_y": 31.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:02.057000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 23.0, "acceleration_z": -63.0, "gyroscope_x": -36.0, "gyroscope_y": 83.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:02.109000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 23.0, "acceleration_z": -62.0, "gyroscope_x": -18.0, "gyroscope_y": 81.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:02.161000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 25.0, "acceleration_z": -60.0, "gyroscope_x": -17.0, "gyroscope_y": 79.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:02.213000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 24.0, "acceleration_z": -61.0, "gyroscope_x": -21.0, "gyroscope_y": 66.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:09.356000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 19.0, "acceleration_z": -61.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:09.408000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 18.0, "acceleration_z": -62.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:09.460000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 20.0, "acceleration_z": -63.0, "gyroscope_x": -7.0, "gyroscope_y": 71.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:09.512000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 18.0, "acceleration_z": -60.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:15.210000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 19.0, "acceleration_z": -60.0, "gyroscope_x": -17.0, "gyroscope_y": 10.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:15.262000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 19.0, "acceleration_z": -79.0, "gyroscope_x": -16.0, "gyroscope_y": 18.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:15.314000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 15.0, "acceleration_z": -72.0, "gyroscope_x": -27.0, "gyroscope_y": 40.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:15.366000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 16.0, "acceleration_z": -70.0, "gyroscope_x": -10.0, "gyroscope_y": 52.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:20.929000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": 58.0, "acceleration_z": -61.0, "gyroscope_x": 444.0, "gyroscope_y": 35.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:20.981000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 213.0, "acceleration_y": 39.0, "acceleration_z": -34.0, "gyroscope_x": 417.0, "gyroscope_y": 172.0, "gyroscope_z": -32.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:21.033000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 158.0, "acceleration_y": 2.0, "acceleration_z": 10.0, "gyroscope_x": 446.0, "gyroscope_y": 114.0, "gyroscope_z": -136.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:21.085000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 312.0, "acceleration_y": 98.0, "acceleration_z": -60.0, "gyroscope_x": 472.0, "gyroscope_y": 136.0, "gyroscope_z": -57.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:26.585000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 16.0, "acceleration_z": -39.0, "gyroscope_x": 100.0, "gyroscope_y": 171.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:26.637000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 44.0, "acceleration_z": -19.0, "gyroscope_x": 12.0, "gyroscope_y": 120.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:26.689000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 24.0, "acceleration_z": -39.0, "gyroscope_x": -44.0, "gyroscope_y": 169.0, "gyroscope_z": -20.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:26.741000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 287.0, "acceleration_y": 50.0, "acceleration_z": -58.0, "gyroscope_x": -53.0, "gyroscope_y": 185.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:32.288000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 34.0, "acceleration_z": -74.0, "gyroscope_x": -14.0, "gyroscope_y": 52.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:32.340000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 32.0, "acceleration_z": -73.0, "gyroscope_x": -4.0, "gyroscope_y": 40.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:32.392000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 39.0, "acceleration_z": -63.0, "gyroscope_x": -7.0, "gyroscope_y": 30.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:32.444000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 44.0, "acceleration_z": -67.0, "gyroscope_x": -16.0, "gyroscope_y": 38.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:37.971000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 35.0, "acceleration_z": -71.0, "gyroscope_x": -16.0, "gyroscope_y": 68.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:38.023000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 35.0, "acceleration_z": -73.0, "gyroscope_x": -13.0, "gyroscope_y": 70.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:38.075000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 36.0, "acceleration_z": -70.0, "gyroscope_x": -19.0, "gyroscope_y": 59.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:38.127000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 37.0, "acceleration_z": -73.0, "gyroscope_x": -12.0, "gyroscope_y": 68.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:43.689000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 36.0, "acceleration_z": -78.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:43.741000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 36.0, "acceleration_z": -76.0, "gyroscope_x": -12.0, "gyroscope_y": 69.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:43.793000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 35.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:43.845000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 35.0, "acceleration_z": -77.0, "gyroscope_x": -16.0, "gyroscope_y": 67.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:49.485000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 34.0, "acceleration_z": -73.0, "gyroscope_x": -13.0, "gyroscope_y": 79.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:49.537000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 32.0, "acceleration_z": -71.0, "gyroscope_x": -10.0, "gyroscope_y": 75.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:49.589000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 33.0, "acceleration_z": -66.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:49.641000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 33.0, "acceleration_z": -70.0, "gyroscope_x": -11.0, "gyroscope_y": 60.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:55.188000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 33.0, "acceleration_z": -68.0, "gyroscope_x": -12.0, "gyroscope_y": 71.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:55.240000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 34.0, "acceleration_z": -71.0, "gyroscope_x": -11.0, "gyroscope_y": 70.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:55.292000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 35.0, "acceleration_z": -67.0, "gyroscope_x": -17.0, "gyroscope_y": 62.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:16:55.344000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 35.0, "acceleration_z": -71.0, "gyroscope_x": -13.0, "gyroscope_y": 68.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:00.875000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 34.0, "acceleration_z": -72.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:00.927000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 36.0, "acceleration_z": -72.0, "gyroscope_x": -17.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:00.979000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 35.0, "acceleration_z": -68.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:01.031000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 35.0, "acceleration_z": -73.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:06.549000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 38.0, "acceleration_z": -280.0, "gyroscope_x": -176.0, "gyroscope_y": 353.0, "gyroscope_z": 122.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:06.601000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 55.0, "acceleration_z": 21.0, "gyroscope_x": -228.0, "gyroscope_y": 180.0, "gyroscope_z": 67.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:06.653000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 90.0, "acceleration_z": -40.0, "gyroscope_x": -338.0, "gyroscope_y": 178.0, "gyroscope_z": 81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:06.705000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 211.0, "acceleration_y": 93.0, "acceleration_z": -65.0, "gyroscope_x": -309.0, "gyroscope_y": -33.0, "gyroscope_z": 21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:12.237000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 72.0, "acceleration_z": 0.0, "gyroscope_x": 125.0, "gyroscope_y": 43.0, "gyroscope_z": -58.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:12.289000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 42.0, "acceleration_z": -52.0, "gyroscope_x": 28.0, "gyroscope_y": 109.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:12.341000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 236.0, "acceleration_y": -29.0, "acceleration_z": -30.0, "gyroscope_x": 66.0, "gyroscope_y": -71.0, "gyroscope_z": 98.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:12.393000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 188.0, "acceleration_y": 77.0, "acceleration_z": -61.0, "gyroscope_x": -14.0, "gyroscope_y": 51.0, "gyroscope_z": -90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:17.954000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 42.0, "acceleration_z": -78.0, "gyroscope_x": -84.0, "gyroscope_y": 75.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:18.006000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 56.0, "acceleration_z": -86.0, "gyroscope_x": -51.0, "gyroscope_y": 81.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:18.058000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 51.0, "acceleration_z": -72.0, "gyroscope_x": -89.0, "gyroscope_y": 101.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:18.110000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 58.0, "acceleration_z": -73.0, "gyroscope_x": -67.0, "gyroscope_y": 113.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:23.649000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 269.0, "acceleration_y": -7.0, "acceleration_z": -43.0, "gyroscope_x": 2.0, "gyroscope_y": 185.0, "gyroscope_z": 12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:23.701000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 39.0, "acceleration_z": -6.0, "gyroscope_x": 12.0, "gyroscope_y": 164.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:23.753000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 33.0, "acceleration_z": -3.0, "gyroscope_x": -16.0, "gyroscope_y": 89.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:23.805000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 44.0, "acceleration_z": -4.0, "gyroscope_x": -84.0, "gyroscope_y": 68.0, "gyroscope_z": 8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:29.337000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": 31.0, "acceleration_z": -64.0, "gyroscope_x": 229.0, "gyroscope_y": 107.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:29.389000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 224.0, "acceleration_y": 40.0, "acceleration_z": -148.0, "gyroscope_x": 222.0, "gyroscope_y": 14.0, "gyroscope_z": 7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:29.441000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": -3.0, "acceleration_z": -25.0, "gyroscope_x": 231.0, "gyroscope_y": 54.0, "gyroscope_z": -26.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:29.493000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 22.0, "acceleration_z": -13.0, "gyroscope_x": 243.0, "gyroscope_y": 70.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:35.779000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 29.0, "acceleration_z": -39.0, "gyroscope_x": 144.0, "gyroscope_y": -15.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:35.831000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 205.0, "acceleration_y": -5.0, "acceleration_z": -63.0, "gyroscope_x": 127.0, "gyroscope_y": -235.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:35.883000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 233.0, "acceleration_y": 2.0, "acceleration_z": -16.0, "gyroscope_x": 217.0, "gyroscope_y": -77.0, "gyroscope_z": -77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:35.935000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 101.0, "acceleration_z": 93.0, "gyroscope_x": 198.0, "gyroscope_y": 9.0, "gyroscope_z": -76.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:41.459000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": -18.0, "acceleration_z": -55.0, "gyroscope_x": 520.0, "gyroscope_y": 259.0, "gyroscope_z": -169.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:41.511000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 291.0, "acceleration_y": 46.0, "acceleration_z": 18.0, "gyroscope_x": 350.0, "gyroscope_y": 286.0, "gyroscope_z": -91.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:41.563000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": -1.0, "acceleration_z": -31.0, "gyroscope_x": 448.0, "gyroscope_y": 211.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:41.615000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 244.0, "acceleration_y": 28.0, "acceleration_z": -37.0, "gyroscope_x": 429.0, "gyroscope_y": 191.0, "gyroscope_z": -118.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:47.250000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 45.0, "acceleration_z": -151.0, "gyroscope_x": -149.0, "gyroscope_y": 7.0, "gyroscope_z": -61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:47.302000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 292.0, "acceleration_y": 41.0, "acceleration_z": -6.0, "gyroscope_x": -231.0, "gyroscope_y": 82.0, "gyroscope_z": -49.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:47.354000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 293.0, "acceleration_y": -40.0, "acceleration_z": 22.0, "gyroscope_x": -120.0, "gyroscope_y": -64.0, "gyroscope_z": -122.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:47.406000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 67.0, "acceleration_z": 14.0, "gyroscope_x": -155.0, "gyroscope_y": 56.0, "gyroscope_z": -104.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:52.928000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 10.0, "acceleration_z": -66.0, "gyroscope_x": -122.0, "gyroscope_y": 109.0, "gyroscope_z": 65.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:52.980000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 327.0, "acceleration_y": 33.0, "acceleration_z": -26.0, "gyroscope_x": -251.0, "gyroscope_y": 77.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:53.032000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 37.0, "acceleration_z": -121.0, "gyroscope_x": -194.0, "gyroscope_y": 22.0, "gyroscope_z": 92.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:53.084000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": 91.0, "acceleration_z": -85.0, "gyroscope_x": -211.0, "gyroscope_y": 161.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:58.596000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 195.0, "acceleration_y": 29.0, "acceleration_z": 43.0, "gyroscope_x": -237.0, "gyroscope_y": 55.0, "gyroscope_z": 148.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:58.648000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 187.0, "acceleration_y": -9.0, "acceleration_z": 34.0, "gyroscope_x": -130.0, "gyroscope_y": -80.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:58.700000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 315.0, "acceleration_y": 75.0, "acceleration_z": -25.0, "gyroscope_x": -132.0, "gyroscope_y": 161.0, "gyroscope_z": 158.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:17:58.752000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 34.0, "acceleration_z": -69.0, "gyroscope_x": -164.0, "gyroscope_y": -44.0, "gyroscope_z": -39.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:04.353000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 41.0, "acceleration_z": -49.0, "gyroscope_x": 5.0, "gyroscope_y": 52.0, "gyroscope_z": -110.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:04.405000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 6.0, "acceleration_z": 13.0, "gyroscope_x": -80.0, "gyroscope_y": -22.0, "gyroscope_z": 74.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:04.457000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 216.0, "acceleration_y": -34.0, "acceleration_z": 93.0, "gyroscope_x": -209.0, "gyroscope_y": -65.0, "gyroscope_z": 158.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:04.509000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 4.0, "acceleration_z": -42.0, "gyroscope_x": -206.0, "gyroscope_y": 44.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:10.033000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 288.0, "acceleration_y": -6.0, "acceleration_z": 70.0, "gyroscope_x": 11.0, "gyroscope_y": 88.0, "gyroscope_z": -115.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:10.085000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 35.0, "acceleration_z": -9.0, "gyroscope_x": 71.0, "gyroscope_y": -25.0, "gyroscope_z": 90.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:10.137000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 294.0, "acceleration_y": 41.0, "acceleration_z": -48.0, "gyroscope_x": -58.0, "gyroscope_y": 268.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:10.189000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 4.0, "acceleration_z": 5.0, "gyroscope_x": -125.0, "gyroscope_y": -47.0, "gyroscope_z": 77.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:15.729000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 48.0, "acceleration_z": 3.0, "gyroscope_x": 640.0, "gyroscope_y": 39.0, "gyroscope_z": -258.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:15.781000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 327.0, "acceleration_y": 40.0, "acceleration_z": -26.0, "gyroscope_x": 732.0, "gyroscope_y": 43.0, "gyroscope_z": -196.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:15.833000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 192.0, "acceleration_y": 38.0, "acceleration_z": -38.0, "gyroscope_x": 745.0, "gyroscope_y": -12.0, "gyroscope_z": -263.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:15.885000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 12.0, "acceleration_z": -44.0, "gyroscope_x": 799.0, "gyroscope_y": -8.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:21.399000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 44.0, "acceleration_z": -32.0, "gyroscope_x": -24.0, "gyroscope_y": 188.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:21.451000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 31.0, "acceleration_z": -15.0, "gyroscope_x": -22.0, "gyroscope_y": 195.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:21.503000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 30.0, "acceleration_z": -12.0, "gyroscope_x": -35.0, "gyroscope_y": 209.0, "gyroscope_z": 14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:21.555000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 20.0, "acceleration_z": -1.0, "gyroscope_x": -47.0, "gyroscope_y": 185.0, "gyroscope_z": 28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:27.299000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -18.0, "gyroscope_x": -16.0, "gyroscope_y": 58.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:27.351000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 34.0, "acceleration_z": -20.0, "gyroscope_x": -14.0, "gyroscope_y": 56.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:27.403000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 34.0, "acceleration_z": -20.0, "gyroscope_x": -13.0, "gyroscope_y": 53.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:27.455000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -18.0, "gyroscope_x": -19.0, "gyroscope_y": 50.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:32.972000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -21.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:33.024000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 35.0, "acceleration_z": -23.0, "gyroscope_x": -10.0, "gyroscope_y": 61.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:33.076000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -20.0, "gyroscope_x": -12.0, "gyroscope_y": 60.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:33.128000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:39.381000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -22.0, "gyroscope_x": -13.0, "gyroscope_y": 62.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:39.433000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -25.0, "gyroscope_x": -14.0, "gyroscope_y": 60.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:39.485000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -22.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:39.537000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -24.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:45.518000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -22.0, "gyroscope_x": -14.0, "gyroscope_y": 75.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:45.570000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -22.0, "gyroscope_x": -19.0, "gyroscope_y": 75.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:45.622000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 31.0, "acceleration_z": -16.0, "gyroscope_x": -19.0, "gyroscope_y": 61.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:45.674000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 32.0, "acceleration_z": -21.0, "gyroscope_x": -28.0, "gyroscope_y": 58.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:51.300000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 36.0, "acceleration_z": -27.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:51.352000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -26.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:51.404000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -28.0, "gyroscope_x": -17.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:51.456000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -26.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:57.807000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -30.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:57.859000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -31.0, "gyroscope_x": -12.0, "gyroscope_y": 72.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:57.911000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -31.0, "gyroscope_x": -14.0, "gyroscope_y": 75.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:18:57.963000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -30.0, "gyroscope_x": -13.0, "gyroscope_y": 75.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:03.462000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -25.0, "gyroscope_x": -16.0, "gyroscope_y": 62.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:03.514000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 35.0, "acceleration_z": -25.0, "gyroscope_x": -15.0, "gyroscope_y": 59.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:03.566000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -27.0, "gyroscope_x": -13.0, "gyroscope_y": 59.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:03.618000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -27.0, "gyroscope_x": -14.0, "gyroscope_y": 60.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:09.121000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 34.0, "acceleration_z": -28.0, "gyroscope_x": -11.0, "gyroscope_y": 65.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:09.173000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -27.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:09.225000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -27.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:09.277000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -26.0, "gyroscope_x": -18.0, "gyroscope_y": 65.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:14.830000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -17.0, "gyroscope_y": 62.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:14.882000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -24.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:14.934000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -25.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:14.986000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -27.0, "gyroscope_x": -11.0, "gyroscope_y": 69.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:20.543000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -25.0, "gyroscope_x": -18.0, "gyroscope_y": 78.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:20.595000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 35.0, "acceleration_z": -19.0, "gyroscope_x": -15.0, "gyroscope_y": 78.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:20.647000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 38.0, "acceleration_z": -32.0, "gyroscope_x": -16.0, "gyroscope_y": 80.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:20.699000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 33.0, "acceleration_z": -25.0, "gyroscope_x": -19.0, "gyroscope_y": 80.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:26.323000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -7.0, "gyroscope_x": -31.0, "gyroscope_y": 48.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:26.375000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 35.0, "acceleration_z": -11.0, "gyroscope_x": -20.0, "gyroscope_y": 41.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:26.427000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 33.0, "acceleration_z": -1.0, "gyroscope_x": -28.0, "gyroscope_y": 33.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:26.479000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -10.0, "gyroscope_x": -28.0, "gyroscope_y": 25.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:31.993000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 36.0, "acceleration_z": -16.0, "gyroscope_x": -24.0, "gyroscope_y": 50.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:32.045000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 39.0, "acceleration_z": -19.0, "gyroscope_x": -16.0, "gyroscope_y": 56.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:32.097000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -21.0, "gyroscope_x": -14.0, "gyroscope_y": 54.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:32.149000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -18.0, "gyroscope_x": -8.0, "gyroscope_y": 56.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:37.760000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 36.0, "acceleration_z": -24.0, "gyroscope_x": -15.0, "gyroscope_y": 57.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:37.812000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 35.0, "acceleration_z": -25.0, "gyroscope_x": -15.0, "gyroscope_y": 55.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:37.864000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -25.0, "gyroscope_x": -16.0, "gyroscope_y": 57.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:37.916000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -25.0, "gyroscope_x": -17.0, "gyroscope_y": 58.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:43.434000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -26.0, "gyroscope_x": -13.0, "gyroscope_y": 59.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:43.486000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -24.0, "gyroscope_x": -13.0, "gyroscope_y": 56.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:43.538000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -25.0, "gyroscope_x": -14.0, "gyroscope_y": 57.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:43.590000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -26.0, "gyroscope_x": -11.0, "gyroscope_y": 60.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:49.085000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -22.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:49.137000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 37.0, "acceleration_z": -21.0, "gyroscope_x": -20.0, "gyroscope_y": 62.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:49.189000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -22.0, "gyroscope_x": -14.0, "gyroscope_y": 60.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:49.241000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -12.0, "gyroscope_y": 62.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:54.758000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 34.0, "acceleration_z": -23.0, "gyroscope_x": -11.0, "gyroscope_y": 65.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:54.810000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:54.862000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -25.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:19:54.914000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:00.445000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -19.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:00.497000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 34.0, "acceleration_z": -23.0, "gyroscope_x": -16.0, "gyroscope_y": 61.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:00.549000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -19.0, "gyroscope_x": -15.0, "gyroscope_y": 61.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:00.601000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -24.0, "gyroscope_x": -13.0, "gyroscope_y": 61.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:06.112000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -21.0, "gyroscope_x": -12.0, "gyroscope_y": 67.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:06.164000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 36.0, "acceleration_z": -24.0, "gyroscope_x": -16.0, "gyroscope_y": 62.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:06.216000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -24.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:06.268000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -21.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:11.798000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 35.0, "acceleration_z": -23.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:11.850000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 34.0, "acceleration_z": -22.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:11.902000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -22.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:11.954000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -22.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:17.483000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -26.0, "gyroscope_x": -16.0, "gyroscope_y": 70.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:17.535000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -25.0, "gyroscope_x": -17.0, "gyroscope_y": 74.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:17.587000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -26.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:17.639000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -18.0, "gyroscope_y": 70.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:23.169000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 36.0, "acceleration_z": -26.0, "gyroscope_x": -16.0, "gyroscope_y": 68.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:23.221000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -29.0, "gyroscope_x": -13.0, "gyroscope_y": 70.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:23.273000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -25.0, "gyroscope_x": -13.0, "gyroscope_y": 69.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:23.325000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 38.0, "acceleration_z": -26.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:28.852000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -26.0, "gyroscope_x": -17.0, "gyroscope_y": 68.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:28.904000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 35.0, "acceleration_z": -27.0, "gyroscope_x": -14.0, "gyroscope_y": 69.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:28.956000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -27.0, "gyroscope_x": -14.0, "gyroscope_y": 68.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:29.008000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -25.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:34.531000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -30.0, "gyroscope_x": -17.0, "gyroscope_y": 101.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:34.583000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -27.0, "gyroscope_x": -22.0, "gyroscope_y": 102.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:34.635000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -28.0, "gyroscope_x": -17.0, "gyroscope_y": 98.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:34.687000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 37.0, "acceleration_z": -22.0, "gyroscope_x": -16.0, "gyroscope_y": 102.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:40.297000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -23.0, "gyroscope_x": -16.0, "gyroscope_y": 70.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:40.349000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -21.0, "gyroscope_x": -17.0, "gyroscope_y": 69.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:40.401000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 35.0, "acceleration_z": -20.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:40.453000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -22.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:46.395000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -20.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:46.447000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 36.0, "acceleration_z": -21.0, "gyroscope_x": -14.0, "gyroscope_y": 69.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:46.499000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 37.0, "acceleration_z": -22.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:46.551000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 37.0, "acceleration_z": -22.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:53.276000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 25.0, "acceleration_z": 13.0, "gyroscope_x": 38.0, "gyroscope_y": 79.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:53.328000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 28.0, "acceleration_z": 37.0, "gyroscope_x": 19.0, "gyroscope_y": 39.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:53.380000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 36.0, "acceleration_z": 27.0, "gyroscope_x": 8.0, "gyroscope_y": 8.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:53.432000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 51.0, "acceleration_z": 8.0, "gyroscope_x": 41.0, "gyroscope_y": 10.0, "gyroscope_z": 11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:58.944000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 13.0, "acceleration_z": -6.0, "gyroscope_x": -79.0, "gyroscope_y": 80.0, "gyroscope_z": 44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:58.996000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 265.0, "acceleration_y": 21.0, "acceleration_z": 1.0, "gyroscope_x": -157.0, "gyroscope_y": -68.0, "gyroscope_z": -24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:59.048000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 123.0, "acceleration_z": -81.0, "gyroscope_x": -23.0, "gyroscope_y": 109.0, "gyroscope_z": -86.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:20:59.100000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 225.0, "acceleration_y": 65.0, "acceleration_z": -26.0, "gyroscope_x": -45.0, "gyroscope_y": 85.0, "gyroscope_z": 2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:04.634000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 303.0, "acceleration_y": 43.0, "acceleration_z": -75.0, "gyroscope_x": 242.0, "gyroscope_y": -211.0, "gyroscope_z": 81.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:04.686000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": -37.0, "acceleration_z": 37.0, "gyroscope_x": 310.0, "gyroscope_y": 97.0, "gyroscope_z": 24.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:04.738000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 30.0, "acceleration_z": 41.0, "gyroscope_x": 349.0, "gyroscope_y": 29.0, "gyroscope_z": 23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:04.790000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 229.0, "acceleration_y": 63.0, "acceleration_z": 126.0, "gyroscope_x": -12.0, "gyroscope_y": 221.0, "gyroscope_z": -109.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:10.439000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 317.0, "acceleration_y": 14.0, "acceleration_z": 41.0, "gyroscope_x": -145.0, "gyroscope_y": 35.0, "gyroscope_z": 101.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:10.491000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 222.0, "acceleration_y": 32.0, "acceleration_z": -41.0, "gyroscope_x": 71.0, "gyroscope_y": 66.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:10.543000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": 15.0, "gyroscope_x": -58.0, "gyroscope_y": 143.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:10.595000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 234.0, "acceleration_y": 39.0, "acceleration_z": -117.0, "gyroscope_x": 111.0, "gyroscope_y": -18.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:16.261000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 227.0, "acceleration_y": 34.0, "acceleration_z": -60.0, "gyroscope_x": -105.0, "gyroscope_y": 60.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:16.313000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 231.0, "acceleration_y": -24.0, "acceleration_z": -52.0, "gyroscope_x": -113.0, "gyroscope_y": -99.0, "gyroscope_z": -34.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:16.365000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 235.0, "acceleration_y": 10.0, "acceleration_z": -7.0, "gyroscope_x": -125.0, "gyroscope_y": -87.0, "gyroscope_z": 44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:16.417000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 197.0, "acceleration_y": 33.0, "acceleration_z": -93.0, "gyroscope_x": -190.0, "gyroscope_y": -138.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:22.254000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 207.0, "acceleration_y": 84.0, "acceleration_z": -19.0, "gyroscope_x": 50.0, "gyroscope_y": 191.0, "gyroscope_z": -83.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:22.306000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 230.0, "acceleration_y": 24.0, "acceleration_z": -19.0, "gyroscope_x": 142.0, "gyroscope_y": 54.0, "gyroscope_z": 30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:22.358000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 261.0, "acceleration_y": 59.0, "acceleration_z": -64.0, "gyroscope_x": 78.0, "gyroscope_y": -56.0, "gyroscope_z": 68.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:22.410000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 300.0, "acceleration_y": 80.0, "acceleration_z": 28.0, "gyroscope_x": 11.0, "gyroscope_y": 140.0, "gyroscope_z": -166.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:27.928000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 29.0, "acceleration_z": 0.0, "gyroscope_x": -97.0, "gyroscope_y": 20.0, "gyroscope_z": 22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:27.980000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 247.0, "acceleration_y": 89.0, "acceleration_z": -16.0, "gyroscope_x": -150.0, "gyroscope_y": -75.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:28.032000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 238.0, "acceleration_y": 25.0, "acceleration_z": -52.0, "gyroscope_x": 3.0, "gyroscope_y": -65.0, "gyroscope_z": -56.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:28.084000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 274.0, "acceleration_y": 86.0, "acceleration_z": -42.0, "gyroscope_x": 17.0, "gyroscope_y": -202.0, "gyroscope_z": -76.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:33.674000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 358.0, "acceleration_y": 52.0, "acceleration_z": -10.0, "gyroscope_x": -661.0, "gyroscope_y": -212.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:33.726000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 425.0, "acceleration_y": 147.0, "acceleration_z": 8.0, "gyroscope_x": -634.0, "gyroscope_y": -146.0, "gyroscope_z": -136.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:33.778000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 228.0, "acceleration_y": 5.0, "acceleration_z": -243.0, "gyroscope_x": -381.0, "gyroscope_y": -214.0, "gyroscope_z": -406.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:33.830000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 412.0, "acceleration_y": 162.0, "acceleration_z": -30.0, "gyroscope_x": -411.0, "gyroscope_y": 48.0, "gyroscope_z": -168.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:39.423000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 197.0, "acceleration_y": 16.0, "acceleration_z": -47.0, "gyroscope_x": -31.0, "gyroscope_y": 51.0, "gyroscope_z": -45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:39.475000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 278.0, "acceleration_y": 65.0, "acceleration_z": -41.0, "gyroscope_x": -44.0, "gyroscope_y": 79.0, "gyroscope_z": 18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:39.527000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 6.0, "acceleration_z": -102.0, "gyroscope_x": 43.0, "gyroscope_y": -82.0, "gyroscope_z": -113.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:39.579000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 286.0, "acceleration_y": 10.0, "acceleration_z": -30.0, "gyroscope_x": 33.0, "gyroscope_y": 27.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:45.117000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 24.0, "acceleration_z": -71.0, "gyroscope_x": -10.0, "gyroscope_y": 39.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:45.169000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 25.0, "acceleration_z": -73.0, "gyroscope_x": -7.0, "gyroscope_y": 41.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:45.221000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -72.0, "gyroscope_x": -11.0, "gyroscope_y": 43.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:45.273000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 26.0, "acceleration_z": -74.0, "gyroscope_x": -13.0, "gyroscope_y": 41.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:50.795000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 25.0, "acceleration_z": -74.0, "gyroscope_x": -10.0, "gyroscope_y": 66.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:50.847000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 25.0, "acceleration_z": -78.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:50.899000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 25.0, "acceleration_z": -74.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:50.951000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 26.0, "acceleration_z": -77.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:56.452000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 30.0, "acceleration_z": -62.0, "gyroscope_x": -19.0, "gyroscope_y": 38.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:56.504000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 30.0, "acceleration_z": -63.0, "gyroscope_x": -11.0, "gyroscope_y": 32.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:56.556000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 30.0, "acceleration_z": -66.0, "gyroscope_x": -11.0, "gyroscope_y": 33.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:21:56.608000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 28.0, "acceleration_z": -74.0, "gyroscope_x": -6.0, "gyroscope_y": 35.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:02.137000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 20.0, "acceleration_z": -57.0, "gyroscope_x": -15.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:02.189000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 23.0, "acceleration_z": -61.0, "gyroscope_x": -13.0, "gyroscope_y": 68.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:02.241000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 18.0, "acceleration_z": -58.0, "gyroscope_x": -10.0, "gyroscope_y": 68.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:02.293000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 25.0, "acceleration_z": -52.0, "gyroscope_x": -16.0, "gyroscope_y": 61.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:07.820000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 18.0, "acceleration_z": -85.0, "gyroscope_x": -47.0, "gyroscope_y": 11.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:07.872000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 14.0, "acceleration_z": -79.0, "gyroscope_x": -16.0, "gyroscope_y": 8.0, "gyroscope_z": -14.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:07.924000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 17.0, "acceleration_z": -72.0, "gyroscope_x": -41.0, "gyroscope_y": 4.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:07.976000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 18.0, "acceleration_z": -70.0, "gyroscope_x": -55.0, "gyroscope_y": 9.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:13.604000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 15.0, "acceleration_z": -80.0, "gyroscope_x": 122.0, "gyroscope_y": -10.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:13.656000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 17.0, "acceleration_z": -92.0, "gyroscope_x": 147.0, "gyroscope_y": 4.0, "gyroscope_z": -44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:13.708000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 19.0, "acceleration_z": -101.0, "gyroscope_x": 153.0, "gyroscope_y": 48.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:13.760000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 26.0, "acceleration_z": -87.0, "gyroscope_x": 156.0, "gyroscope_y": 60.0, "gyroscope_z": -62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:19.305000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 27.0, "acceleration_z": -45.0, "gyroscope_x": -5.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:19.357000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 25.0, "acceleration_z": -41.0, "gyroscope_x": -4.0, "gyroscope_y": 65.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:19.409000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -45.0, "gyroscope_x": -9.0, "gyroscope_y": 59.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:19.461000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 27.0, "acceleration_z": -44.0, "gyroscope_x": -12.0, "gyroscope_y": 53.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:28.474000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -48.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:28.526000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 25.0, "acceleration_z": -47.0, "gyroscope_x": -14.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:28.578000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 27.0, "acceleration_z": -48.0, "gyroscope_x": -9.0, "gyroscope_y": 67.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:28.630000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 27.0, "acceleration_z": -46.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:34.327000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 260.0, "acceleration_y": 25.0, "acceleration_z": -48.0, "gyroscope_x": -14.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:34.379000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 27.0, "acceleration_z": -49.0, "gyroscope_x": -12.0, "gyroscope_y": 67.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:34.431000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -48.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:34.483000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -46.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:40.267000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -49.0, "gyroscope_x": -11.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:40.319000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 25.0, "acceleration_z": -49.0, "gyroscope_x": -16.0, "gyroscope_y": 67.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:40.371000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -48.0, "gyroscope_x": -17.0, "gyroscope_y": 63.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:40.423000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 25.0, "acceleration_z": -50.0, "gyroscope_x": -15.0, "gyroscope_y": 62.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:46.338000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 24.0, "acceleration_z": -74.0, "gyroscope_x": -1.0, "gyroscope_y": 31.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:46.390000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 22.0, "acceleration_z": -72.0, "gyroscope_x": 32.0, "gyroscope_y": 44.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:46.442000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 22.0, "acceleration_z": -69.0, "gyroscope_x": 43.0, "gyroscope_y": 43.0, "gyroscope_z": -21.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:46.494000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 25.0, "acceleration_z": -68.0, "gyroscope_x": 53.0, "gyroscope_y": 50.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:52.016000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 17.0, "acceleration_z": -66.0, "gyroscope_x": 129.0, "gyroscope_y": 164.0, "gyroscope_z": -30.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:52.068000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 21.0, "acceleration_z": -60.0, "gyroscope_x": 170.0, "gyroscope_y": 172.0, "gyroscope_z": -31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:52.120000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 28.0, "acceleration_z": -53.0, "gyroscope_x": 180.0, "gyroscope_y": 158.0, "gyroscope_z": -28.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:52.172000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 30.0, "acceleration_z": -55.0, "gyroscope_x": 157.0, "gyroscope_y": 145.0, "gyroscope_z": -29.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:57.711000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 26.0, "acceleration_z": -57.0, "gyroscope_x": -12.0, "gyroscope_y": 67.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:57.763000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 24.0, "acceleration_z": -56.0, "gyroscope_x": -12.0, "gyroscope_y": 68.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:57.815000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -56.0, "gyroscope_x": -14.0, "gyroscope_y": 68.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:22:57.867000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 28.0, "acceleration_z": -57.0, "gyroscope_x": -9.0, "gyroscope_y": 67.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:03.570000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 30.0, "acceleration_z": -105.0, "gyroscope_x": -18.0, "gyroscope_y": -98.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:03.622000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 50.0, "acceleration_z": -94.0, "gyroscope_x": -32.0, "gyroscope_y": -55.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:03.674000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 48.0, "acceleration_z": -107.0, "gyroscope_x": -7.0, "gyroscope_y": -39.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:03.726000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 249.0, "acceleration_y": 37.0, "acceleration_z": -102.0, "gyroscope_x": 0.0, "gyroscope_y": -38.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:09.314000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 31.0, "acceleration_z": -39.0, "gyroscope_x": -9.0, "gyroscope_y": 76.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:09.366000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 26.0, "acceleration_z": -37.0, "gyroscope_x": -11.0, "gyroscope_y": 74.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:09.418000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 27.0, "acceleration_z": -42.0, "gyroscope_x": -7.0, "gyroscope_y": 83.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:09.470000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 27.0, "acceleration_z": -38.0, "gyroscope_x": -10.0, "gyroscope_y": 67.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:14.990000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 25.0, "acceleration_z": -50.0, "gyroscope_x": -18.0, "gyroscope_y": 53.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:15.042000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 26.0, "acceleration_z": -51.0, "gyroscope_x": -17.0, "gyroscope_y": 46.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:15.094000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 28.0, "acceleration_z": -53.0, "gyroscope_x": -13.0, "gyroscope_y": 42.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:15.146000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 24.0, "acceleration_z": -56.0, "gyroscope_x": -8.0, "gyroscope_y": 43.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:20.657000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 331.0, "acceleration_y": 35.0, "acceleration_z": 28.0, "gyroscope_x": -472.0, "gyroscope_y": 24.0, "gyroscope_z": -63.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:20.709000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 187.0, "acceleration_y": 35.0, "acceleration_z": -59.0, "gyroscope_x": -399.0, "gyroscope_y": -53.0, "gyroscope_z": -46.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:20.761000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 226.0, "acceleration_y": 59.0, "acceleration_z": -5.0, "gyroscope_x": -369.0, "gyroscope_y": -69.0, "gyroscope_z": -18.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:20.813000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": 34.0, "acceleration_z": -87.0, "gyroscope_x": -182.0, "gyroscope_y": -20.0, "gyroscope_z": -53.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:26.316000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 237.0, "acceleration_y": 63.0, "acceleration_z": 26.0, "gyroscope_x": -7.0, "gyroscope_y": 76.0, "gyroscope_z": -38.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:26.368000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 511.0, "acceleration_y": 150.0, "acceleration_z": 29.0, "gyroscope_x": -113.0, "gyroscope_y": 127.0, "gyroscope_z": 195.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:26.420000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 270.0, "acceleration_y": -97.0, "acceleration_z": -65.0, "gyroscope_x": -164.0, "gyroscope_y": -29.0, "gyroscope_z": 242.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:26.472000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 95.0, "acceleration_z": -29.0, "gyroscope_x": -118.0, "gyroscope_y": 18.0, "gyroscope_z": 61.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:32.002000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 511.0, "acceleration_y": 175.0, "acceleration_z": -52.0, "gyroscope_x": 540.0, "gyroscope_y": 57.0, "gyroscope_z": 51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:32.054000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 419.0, "acceleration_y": 42.0, "acceleration_z": -131.0, "gyroscope_x": 563.0, "gyroscope_y": 287.0, "gyroscope_z": 73.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:32.106000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 388.0, "acceleration_y": 55.0, "acceleration_z": -114.0, "gyroscope_x": 549.0, "gyroscope_y": 203.0, "gyroscope_z": -111.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:32.158000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 433.0, "acceleration_y": 161.0, "acceleration_z": 20.0, "gyroscope_x": 603.0, "gyroscope_y": 584.0, "gyroscope_z": 44.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:37.979000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 243.0, "acceleration_y": 31.0, "acceleration_z": 75.0, "gyroscope_x": -90.0, "gyroscope_y": 55.0, "gyroscope_z": 69.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:38.031000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 285.0, "acceleration_y": 49.0, "acceleration_z": -17.0, "gyroscope_x": -212.0, "gyroscope_y": 136.0, "gyroscope_z": -33.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:38.083000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": -2.0, "acceleration_z": -25.0, "gyroscope_x": -136.0, "gyroscope_y": 82.0, "gyroscope_z": -107.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:38.135000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 177.0, "acceleration_y": 56.0, "acceleration_z": 25.0, "gyroscope_x": -203.0, "gyroscope_y": 129.0, "gyroscope_z": -126.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:43.681000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 37.0, "acceleration_z": -53.0, "gyroscope_x": -3.0, "gyroscope_y": 9.0, "gyroscope_z": -16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:43.733000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 40.0, "acceleration_z": -61.0, "gyroscope_x": -1.0, "gyroscope_y": 12.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:43.785000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 45.0, "acceleration_z": -49.0, "gyroscope_x": -13.0, "gyroscope_y": 7.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:43.837000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 257.0, "acceleration_y": 48.0, "acceleration_z": -51.0, "gyroscope_x": -6.0, "gyroscope_y": -8.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:49.358000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 42.0, "acceleration_z": -44.0, "gyroscope_x": -9.0, "gyroscope_y": 64.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:49.410000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 44.0, "acceleration_z": -48.0, "gyroscope_x": -14.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:49.462000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 42.0, "acceleration_z": -41.0, "gyroscope_x": -18.0, "gyroscope_y": 68.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:49.514000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 45.0, "acceleration_z": -44.0, "gyroscope_x": -12.0, "gyroscope_y": 62.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:55.068000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 53.0, "acceleration_z": -54.0, "gyroscope_x": -30.0, "gyroscope_y": 44.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:55.120000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 258.0, "acceleration_y": 58.0, "acceleration_z": -45.0, "gyroscope_x": -47.0, "gyroscope_y": 30.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:55.172000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 49.0, "acceleration_z": -52.0, "gyroscope_x": -61.0, "gyroscope_y": 32.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:23:55.224000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 248.0, "acceleration_y": 55.0, "acceleration_z": -49.0, "gyroscope_x": -57.0, "gyroscope_y": -7.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:00.934000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 49.0, "acceleration_z": -68.0, "gyroscope_x": -4.0, "gyroscope_y": 80.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:00.986000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 40.0, "acceleration_z": -87.0, "gyroscope_x": -17.0, "gyroscope_y": 131.0, "gyroscope_z": 6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:01.038000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 256.0, "acceleration_y": 39.0, "acceleration_z": -78.0, "gyroscope_x": -12.0, "gyroscope_y": 161.0, "gyroscope_z": 8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:01.090000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 34.0, "acceleration_z": -65.0, "gyroscope_x": -32.0, "gyroscope_y": 199.0, "gyroscope_z": 4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:06.633000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -79.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:06.685000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 45.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:06.737000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 45.0, "acceleration_z": -77.0, "gyroscope_x": -16.0, "gyroscope_y": 60.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:06.789000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 46.0, "acceleration_z": -78.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:12.320000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 42.0, "acceleration_z": -82.0, "gyroscope_x": -13.0, "gyroscope_y": 78.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:12.372000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 44.0, "acceleration_z": -79.0, "gyroscope_x": -13.0, "gyroscope_y": 128.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:12.424000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 268.0, "acceleration_y": 41.0, "acceleration_z": -80.0, "gyroscope_x": -21.0, "gyroscope_y": 73.0, "gyroscope_z": -79.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:12.476000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 242.0, "acceleration_y": 47.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 79.0, "gyroscope_z": 1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:17.989000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 45.0, "acceleration_z": -79.0, "gyroscope_x": -14.0, "gyroscope_y": 66.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:18.041000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -77.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:18.093000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:18.145000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 43.0, "acceleration_z": -77.0, "gyroscope_x": -17.0, "gyroscope_y": 65.0, "gyroscope_z": -12.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:23.679000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:23.731000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -78.0, "gyroscope_x": -16.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:23.783000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -17.0, "gyroscope_y": 64.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:23.835000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 45.0, "acceleration_z": -79.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:29.348000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 33.0, "acceleration_z": -107.0, "gyroscope_x": -27.0, "gyroscope_y": 77.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:29.400000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 42.0, "acceleration_z": -109.0, "gyroscope_x": 33.0, "gyroscope_y": 95.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:29.452000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 47.0, "acceleration_z": -93.0, "gyroscope_x": 46.0, "gyroscope_y": 116.0, "gyroscope_z": -15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:29.504000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 40.0, "acceleration_z": -71.0, "gyroscope_x": 42.0, "gyroscope_y": 102.0, "gyroscope_z": -23.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:35.153000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 45.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:35.205000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:35.257000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": 0.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:35.309000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -13.0, "gyroscope_y": 65.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:40.809000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -13.0, "gyroscope_y": 63.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:40.861000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 44.0, "acceleration_z": -76.0, "gyroscope_x": -14.0, "gyroscope_y": 62.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:40.913000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:40.965000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:46.481000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:46.533000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:46.585000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -78.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:46.637000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 43.0, "acceleration_z": -77.0, "gyroscope_x": -12.0, "gyroscope_y": 65.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:52.341000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:52.393000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 45.0, "acceleration_z": -77.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:52.445000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -79.0, "gyroscope_x": -15.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:52.497000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:58.060000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 45.0, "acceleration_z": -77.0, "gyroscope_x": -13.0, "gyroscope_y": 60.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:58.112000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 42.0, "acceleration_z": -80.0, "gyroscope_x": -14.0, "gyroscope_y": 62.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:58.164000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 45.0, "acceleration_z": -80.0, "gyroscope_x": -14.0, "gyroscope_y": 66.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:24:58.216000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 44.0, "acceleration_z": -76.0, "gyroscope_x": -17.0, "gyroscope_y": 67.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:03.901000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -12.0, "gyroscope_y": 63.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:03.953000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -17.0, "gyroscope_y": 66.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:04.005000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -77.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:04.057000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -76.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:09.691000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -11.0, "gyroscope_y": 65.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:09.743000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:09.795000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:09.847000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -79.0, "gyroscope_x": -16.0, "gyroscope_y": 62.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:15.472000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 45.0, "acceleration_z": -77.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:15.524000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -16.0, "gyroscope_y": 65.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:15.576000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -78.0, "gyroscope_x": -13.0, "gyroscope_y": 65.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:15.628000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:21.139000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -13.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:21.191000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -78.0, "gyroscope_x": -16.0, "gyroscope_y": 68.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:21.243000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 43.0, "acceleration_z": -78.0, "gyroscope_x": -15.0, "gyroscope_y": 66.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:21.295000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -15.0, "gyroscope_y": 65.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:26.813000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:26.865000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 251.0, "acceleration_y": 44.0, "acceleration_z": -77.0, "gyroscope_x": -15.0, "gyroscope_y": 67.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:26.917000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 43.0, "acceleration_z": -76.0, "gyroscope_x": -16.0, "gyroscope_y": 67.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:25:26.969000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 44.0, "acceleration_z": -78.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 19:52:48.872000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 29.0, "acceleration_z": -71.0, "gyroscope_x": -9.0, "gyroscope_y": 90.0, "gyroscope_z": -60.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:38.465000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 245.0, "acceleration_y": 29.0, "acceleration_z": -75.0, "gyroscope_x": -20.0, "gyroscope_y": 70.0, "gyroscope_z": 9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:43.635000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 267.0, "acceleration_y": 26.0, "acceleration_z": -65.0, "gyroscope_x": 130.0, "gyroscope_y": 136.0, "gyroscope_z": 10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:43.687000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 30.0, "acceleration_z": -28.0, "gyroscope_x": 147.0, "gyroscope_y": 97.0, "gyroscope_z": -17.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:43.739000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 264.0, "acceleration_y": 27.0, "acceleration_z": -31.0, "gyroscope_x": 110.0, "gyroscope_y": 49.0, "gyroscope_z": 16.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:43.791000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 271.0, "acceleration_y": 30.0, "acceleration_z": -80.0, "gyroscope_x": 158.0, "gyroscope_y": 315.0, "gyroscope_z": -25.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:49.459000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 239.0, "acceleration_y": 68.0, "acceleration_z": 1.0, "gyroscope_x": 105.0, "gyroscope_y": 241.0, "gyroscope_z": 8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:49.511000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 240.0, "acceleration_y": 75.0, "acceleration_z": -48.0, "gyroscope_x": 8.0, "gyroscope_y": 250.0, "gyroscope_z": 3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:49.563000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 262.0, "acceleration_y": 27.0, "acceleration_z": 8.0, "gyroscope_x": -28.0, "gyroscope_y": 215.0, "gyroscope_z": 31.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:49.615000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 311.0, "acceleration_y": 17.0, "acceleration_z": 26.0, "gyroscope_x": -163.0, "gyroscope_y": 249.0, "gyroscope_z": 83.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:55.129000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 279.0, "acceleration_y": 84.0, "acceleration_z": -107.0, "gyroscope_x": -12.0, "gyroscope_y": 188.0, "gyroscope_z": 72.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:55.181000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 215.0, "acceleration_y": 10.0, "acceleration_z": 47.0, "gyroscope_x": -36.0, "gyroscope_y": 101.0, "gyroscope_z": -70.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:55.233000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 299.0, "acceleration_y": 11.0, "acceleration_z": -32.0, "gyroscope_x": -63.0, "gyroscope_y": 343.0, "gyroscope_z": 225.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:06:55.285000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 259.0, "acceleration_y": 36.0, "acceleration_z": -62.0, "gyroscope_x": -106.0, "gyroscope_y": 304.0, "gyroscope_z": 15.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:07:01.130000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 241.0, "acceleration_y": 112.0, "acceleration_z": -103.0, "gyroscope_x": -311.0, "gyroscope_y": 36.0, "gyroscope_z": -1.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:07:01.182000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 273.0, "acceleration_y": 149.0, "acceleration_z": -105.0, "gyroscope_x": -248.0, "gyroscope_y": 173.0, "gyroscope_z": 45.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:07:01.234000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 210.0, "acceleration_y": 125.0, "acceleration_z": -165.0, "gyroscope_x": -132.0, "gyroscope_y": 316.0, "gyroscope_z": 62.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:07:01.286000", "trip_id": 0, "vibration": 1, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 280.0, "acceleration_y": 135.0, "acceleration_z": -102.0, "gyroscope_x": -65.0, "gyroscope_y": 74.0, "gyroscope_z": -13.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:04.603000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 246.0, "acceleration_y": 24.0, "acceleration_z": -67.0, "gyroscope_x": -12.0, "gyroscope_y": 114.0, "gyroscope_z": -22.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:09.776000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -81.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:09.828000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -84.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:09.880000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -83.0, "gyroscope_x": -13.0, "gyroscope_y": 67.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:09.932000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 23.0, "acceleration_z": -82.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:15.653000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 25.0, "acceleration_z": -82.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -3.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:15.705000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": -14.0, "gyroscope_y": 63.0, "gyroscope_z": -6.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:15.757000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 25.0, "acceleration_z": -81.0, "gyroscope_x": -14.0, "gyroscope_y": 65.0, "gyroscope_z": -7.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:15.809000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 24.0, "acceleration_z": -83.0, "gyroscope_x": -13.0, "gyroscope_y": 62.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:21.404000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -81.0, "gyroscope_x": -15.0, "gyroscope_y": 64.0, "gyroscope_z": -5.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:21.456000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": -14.0, "gyroscope_y": 67.0, "gyroscope_z": -10.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:21.508000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 23.0, "acceleration_z": -83.0, "gyroscope_x": -12.0, "gyroscope_y": 64.0, "gyroscope_z": -11.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:21.560000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": -12.0, "gyroscope_y": 66.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:27.165000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": -14.0, "gyroscope_y": 62.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:27.217000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 25.0, "acceleration_z": -82.0, "gyroscope_x": -13.0, "gyroscope_y": 64.0, "gyroscope_z": -9.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:27.269000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -82.0, "gyroscope_x": -16.0, "gyroscope_y": 64.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:27.321000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 25.0, "acceleration_z": -82.0, "gyroscope_x": -16.0, "gyroscope_y": 62.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:33.003000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 23.0, "acceleration_z": -82.0, "gyroscope_x": -16.0, "gyroscope_y": 66.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:33.055000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 25.0, "acceleration_z": -82.0, "gyroscope_x": -14.0, "gyroscope_y": 64.0, "gyroscope_z": -2.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:33.107000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -83.0, "gyroscope_x": -14.0, "gyroscope_y": 68.0, "gyroscope_z": -4.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:33.159000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 253.0, "acceleration_y": 24.0, "acceleration_z": -81.0, "gyroscope_x": -16.0, "gyroscope_y": 63.0, "gyroscope_z": -8.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:38.642000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 252.0, "acceleration_y": 37.0, "acceleration_z": -82.0, "gyroscope_x": -30.0, "gyroscope_y": 34.0, "gyroscope_z": -51.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:38.694000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 250.0, "acceleration_y": 34.0, "acceleration_z": -82.0, "gyroscope_x": -35.0, "gyroscope_y": 6.0, "gyroscope_z": -54.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:38.746000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 255.0, "acceleration_y": 34.0, "acceleration_z": -86.0, "gyroscope_x": -31.0, "gyroscope_y": 13.0, "gyroscope_z": -42.0, "crash": null, "terrain": null}, {"time": "2023-11-06 21:08:38.798000", "trip_id": 0, "vibration": 0, "latitude": 0.0, "longitude": 0.0, "acceleration_x": 254.0, "acceleration_y": 34.0, "acceleration_z": -101.0, "gyroscope_x": -23.0, "gyroscope_y": 118.0, "gyroscope_z": 40.0, "crash": null, "terrain": null}]'

In [4]:
FAKE_DATE = pd.to_datetime("1.1.2000")

LR = 1e-2
MOMENTUM = 0.9
NUM_EPOCHS = 10
HIDDEN_LAYERS = 128
INPUT_SIZE = 8

NORMALIZE_SIZE = True

pavement_start = pd.Timestamp(year=2023, month=11, day=6, hour=18, minute=34)
pavement_end = pd.Timestamp(year=2023, month=11, day=6, hour=18, minute=54)
asphalt_start_1 = pd.Timestamp(year=2023, month=11, day=6, hour=19, minute=10)
asphalt_end_1 = pd.Timestamp(year=2023, month=11, day=6, hour=19, minute=16)
asphalt_start_2 = pd.Timestamp(year=2023, month=11, day=6, hour=19, minute=31)
asphalt_end_2 = pd.Timestamp(year=2023, month=11, day=6, hour=19, minute=50)


def data_preparation(data):
    df = pd.read_json(data)

    df['time'] = pd.to_datetime(df['time'], format='mixed')
    for i, row in df.iterrows():
        if (row.time >= pavement_start) and (row.time <= pavement_end):
            df.at[i, 'terrain'] = config.map_to_int('pavement')
        elif (row.time >= asphalt_start_1) and (row.time <= asphalt_end_1):
            df.at[i, 'terrain'] = config.map_to_int('asphalt')
        elif (row.time >= asphalt_start_2) and (row.time <= asphalt_end_2):
            df.at[i, 'terrain'] = config.map_to_int('asphalt')

    df.dropna(subset=['terrain'], inplace=True)

    df['time_second'] = df.time.map(lambda x: pd.Timestamp(x).floor(freq='S'))
    df['time'] = df.time.map(pd.Timestamp.timestamp)

    grouped = df.groupby([df.trip_id, df.time_second])  # grouped.get_group(1)
    x = []
    y = []
    for i, (trip_seconds, table) in enumerate(grouped):
        if (i + 1) % 100 == 0:
            print("# trip seconds: " + str(i + 1))

        train_input = table.drop(columns=['terrain', 'trip_id', 'crash', 'time_second', 'latitude', 'longitude'])
        n_cols = len(train_input.columns)

        train_input = train_input.to_numpy()

        if NORMALIZE_SIZE:
            n_missing_rows = 20 - len(train_input)
            for _ in range(n_missing_rows):
                fake_array = np.array([None] * n_cols)
                np.append(train_input, fake_array)
            train_input = np.ndarray.flatten(train_input)

        train_target = table.terrain.min()

        x.append(train_input)
        y.append(train_target)

    return x, np.array(y)


# raw_data = dataprocessing.get_data_db()
X, Y = data_preparation(raw_data)

/var/folders/q4/b_qkk2qx16zdglt42g7c0dlr0000gn/T/ipykernel_28982/2884414884.py:20: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data)


# trip seconds: 100
# trip seconds: 200


### Training the Model

In [5]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [6]:
def to_loader(x, y):
    return list(zip(x, y))

In [7]:
import torch.nn.functional as F

def gen_dataloader(x, y, test_size=0.2, random_state=0):
    scaler = MinMaxScaler()  # TODO: choose scaler
    # scaler.fit(flatten(x))  # TODO: transform data without trip_id

    # x = torch.tensor(x)
    # y = torch.tensor(y)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size, random_state=random_state)
    
    train_y = torch.from_numpy(train_y).long()
    test_y = torch.from_numpy(test_y).long()
    
    train_y = F.one_hot(train_y, len(CLASSES)).to(torch.float32)
    test_y = F.one_hot(test_y, len(CLASSES)).to(torch.float32)

    return to_loader(train_x, train_y), to_loader(test_x, test_y), scaler


train_loader, test_loader, scaler = gen_dataloader(X, Y)  # with default values

In [8]:
from torch.autograd import Variable
def train_model(model, train_loader, scaler, num_epochs=NUM_EPOCHS, lr=LR, momentum=MOMENTUM, device=None):
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.MSELoss()
    # criterion = nn.NLLLoss()

    # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    # optimizer = optim.Adam(model.parameters(), lr=lr)

    print("Training STARTED")
    # model.to(device)

    for e in range(0, num_epochs):
        model.train()  # set the model in training mode
        total_train_loss = 0  # initialize the total training and validation loss

        for i, (training_input, target) in enumerate(train_loader):  # loop over the training set
            hidden = model.initHidden()
            model.zero_grad()

            for data_row in training_input:
                input = torch.tensor([data_row]).float()
                output, hidden = model(input, hidden)

            print(output[0])
            print(target)
            loss = criterion(torch.tensor(output[0]), target)
            loss = Variable(loss, requires_grad=True)
            loss.backward()

            # for p in model.parameters():
            #     p.data.add_(p.grad.data, alpha=-lr)

            # optimizer.step()
            # add the loss to the total training loss so far and calculate the number of correct predictions
            total_train_loss += loss.item()

        if (e + 1) % 10 == 0 | e == 0:
            print("Epoch", e, "Training Loss:", total_train_loss.item())

        print("Training FINISHED")
        
    optimizer = "there is no optimizer because our model is best"

    return model, criterion, optimizer


model = sf.RNN(INPUT_SIZE, HIDDEN_LAYERS, len(CLASSES))
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")  # for testing on my Mac
model, criterion, optimizer = train_model(model=model, train_loader=train_loader, scaler=scaler)


/var/folders/q4/b_qkk2qx16zdglt42g7c0dlr0000gn/T/ipykernel_28982/401895380.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  input = torch.tensor([data_row]).float()
/var/folders/q4/b_qkk2qx16zdglt42g7c0dlr0000gn/T/ipykernel_28982/401895380.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(torch.tensor(output[0]), target)


Training STARTED
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.5994e+08, -1.2070e+08, -1.3961e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1.7091e+08, -1.1833e+08, -1.5175e+08],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.])
tensor([ 0.0000e+00, -1

## Testing

In [9]:
def print_training_accuracy(model, train_loader, criterion, classes, device=None):
    training_loss, class_correct, class_total = util.compute_accuracy(model, train_loader, device, criterion)

    # average training loss
    training_loss = training_loss / len(train_loader.dataset)
    print('Training Loss: {:.6f}\n'.format(training_loss))
    for i in range(10):
        if class_total[i] > 0:
            print('Training Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100.0 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Training Accuracy of %5s: N/A ' % (classes[i]))

    print('\Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))


print_training_accuracy(model, train_loader, criterion, CLASSES)

TypeError: forward() missing 1 required positional argument: 'hidden'

In [ ]:
def print_testing_accuracy(model, test_loader, device, classes, criterion=None):
    test_loss, class_correct, class_total = util.compute_accuracy(model, test_loader, device, criterion)

    # average test loss
    test_loss = test_loss / len(test_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100.0 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))


print_testing_accuracy(model, test_loader, criterion, CLASSES)